In [1]:
import graphviz;from graphviz import Digraph;import math;import multiprocessing
import random;from joblib import Parallel, delayed;from collections import ChainMap;from matplotlib.patches import Patch
num_cores = 5;import pandas as pd;import numpy as np
import seaborn as sb;import matplotlib.pyplot as plt;import matplotlib as mpl;import glob;import os
from matplotlib_venn import venn2;import gzip;import shutil;from tqdm import tqdm_notebook as tq;import warnings
import scipy.stats as stats;import pdb;import gseapy as GSEA;import gc;import networkx as nx
warnings.simplefilter('ignore');cmapSpecial = mpl.colors.LinearSegmentedColormap.from_list("", ["blue",'white',"red"])
cmapSpecial2 = mpl.colors.LinearSegmentedColormap.from_list("", ["red",'white',"blue"])
cmapSpecial3 = mpl.colors.LinearSegmentedColormap.from_list("", ['white',"red"])
cmapSpecial4 = mpl.colors.LinearSegmentedColormap.from_list("", [(0,'black'),(0.4999,'grey'),(0.5,"blue"),
                                                                 (0.75,'white'),(1,"red")],N=256)
__debug=False;mpl.rcParams['figure.dpi'] = 300;plt.rcParams["figure.facecolor"] = "w";import time

C:\Users\USER\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
def __removeLstFunc(lst,rmLst,isLst=False):
    out=[]
    if not(isLst):rmLst = [rmLst]
    for i in lst:
        if not(i in rmLst):out.append(i)
    return out
def readLst(file):
    """
    readLst:
        Input:
            file, name of the file you want to get a list from
        Actions: 
            Reads through the given file, turing each line into an element of a list
        Returns: 
            List of the files contents
    """
    out=[]
    with open(file) as f:
        for line in f:out += line.split()
    return out
def saveLst(lst,file):
    """
    saveLst:
        Input:
            lst, a list
            file, name of the file to create and save output to
        Actions: 
            Each element in the given list, is saved as a line in the file 
        Returns: 
            The name of the created file
    """
    with open(file,'w') as f:
        for num in range(len(lst)-1):
            elem = str(lst[num]);f.write(elem+'\n')
        elem = str(lst[len(lst)-1]);f.write(elem)
    return file
def __renameTPM(tpm,num=3,inplace=True):
    if(inplace):df = tpm
    else:df = tpm.copy()
    colLst=[]
    for col in df.columns:
        col = '.'.join(col.split('.')[0:num])
        if(col in colLst):col = col+'.1'
        colLst.append(col)
    df.columns=colLst;noDups=[]
    for i in df.columns:
        if not('.1' in i):noDups.append(i)
    df = df[noDups]
    return df
def __renameLst(lst,num=3):
    colLst=[]
    for col in lst:
        col = '.'.join(col.split('.')[0:num])
        if(col in colLst):col = col+'.1'
        colLst.append(col)
    lst=colLst
    return lst
def __rmvTCGA(lst,num=2):
    outLst=[]
    for i in __renameLst(lst):
        j = '.'.join(i.split('.')[0:num]);k = i.split('.')[num];k = k.replace('TCGA','');j += '.'+k;outLst.append(j)
    return outLst
def __repeatAppend(lst,num,elem):
    for i in range(num):lst.append(elem)
    return lst
def __colDataFunc(hot,cold,hotName='Hot',coldName='Cold',groupName='Group'):
    names=[];colData=[]
    for i in hot:
        if not(i in names):
            names.append(i);colData.append(hotName)
    for i in cold:
        if not(i in names):
            names.append(i);colData.append(coldName)
    test = pd.DataFrame(colData,names,columns=[groupName])
    return test

__kobePathways = pd.read_excel('Genelist_desert.xlsx',index_col=1);__msigDict={};__msigDict['T-effector'] = []
__msigDict['IFNG induced'] = ['HALLMARK_INTERFERON_GAMMA_RESPONSE','BIOCARTA_IFNG_PATHWAY','PID_IFNG_PATHWAY',
                            'REACTOME_REGULATION_OF_IFNG_SIGNALING']
__msigDict['Antigen processing machinery'] = ['KEGG_ANTIGEN_PROCESSING_AND_PRESENTATION',
                                            'REACTOME_ANTIGEN_PROCESSING_CROSS_PRESENTATION']
__msigDict['GABA'] = ['BIOCARTA_GABA_PATHWAY','REACTOME_GABA_RECEPTOR_ACTIVATION']
__msigDict['Wnt'] = ['HALLMARK_WNT_BETA_CATENIN_SIGNALING','BIOCARTA_WNT_PATHWAY','KEGG_WNT_SIGNALING_PATHWAY',
                   'PID_WNT_SIGNALING_PATHWAY','REACTOME_SIGNALING_BY_WNT_IN_CANCER']
__msigDict['Fatty acid'] = ['HALLMARK_FATTY_ACID_METABOLISM','KEGG_FATTY_ACID_METABOLISM','REACTOME_FATTY_ACYL_COA_BIOSYNTHESIS']
__msigDict['Cholesterol synthesis'] = ['HALLMARK_CHOLESTEROL_HOMEOSTASIS','REACTOME_CHOLESTEROL_BIOSYNTHESIS']
__msigDict['PI3K/AKT'] = ['HALLMARK_PI3K_AKT_MTOR_SIGNALING','BIOCARTA_AKT_PATHWAY','PID_PI3KCI_AKT_PATHWAY',
                        'REACTOME_PI3K_AKT_SIGNALING_IN_CANCER']
__msigDict['MTOR'] = ['HALLMARK_MTORC1_SIGNALING','BIOCARTA_MTOR_PATHWAY','KEGG_MTOR_SIGNALING_PATHWAY','PID_MTOR_4PATHWAY',
                   'REACTOME_MTOR_SIGNALLING']

In [3]:
import requests;__tpyDict={}
for __path in __msigDict:
    __pathLst = __msigDict[__path];__tpyLst=[]
    for __name in __pathLst:
        __tpy = __name.split('_')[0]
        __url='https://www.gsea-msigdb.org/gsea/msigdb/download_geneset.jsp?geneSetName='+__name+'&fileType=txt'
        __r = requests.get(__url);__test = str(__r.content);__test = __test.split('\\n')[2:]
        __test[len(__test)-1] = __test[len(__test)-1].replace("'","");__tpyLst.append((__tpy,__test))
    __kobeLst = list(__kobePathways.loc[__path]['genes']);__tpyLst.append(('KOBE',__kobeLst));__tpyDict[__path] = __tpyLst
overAllDict={}
for __path in __tpyDict:
    __pathDict={};__pathLst = __tpyDict[__path];__hallLst=[];__bioLst=[];__keggLst=[];__reactLst=[];__pidLst=[];__kobeLst=[]
    for __group in __pathLst:
        __name = __group[0];__lst = __group[1]
        if(__name == 'HALLMARK'):__hallLst += __lst
        if(__name == 'BIOCARTA'):__bioLst += __lst
        if(__name == 'KEGG'):__keggLst += __lst
        if(__name == 'REACTOME'):__reactLst += __lst
        if(__name =='PID'):__pidLst += __lst
        if(__name =='KOBE'):__kobeLst += __lst
    __pathDict['HALLMARK'] = __hallLst
    if(__path =='PI3K/AKT'):__path = 'PI3K-AKT'
    __pathDict['KEGG'] = __keggLst;__pathDict['REACTOME'] = __reactLst;__pathDict['KOBE'] = __kobeLst
    overAllDict[__path] = __pathDict

In [4]:
def __runGenes(dic):
    out=[];final=[]
    for path in dic:
        lst = dic[path];out += lst
    for gene in out:
        if not(gene in final):final.append(gene)
    return final
def __getPaths(lst,dic,path):
    __pathLst = dic[path];heatLst=[]
    for gene in lst:
        if(gene in __pathLst):heatLst.append('Hot')
        else:heatLst.append('Cold')
    df=pd.DataFrame(heatLst,lst,columns=['Group'])
    return df
def __getRowColors(path,color,col='Group'):
    row_palette = dict(zip(['Hot','Cold'], [color,"white"]));row_colors = path[col].map(row_palette)
    return row_colors
# __goodGenes: takes 2 lists and outputs all elements they have in common
def __goodGenes(lst1,lst2):
    out = [value for value in lst1 if value in lst2] 
    return out

def __reOrderLst(genes,path):
    reOrder = readLst('ClusterOrder/'+path+'RowCorrelationReorder.txt');orderLst=[];length = len(genes)
    for idx in reOrder:
        idx = int(idx)
        if(idx < length):
            g = genes[idx];orderLst.append(g)
    return orderLst
def __numCompare(gene,df,compare,num=0.35):
    x = 0
    for g in df.columns:
        value = df.loc[gene][g]
        if(compare=='Up'):
            if(value > num):x += 1
        if(compare=='Down'):
            if(value < num):x += 1
    return x
def __getPurityLst(cor,threshold,compare='Up',num=0.35,percent=False):
    outLst=[]
    for g in cor.index:
        numAbove = __numCompare(g,cor,compare,num)
        if(percent):
            if not(len(cor.index)==0):numAbove = numAbove/len(cor.index)
            else:numAbove = 0
        if(numAbove > threshold):outLst.append(g)
    return outLst
def clusterHeatMap(out,colData,path,title='',clusterRow=True,clusterCol=True,show=True,vmin=-1,vmax=1):
    genes = out.index;HallColors = __getRowColors(__getPaths(genes,overAllDict[path],'HALLMARK'),'orange')
    KeggColors = __getRowColors(__getPaths(genes,overAllDict[path],'KEGG'),'blue')
    ReactomeColors = __getRowColors(__getPaths(genes,overAllDict[path],'REACTOME'),'green')
    BiocartaColors = __getRowColors(__getPaths(genes,overAllDict[path],'BIOCARTA'),'purple')
    PidColors = __getRowColors(__getPaths(genes,overAllDict[path],'PID'),'red')
    KobeColors = __getRowColors(__getPaths(genes,overAllDict[path],'KOBE'),'black')
    row_colors=[HallColors,KeggColors,ReactomeColors,BiocartaColors,PidColors,KobeColors]
    if(colData is not None):
        col_palette = dict(zip(colData.Group.unique(), ["Red","Blue",'White'])) ;col_colors1 = colData.Group.map(col_palette)
    else:
        col_colors1 = None
    cg = sb.clustermap(out,cmap=cmapSpecial,row_cluster=clusterRow,standard_scale=None,col_cluster=clusterCol,
                       row_colors=row_colors,col_colors=col_colors1,vmin=vmin,vmax=vmax,yticklabels=1,xticklabels=1)
    c = ['orange','blue','green','purple','red','black']
    for x,y in zip(["Hallmark", "KEGG", "Reactome", "Biocarta", "Pid", "Kobe"], c):
        cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=10, solid_capstyle="butt")
    cg.ax_heatmap.legend(loc=(0,1), title="Gene List")
    if(show):
        plt.title(title);plt.show();print(title);return title
    else:return row
def __pureLst(df):
    purity = pd.read_csv('TCGA_Purity_Ploidy.txt',index_col=0,sep='\t');sampleLst=[];pureL=[]
    for i in df.columns:
        if(i in purity.index):
            sampleLst.append(i);pure = purity.loc[i]['purity'];pureL.append(pure)
    return sampleLst, pureL
def __pureDf(dfIn,zscore=False):
    df = dfIn.copy();__renameTPM(df);sampleLst,pLst = __pureLst(df)
    if(zscore):pLst = list(stats.zscore(pLst))
    df2 = df[sampleLst];just__pureDf = pd.DataFrame([pLst],['Purity Scores'], columns=sampleLst)
    pDf = just__pureDf.append(df2);pDf.index.name = 'Genes';return pDf
def __withinThres(y,threshold=1):
    num = 0;total = len(y)
    for val in y:
        absVal = abs(val)
        if(absVal<=threshold):num += 1
    percent = num/total;return percent
def __skinyGene(gene,threshold=1,skinyPercent=0.9):
    percent = __withinThres(gene,threshold)
    if(percent >= skinyPercent):return True
    return False
def __PurityCorrection(df,save=False,fileName='',place='',correctNeg=False,thres=1,
                       percent=0.9,skiny=False,shiftZeros=False,scale=0.2):
    pDf = __pureDf(df);pDf.dropna(inplace=True);out = pd.DataFrame();x = pDf.loc['Purity Scores'];x2 = x.copy()
    for gene in tq(pDf.index,desc='Correcting Genes for '+place):
        if (not gene=='Purity Scores'):
            y = pDf.loc[gene];y2 = y.copy()
            if(shiftZeros):
                minArea = y.min() + 0.4;y2 = y[y > minArea];x2 = x[y2.index];y3 = y[y <= minArea];maxZero = y3.max()
            residual = y.copy()
            if(not __skinyGene(y,thres,percent)):
                try:m, b = np.polyfit(x2, y2, 1)
                except:m, b = np.polyfit(x2, y2, 1)
                if(m>0 or correctNeg):
                    regLine = m*x + b;residual = y - regLine
                    if(shiftZeros):
                        y2Res = residual.loc[y2.index];curVal = y2Res.min();lst=[]
                        for i in y.index:
                            if(i in y2.index):lst.append(residual[i])
                            else:
                                dif = abs(maxZero-y3[i]);newVal=dif*scale;newVal=curVal-newVal;lst.append(newVal)
                        data = np.array(lst);residual = pd.Series(data, index = y.index)
            out[gene] = residual
    out.index=pDf.columns;out = out.T
    if(save):out.to_csv(fileName+'_tpm.tsv',sep='\t')
    return out
def masterPurityCorrection(placeLst,skipLst=[],save=True,correctNeg=False,onlyDict=False,dictIn={},
                           correctSkinny=False,fixOut=True,thres=1,percent=0.9,shiftZeros=True,scale=0.2):
    """masterPurityCorrection:
                                Input:
                                    placeLst, a list of cancer areas to correct
                                        no default
                                    skipLst, a list of cancer areas that are already corrected and can be skipped,
                                             though they arn't skiped in sparse removal
                                        defualt = []
                                    save, boolean telling if to save the data (True) or not (False, but its a big waste of time for nothing)
                                        default = True
                                    correctNeg, boolean telling if to correct genes negatively correlated with purity (True)
                                                or leave them as is (False)
                                        default = False
                                    correctSkinny, boolean telling if to correct low variance genes, based on thres and percent (True)
                                                   or leave them as is (False)
                                        default = False
                                    fixOut, boolean telling if to correct outliers (True)
                                            or leave them as is (False)
                                        default = False
                                    thres, float telling the STD threshold a certain number (defined by percent) that a gene's samples
                                           must be below to be considered "skinny"/low variance
                                        default = 1
                                    percent, float between 0 and 1 telling the percentage of a gene's samples that need to be 
                                             below a certain STD value (defned by thres) in order to be considered "skinny"/low variance
                                        default = 0.9
                                Actions:
                                        1) Removes Sparse genes from all data
                                        2) Corrects the given cancer areas data to be post-purity (eliminating correlation with purity)
                                           with the correction being based on the parameters given
                                        3) For each cancer area, saves the newly corrected data in the folder matching the correction type
                                Returns:
                                        On success return 1
    """
    for place in tq(placeLst,desc='Overall'):
        if not(place in skipLst):
            if(fixOut):prePure = pd.read_csv('Purity/TPM/Outliers Fixed/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
            else:
                prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
                __renameTPM(prePure);prePure = np.log2(prePure+1);prePure = __removeBadPlace(prePure)
                prePure = pd.DataFrame(stats.zscore(prePure,axis=1),prePure.index,columns=prePure.columns)
            if(onlyDict):
                lst = __getSuperLst(dictIn);lst = __goodGenes(lst,prePure.index);prePure = prePure.loc[lst]
            if(len(prePure.columns)>0):
                if not(correctNeg):fileName='Purity/TPM/v1/TCGA_'+place
                else:
                    if not(correctSkinny):
                        if(fixOut):
                            if(shiftZeros):fileName='Purity/TPM/v7/TCGA_'+place
                            else:fileName='Purity/TPM/v8/TCGA_'+place
                        else:fileName='Purity/TPM/v2/TCGA_'+place
                    else:
                        if(fixOut):
                            if(shiftZeros):fileName='Purity/TPM/v7.2/TCGA_'+place
                            else:fileName='Purity/TPM/v8.2/TCGA_'+place
                        else:fileName='Purity/TPM/v3/TCGA_'+place
                out = __PurityCorrection(prePure,save=save,place=place,fileName=fileName,
                                         correctNeg=correctNeg,thres=thres,percent=percent,
                                         skiny=correctSkinny,shiftZeros=shiftZeros,scale=scale)
    return 1
def __removeSparse(placeLst,percentThres=0.1,samThres=3):
    dfLst=[]
    for place in placeLst:
        prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
        __renameTPM(prePure);dfLst.append(prePure)
    bad=[]
    for gene in tq(dfLst[0].index,desc='Sparse Genes'):
        samNum=0
        for df in dfLst:
            zeros = list(df.loc[gene].values).count(0);sams = len(df.columns);div = zeros/sams;div = 1 - div
            if(div>percentThres):samNum +=1
            if(samNum > samThres):break
        if(samNum <= samThres):bad.append(gene)
    return bad
def __renamePathCards(name):
    if not(name =='Immune Response CCR3 Signaling in Eosinophils'):
        name = name.lower();name = name.replace(' ','_');name = name.replace(':','');name= name.replace(',','')
        name = name.replace('/','');name = name.replace('.','')
    else:name = 'immune_response__ccr3_signaling_in_eosinophils'
    return name

def __getPathCards(nameIn):
    import requests;name = __renamePathCards(nameIn);url='https://pathcards.genecards.org/Card/'+name
    r = requests.get(url);test = str(r.content);test = test.split('<table class="table">\\r\\n')[2];lst=[]
    for i in range(len(test.split('</a>\\r\\n'))-2):lst.append(test.split('</a>\\r\\n')[i].split('>')[-1])
    return lst

In [5]:
__pcDict={};__pcDict['T-effector'] = 'T cell receptor signaling pathway'
__pcDict['IFNG induced'] = 'Type II interferon signaling (IFNG)'
__pcDict['Antigen processing machinery'] = 'Antigen processing-Cross presentation'
__pcDict['GABA'] = 'GABA receptor activation';__pcDict['Wnt'] = 'WNT Signaling'
__pcDict['Fatty acid'] = 'Fatty Acyl-CoA Biosynthesis'
__pcDict['Cholesterol synthesis'] = 'cholesterol biosynthesis III (via desmosterol)'
__pcDict['PI3K-AKT'] = 'PI3K / Akt Signaling';__pcDict['MTOR'] = 'mTOR signalling';pathCardsDict={}
for path in __pcDict:
    __card = __pcDict[path];__lst = __getPathCards(__card);pathCardsDict[path] = __lst
everyDict={}
for path in overAllDict:
    pathDict = overAllDict[path].copy();pathCardsLst = pathCardsDict[path]
    pathDict['Path Cards'] = pathCardsLst;everyDict[path] = pathDict

In [6]:
clusterLst=['IFNg','WNT Signaling','Antigen presentation','GABA','Cholesterol','Fatty Acid','T-effector','PI3K-AKT-mTOR']
def setupGSEA(placeLst,DictIn,removeBad=False,purity=True,correctNeg=True,directory='GSEA Input/',title='',preRank=False,
               dbs=False,skipSets=False,skipExpres=False,shiftZeros=True,HvC=True,skipImmune=False,useEveryDict=True):
    """setupGSEA:
                Input:
                    placeLst, a list of the cancer areas you wish to use, 
                        no default
                    Dict, a dictionary with the paths you want you want to use for your gene sets
                          and the genes within each set
                        no defualt
                    removeBad, boolean telling if you want to remove 'bad' genes (True) 
                               based on their correlation to other genes in a path, or keep them in (False) 
                        default = False
                    puriy, a boolean telling if you want to use pre (False) or post (True) purity values
                        default = True
                    correctAll, boolean telling if you want to have
                                genes negativly correlated with purity corrected or not
                        default = True
                    directory, the directory in which you want the files saved, 
                               each cancer area will be saved within its own folder within this directory
                        default = 'GSEA Input/'
                    dbs, boolean telling if to create seperate gene sets for each database (True),
                         or one superList (False)
                        default = False
                    HvC, boolean telling if to order gsea output as Hot vs Cold (True) or Cold vs Hot (False)
                        default = True
                    skipImmune, boolean telling if to skip immune pathways (True) or include them in the gene sets (False)
                        deafult = False
                Action:
                    Saves a geneSet file, 
                          as well as geneExpression and PhenotypeLabels files for each cancer area.
                          In the given directory. 
                    All of which can be used as input into GSEA
                Ouput:
                    The name of the directory in which all those files are saved."""
    Dict = DictIn.copy()
    if not(skipSets):
        if(dbs and useEveryDict):
            frstDic = list(Dict.keys())[0];dbLst=[]
            for db in Dict[frstDic]:dbLst.append(db)
            for db in dbLst:
                dbDict = __getDBList(Dict,db)
                __GSEAGeneSet(dbDict,placeLst,removeBad,Setname=db+'_GeneSet',skipImmune=skipImmune)
        else:
            if useEveryDict:name = 'SuperLst_GeneSet'
            else:name = 'MSigDB'+title+'_GeneSet'
            __GSEAGeneSet(Dict,placeLst,removeBad,Setname=name,skipImmune=skipImmune,useEveryDict=useEveryDict)
    if preRank:__GSEAPreRank(placeLst,purity,correctNeg,shiftZeros,setName=title)
    if not(skipExpres):
        outDict = __GSEAGeneExpression(placeLst,purity,correctNeg=correctNeg,directory=directory,shiftZeros=shiftZeros,HvC=HvC)
        __GSEAPhenLabels(placeLst,outDict,directory,purity,HvC=HvC)
    return directory

def __GSEAPreRank(placeLst,purity,correctNeg,shiftZeros,setName=''):
    out = 'GSEA Input/Ranks'
    if not os.path.exists(out):os.makedirs(out)
    candLst=['CD8A','GZMA','GZMB','PRF1']
    for place in tq(placeLst,desc='Making Rnk Files'):
        if(purity):
            if(correctNeg):
                if(shiftZeros):fileName = 'Purity/TPM/v7/TCGA_'+place+'_tpm.tsv'
                else:fileName = 'Purity/TPM/v8/TCGA_'+place+'_tpm.tsv'
            else:fileName = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv'
            df = pd.read_csv(fileName,index_col=0,sep='\t');heat=df.loc[candLst].mean();df=df.T
            pear=df.corrwith(heat,method='pearson');spear=df.corrwith(heat,method='spearman')
            corr = (pear+spear)/2;out2 = out+'/'+place
            if not os.path.exists(out2):os.makedirs(out2)
            out2 = out2+'/'+setName+'.rnk'
            with open(out2,'w') as f:
                for g in corr.index:
                    num=round(corr[g],3);f.write(g+'\t'+str(num)+'\n')
            f.close()
    return 1
def __GSEAGeneSet(Dict,placeLst,removeBad=False,Setname='',skipImmune=False,useEveryDict=True):
    if removeBad:placeLst2 = placeLst.copy()
    else:placeLst2=[placeLst[0]]
    for place in placeLst2:
        geneSet=pd.Series()
        for path in Dict:
            forward=True
            if(skipImmune):
                immuneLst=['T-effector','IFNG induced','Antigen processing machinery']
                if path in immuneLst:forward=False
            if(forward):
                pathGSEA = Dict[path]
                if useEveryDict:pathGSEA = __runGenes(pathGSEA)
                if(removeBad):
                    prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
                    goodPathGSEA = __goodGenes(pathGSEA,prePure.index)
                    cor = prePure.loc[goodPathGSEA].T.corr(method='spearman')
                    badLst = __getPurityLst(cor,0.5,'Down',0,percent=True)
                    pathGSEA = __removeLstFunc(pathGSEA,badLst,True)
                signature=pathGSEA;signature.insert(0,'na');geneSet[path]=signature
    out = 'GSEA Input/GeneSets'
    if not os.path.exists(out):os.makedirs(out)
    out = out+'/'+Setname+'.gmt'
    with open(out,'w') as f:
        for clus in geneSet.index:
            f.write(clus+'\t')
            for i in geneSet[clus]:f.write(i+'\t')
            f.write('\n')
    f.close();return Setname

def __GSEAGeneExpression(placeLst,purity=True,correctNeg=True,directory='GSEA Input/',
                         shiftZeros=True,name='GeneExpression',HvC=True):
    outDict={}
    for place in tq(placeLst,desc='GSEA Gene Expression'):
        fileName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
        df = pd.read_csv(fileName,index_col=0,sep='\t');__renameTPM(df);Hot,Cold = __getHotCold(df)
        if(purity):
            if(correctNeg):
                if(shiftZeros):fileName = 'Purity/TPM/v7/TCGA_'+place+'_tpm.tsv'
                else:fileName = 'Purity/TPM/v8/TCGA_'+place+'_tpm.tsv'
            else:fileName = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv'
            df = pd.read_csv(fileName,index_col=0,sep='\t');name = 'GeneExpression'+'PostPurity'
        else:name = 'GeneExpression'+'PrePurity'
        __renameTPM(df);__renameTPM(Hot);__renameTPM(Cold);df.dropna(inplace=True)
        Hot = __goodGenes(Hot.columns,df.columns);Cold = __goodGenes(Cold.columns,df.columns)
        if(HvC):colData = __colDataFunc(Hot,Cold)
        else:colData = __colDataFunc(Cold,Hot,hotName='Cold',coldName='Hot')
        rmvLst=[];noDups = __noDupsLst(colData.index);colData = colData.loc[noDups]
        df = df[noDups];df.insert(0,'Description','na');df = df.rename_axis('Name',axis=0);out=directory+place
        if not os.path.exists(out):os.makedirs(out)
        if(purity):out = out+'/'+'PostPurity'
        else:out = out+'/'+'PrePurity'
        if not os.path.exists(out):os.makedirs(out)
        df.to_csv(out+'/'+place+name+'.txt',sep='\t');outDict[place] = (colData)
    return outDict
def __GSEAPhenLabels(placeLst,outDict,directory='GSEA Input/',purity=True,HvC=True):
    for place in placeLst:
        colData = outDict[place];length = len(colData.index)
        qHot = colData[colData['Group'] == 'Hot'].index;qCold = colData[colData['Group'] == 'Cold'].index
        hots = __repeatStr('Hot',len(qHot),'\t');colds = __repeatStr('Cold',len(qCold),'\t');out = directory+'/'+place
        if not os.path.exists(out):os.makedirs(out)
        if(purity):out = out+'/'+'PostPurity'
        else:out = out+'/'+'PrePurity'
        if not os.path.exists(out):os.makedirs(out)
        name = out+'/'+place+'PrePurity'
        if(purity):name = out+'/'+place+'PostPurity'
        with open(name+'PhenotypeLabels.cls','w') as f:
            f.write(str(length)+'\t2\t1\n')
            if(HvC):
                f.write('#\tHot\tCold\n');f.write(hots+'\t'+colds)
            else:
                f.write('#\tCold\tHot\n');f.write(colds+'\t'+hots)
        f.close()
    return directory
def __repeatStr(string,num,sep):
    out=''
    for i in range(num-1):
        out += string;out += sep
    out += string;return out
def __getHotCold(df,perList=['25','75'],candLst=['CD8A','GZMA','GZMB','PRF1'],extra=''):
    percents={};candLst = __goodGenes(candLst,df.index);length = len(candLst)
    for i in candLst:
        x = np.quantile(df.loc[i],[0.25,0.75]);percents[i] = x
        df2 = df.loc[i];df2 = df2.transpose();df2 = df2[df2 <= percents[i][0]];df2.to_csv(extra+i+' 25Percentile')
        df2 = df.loc[i];df2 = df2.transpose();df2 = df2[df2 >= percents[i][1]];df2.to_csv(extra+i+' 75Percentile')
    for per in perList:
        candSamples=[]
        if(length > 0):
            g0 = pd.read_csv(extra+candLst[0]+' '+per+'Percentile',index_col=0);g0 = list(g0.index);candSamples = g0
        if(length > 1):
            g1 = pd.read_csv(extra+candLst[1]+' '+per+'Percentile',index_col=0);g1 = list(g1.index);candSamples = g0+g1
        if(length > 2):
            g2 = pd.read_csv(extra+candLst[2]+' '+per+'Percentile',index_col=0);g2 = list(g2.index);candSamples = g0+g1+g2
        if(length > 3):
            g3 = pd.read_csv(extra+candLst[3]+' '+per+'Percentile',index_col=0);g3 = list(g3.index);candSamples = g0+g1+g2+g3
        candSamples = __rmvTCGA(candSamples);candSamples = __goodGenes(candSamples,df.columns)
        if(per=='75'):dfHot = df[candSamples]
        if(per=='25'):dfCold = df[candSamples]
    for j in candLst:
        os.remove(extra+j+' 25Percentile');os.remove(extra+j+' 75Percentile')
    return dfHot,dfCold

def __onlyGroup(Dict,group):
    outDict={}
    for path in Dict:
        dict2 = Dict[path];grpDict={}
        for grp in dict2:
            if(grp == group):
                lst = dict2[grp];grpDict[grp] = lst
        outDict[path] = grpDict
    return outDict
def __notHotOrCold(df,hot,cold):
    notLst=[]
    for col in df.columns:
        if(col not in hot.columns and col not in cold.columns):notLst.append(col)
    out = df[notLst]
    return out

def __getDBList(Dict,db):
    outDict={}
    for path in Dict:
        outDictPath={};pathDict=Dict[path];dbLst=pathDict[db];outDictPath[db]=dbLst;outDict[path]=outDictPath
    return outDict
def __geneSetLst(placeLst,Setname,frst=True):
    if(frst):placeLst=[placeLst[0]]
    for palce in placeLst:
        directory = 'GSEA Input/GeneSets';read = directory+'/'+place+Setname+'.gmt'
        df = pd.read_csv(read,index_col=0,sep='\t', names=list(range(500))).dropna(axis='columns', how='all')
    return df
def __getBads(geneLst,badLst):
    heatLst=[]
    for gene in geneLst:
        if(gene in badLst):heatLst.append('Hot')
        else:heatLst.append('Cold')
    df=pd.DataFrame(heatLst,geneLst,columns=['Removed'])
    return df

def __fixCor(df,path,rowPosition,colPosition=[]):
    geneOrder = rowPosition[path];genes = __goodGenes(geneOrder,df.index)
    if(colPosition == []):df2 = df.loc[genes][genes]
    else:
        colOrder =colPosition[path];cols = __goodGenes(colOrder,df.columns);df2 = df.loc[genes][cols]
    return df2
def __getClusterPosition(df,more=False):
    cg = sb.clustermap(df);row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
    if(more):
        dfIndx = df.index;geneOrder=[]
        for num in row:
            gene = dfIndx[num];geneOrder.append(gene)
        dfIndx = df.columns;geneOrder2=[]
        for num in col:
            gene = dfIndx[num];geneOrder2.append(gene)
        return geneOrder,geneOrder2
    return col, row

def __logFoldChange(placeLst,mean=True,ttest=True,correctAll=False):
    maxPlace = __maxLenPlace(placeLst,correctAll=correctAll)
    if(correctAll):filename = 'Purity/TPM/v4/TCGA_'+maxPlace+'_tpm.tsv'
    else:filename = 'Purity/TPM/v1/TCGA_'+maxPlace+'_tpm.tsv'
    maxPure = pd.read_csv(filename,index_col=0,sep='\t')
    maxPure.dropna();maxHot,maxCold = __getHotCold(maxPure);colDataMax = __colDataFunc(maxHot.columns,maxCold.columns)
    maxPure = maxPure[colDataMax.index];out = pd.DataFrame();out2 = pd.DataFrame();titleLst=[]
    for place in tq(placeLst,desc='Fold Change and P-value'):
        LFCLst=[];pvalLst=[]
        if(correctAll):filename = 'Purity/TPM/v4/TCGA_'+place+'_tpm.tsv'
        else:filename = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv'
        postPure = pd.read_csv(filename,index_col=0,sep='\t');postPure.dropna();minVal = min(postPure[min(postPure)])
        postPure = postPure - minVal;postHot,postCold = __getHotCold(postPure)
        colDataPost = __colDataFunc(postHot.columns,postCold.columns);postPure = postPure[colDataPost.index]
        if(mean):
            title = place+' LFC by Mean';postHotM = postHot.mean(axis=1);postColdM = postCold.mean(axis=1)
        else:
            title = place+' LFC by Median';postHotM = postHot.median(axis=1);postColdM = postCold.median(axis=1)
        titleLst.append(title)
        if(ttest):
            tTestLst = stats.ttest_ind(postHot.T,postCold.T)[1];title = place+' P-Val by T-test'
        else:title = place+' P-Val by RankSum'
        titleLst.append(title);num = 0
        for gene in maxPure.index:
            if(gene in postPure.index):
                A = postColdM[gene];B = postHotM[gene];A = A+0.0000001;B = B+0.0000001;LFC = np.log2((A/B));LFCLst.append(LFC)
                if(ttest):pval = tTestLst[num]
                else:
                    pval = stats.ranksums(postHot.loc[gene].values,postCold.loc[gene].values);pval = pval.pvalue
                num += 1;pvalLst.append(pval)
            else:
                LFCLst.append(np.nan);pvalLst.append(np.nan)
        out[place] = LFCLst;out2[place] = pvalLst
    out.index = maxPure.index;out.index.name = 'Genes';out2.index = maxPure.index;out2.index.name = 'Genes';out = out.dropna()
    out2 = out2.dropna();index = __goodGenes(out.index,out2.index);out = out.loc[index];out2 = out2.loc[index]
    return out, out2

def __getFirst(lst,col):
    for i in range(len(lst)):
        if(lst[i].columns[0]==col):return i 
def __mergeCols(fileName,lst,cols,mergeOn,save=True):
    dfNum = __getFirst(lst,cols[0]);df = lst[dfNum]
    for i in range(dfNum+1,len(lst)):
        if(i%100==0 and save):df.to_csv('HotCold Samples/'+fileName)
        if within(lst[i].columns[0],cols):df = df.merge(lst[i],on=mergeOn)
    if(save):df.to_csv('HotCold Samples/'+fileName)
    return df
def plotXCell(placeLst,save=False,ratio=True,heat=True,thres=0.02,filt=False):
    """plotXCell:
                Input:
                    placeLst, a list of cancer areas for which to create xCell plots
                        no default
                    save, boolean telling if to save the cancer area xCell data
                          in the 'Share Data/' folder
                        default = False
                    ratio, boolen telling if to get ratios of each types percent in each sample (if Ture)
                           or keep raw values (if False)
                        default = True
                    heat, boolean telling if to plot a heatmap of the xCell data for each cancer area
                        default = True
                    thres, the threshold a row in the data should be above
                           makes the most sense in when ratio = True
                        default = 0.02 
                    filt, boolean telling if to filter out rows that failed to be > thres
                        default = False
                Action:
                    Plots a heatmap of the xCell data, in either ratio or raw value, 
                    and maybe filtered bvy thres, for each cancer area in placeLst.
                    May also save that data if told to do so.
                Output: Upon success, gives back a 1 
    """
    xCell = pd.read_csv('xCell_TCGA_RSEM.txt',index_col=0,sep='\t')
    __renameTPM(xCell)
    for place in tq(placeLst,desc='Plot XCell'):
        prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
        __renameTPM(prePure)
        cols = __goodGenes(xCell.columns,prePure.columns)
        xCellCancer = xCell[cols]
        title = 'XCell Values, '+place
        if(ratio):
            title = 'XCell Ratios, '+place
            for col in xCellCancer.columns:
                ser = xCellCancer[col]
                serMin = ser.min()
                shiftSer = ser - serMin
                shiftMax = shiftSer.max()
                percents = shiftSer / shiftMax
                xCellCancer[col] = percents
            if(filt):
                thresName = str(round(thres*100))
                title = 'XCell Ratios, Filtered < '+thresName+'%, '+place
                xCellCancer = xCellCancer.T
                rowLength = len(xCellCancer.index)
                xCellCancer = xCellCancer[(xCellCancer.sum() > (thres*rowLength)).replace(False,np.nan).dropna().index]
                xCellCancer = xCellCancer.T
        if(save):
            saveTitle = title.replace(' ','_')
            xCellCancer.to_csv('Share Data/'+saveTitle)
        if(heat):
            cg = sb.heatmap(xCellCancer,cmap=cmapSpecial);plt.title(title);plt.show();print(title)
    return 1

def getQualifedGenes(placeLst,mean=True,ttest=True,threshold=2,upCold=True,lfc=0,pv=1):
    LFC, pval = __logFoldChange(placeLst,mean=mean,ttest=ttest);geneLst=[]
    for gene in tq(LFC.index,desc='Qual genes'):
        if(upCold):areasL = LFC.loc[gene][LFC.loc[gene] > lfc].index
        else:areasL = LFC.loc[gene][LFC.loc[gene] < (-1*lfc)].index
        areasP = pval.loc[gene][pval.loc[gene] < pv].index;num = 0
        for cancer in areasL:
            if cancer in areasP:num += 1
        if(num >= threshold):geneLst.append(gene)
    return geneLst
def partitionLst(Lst,div=4):
    """partitionLst:
                    Input:
                        Lst, a list of values
                            no default
                        div, number of elements each new divded list must contain
                            default = 4
                    Action:
                        Divides the inputed list into similar sized lists each with
                        the inputed number of elements
                    Example:
                        In: partionLst([1,2,3,4,5,6,7],3)
                        Out: [[1,2,3],[4,5,6],[7]]
                    Output:
                        A list of lists, each with the inputed number of elements in them 
        """
    outLst=[];start=0;current = div;length = len(Lst);num = int(length/div);last = length%div
    while(length > 0):
        outLst.append(Lst[start:current]);start += div;current += div;length = length - div
    return outLst

def __noDupsLst(lst,onlyDups=False):
    out = [];outRep=[]
    for i in lst:
        if(not i in out):
            out.append(i)
        else:
            if(onlyDups):
                if(not i in outRep):
                    outRep.append(i)
    if(onlyDups):
        out = outRep
    return out

def __getNameGSEA(file):
    f = file;f = f.split('\\')[-1];f = f.split('.')[0];f = f.replace('_',' ')
    return f

def __getGSEAdfs(date,tpy,placeLst,desktop=True,skipImmune=False,preRank=False):
    pathDict = __getPathDict(date=date,tpy=tpy,placeLst=placeLst,desktop=desktop,preRank=preRank)
    pathLst = list(pathDict.keys());dfNES = pd.DataFrame();dfFDR = pd.DataFrame()
    for gsea in glob.glob('GSEA Output/'+date+'/'+tpy+'/*'):
        cancer = __getNameGSEA(gsea)
        if(cancer in placeLst):
            nesLst=__repeatAppend([],len(pathDict),0);fdrLst=__repeatAppend([],len(pathDict),0)
            if(desktop):
                fileName=gsea+'/*report*.xls';sep='\t';nCol='NES';fCol='FDR q-val'
            else:
                if preRank:fileName =gsea+'/gseapy.prerank.gene_sets.report.csv'
                else:fileName =gsea+'/gseapy.gsea.phenotype.report.csv'
                sep=',';nCol = 'es';fCol = 'fdr'
            for file in glob.glob(fileName):
                df = pd.read_csv(file,index_col=0,sep=sep)
                for path in df.index:
                    Nes = df.loc[path][nCol];fdr = df.loc[path][fCol]
                    path = path.upper();nesLst[pathDict[path]] = Nes;fdrLst[pathDict[path]] = fdr
            dfNES[cancer] = nesLst;dfFDR[cancer] = fdrLst
    dfNES.index = pathLst;dfFDR.index = pathLst
    __addKs(dfNES,axis=0,lst=dfNES.index,k=tpy+'_',inplace=True)
    __addKs(dfFDR,axis=0,lst=dfFDR.index,k=tpy+'_',inplace=True)
    dfNES.index.name = 'Paths';__addKs(dfNES,lst=kobePlaceLst1,inplace=True)
    __addKs(dfFDR,lst=kobePlaceLst1,inplace=True);dfNES.columns = getUniqueNames(dfNES.columns)
    dfFDR.index.name = 'Paths';dfFDR.columns = getUniqueNames(dfFDR.columns)
    dfNES = dfNES.replace(0, np.nan);dfNES.dropna(axis=0,how='all',inplace=True);dfNES.dropna(axis=1,inplace=True)
    dfFDR = dfFDR.loc[dfNES.index];dfFDR = dfFDR[dfNES.columns];dfNES = dfNES.replace(np.inf,0);dfFDR = dfFDR.replace(np.inf,0)
    if(skipImmune):
        dfNES = __skipImmuneDf(dfNES);dfFDR = __skipImmuneDf(dfFDR)
    return dfNES,dfFDR

def __skipImmuneDf(dfIn,immuneLst=['T-EFFECTOR','IFNG INDUCED','ANTIGEN PROCESSING MACHINERY'],inplace=True):
    if(inplace):df = dfIn
    else:df = dfIn.copy()
    newId=[]
    for path in df.index:
        if not any(i in path for i in immuneLst):newId.append(path)
    df = df.loc[newId]
    return df
def getUniqueNames(lst,thres=10):
    newNum = 1;dups = [''];numLst = __repeatAppend([],len(lst),0)
    while True:
        nameLst=[]
        for idx in range(len(lst)):
            name = lst[idx];num = numLst[idx];frst = ' '.join(name.split()[:num])
            if(frst in dups):
                frst = ' '.join(name.split()[:newNum]);numLst[idx] = newNum
            nameLst.append(frst)
        dups = __noDupsLst(nameLst,onlyDups=True)
        if(dups == []):
            newCol=[]
            for col in nameLst:
                if(col=='Breast'):col = 'Breast HR+'
                newCol.append(col)
            nameLst = newCol
            return nameLst
        newNum += 1
        if(newNum > thres):
            print('getUniqueNames warning: Reached Threshold of '+
                  str(thres)+
                  ' before getting complete unique names\nEither threshold is too small or certain names are duplicates')
            return nameLst
def __setCluster(inputDF,row,col,inplace=False):
    if(inplace):df = inputDF
    else:df = inputDF.copy()
    rowLst=[];colLst=[]
    for num in row:
        elem = df.index[num];rowLst.append(elem)
    for num in col:elem = df.columns[num];colLst.append(elem)
    df = df[colLst];df = df.loc[rowLst]
    return df

def __addKs(dfIn,k='K_',axis=1,inplace=False,lst=[]):
    if(inplace):df = dfIn
    else:df = dfIn.copy()
    if(axis==1):
        colLst=[]
        for col in df.columns:
            if col in lst:
                col = k+col
            colLst.append(col)
        df.columns = colLst
    if(axis==0):
        rowLst=[]
        for row in df.index:
            if(row in lst):row = k+row
            rowLst.append(row)
        df.index = rowLst
    return df
def plotCorrelation(placeLst,Dict,prePure=False,getPosition=False,position={},meth='spearman',cmap=cmapSpecial,
                   size=(15,5),padd=3,name='',save=False):
    titleLst=[]
    if(getPosition):placeLst = [placeLst[0]];positon={}
    for place in tq(placeLst,desc='plot Correlation'):
        title=''
        if(prePure):
            title += 'Pre-Purity, '+place+', '
            df = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
            __renameTPM(df);df = np.log2(df+1);df = pd.DataFrame(stats.zscore(df,axis=1),df.index,columns=df.columns)
        else:
            title += 'Post-Purity, '+place+', ';df = pd.read_csv('Purity/TPM/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        df.dropna(inplace=True);Hot,Cold = __getHotCold(df);Hot.dropna(inplace=True);Cold.dropna(inplace=True)
        for path in Dict:
            title2 = title+path+', ';pathDict = Dict[path];genes = __runGenes(pathDict);genes = __goodGenes(genes,df.index)
            hotPath = Hot.loc[genes];hotPath = hotPath.T;coldPath = Cold.loc[genes]
            coldPath = coldPath.T;hotCor = hotPath.corr(method=meth);coldCor = coldPath.corr(method=meth)
            if(getPosition):
                col, row = __getClusterPosition(hotCor);dfIndx = hotCor.index;geneOrder=[]
                for num in col:
                    gene = dfIndx[num];geneOrder.append(gene)
                position[path] = geneOrder
            else:
                hotCor = __fixCor(hotCor,path,position);coldCor = __fixCor(coldCor,path,position)
                fig, axes = plt.subplots(nrows=1, ncols=2, figsize=size);ax1 = axes[0];ax2 = axes[1];fig.tight_layout(pad=padd)
                title2 += 'Hot';sb.heatmap(hotCor,cmap=cmap,ax=ax1,vmin=-1,vmax=1);ax1.set_title(title2)
                title2 = title2.replace('Hot','Cold');sb.heatmap(coldCor,cmap=cmap,ax=ax2,vmin=-1,vmax=1)
                ax2.set_title(title2);title2 = title2.replace('Cold','Samples Correlation, ');title2 += meth
                if(save):
                    plt.savefig(title2+'.png',bbox_inches = "tight");titleLst.append(title2+'.png')
                plt.show();print(title2)
    if(getPosition):
        return position
    if(save):
        __mergePDFs(titleLst,name)
    return name

def __mergePDFs(fileNames,name,erase=False):
    if os.path.exists(name):
        os.remove(name)
    import PyPDF2
    from PyPDF2 import PdfFileWriter, PdfFileReader
    input_streams = [];output_stream = open(name+".pdf", 'w+b')
    try:
        for input_file in fileNames:
            input_streams.append(open(input_file, 'rb'))
        writer = PdfFileWriter()
        for reader in map(PdfFileReader, input_streams):
            for n in range(reader.getNumPages()):
                writer.addPage(reader.getPage(n))
            writer.write(output_stream)
    finally:
        for f in input_streams:
            f.close()
        output_stream.close()
    if(erase):
        for file in fileNames:
            os.remove(file)
    return name

def __maxLenPlace(placeLst,correctAll=False,minn=False):
    if(minn):
        maxLen = 1000000000
    else:
        maxLen = 0
    maxPlace=placeLst[0]
    for place in tq(placeLst,desc='Max Len Place'):
        if(correctAll):
            filename = 'Purity/TPM/v4/TCGA_'+place+'_tpm.tsv'
        else:
            filename = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv'
        postPure = pd.read_csv(filename,index_col=0,sep='\t');postPure.dropna()
        postHot,postCold = __getHotCold(postPure);colDataPost = __colDataFunc(postHot.columns,postCold.columns)
        postPure = postPure[colDataPost.index];length = len(postPure.index)
        if(minn):
            if(length < maxLen):
                maxLen = length;maxPlace = place
        else:
            if(length > maxLen):
                maxLen = length;maxPlace = place
    return maxPlace

def plotLFC(placeLst,Dict,mean=True,ttest=True,getPosition=False,row_position={},col_position={},
           size=(12,4),padd=3,correctAll=False):
    LFC, pval = __logFoldChange(placeLst,mean=mean,ttest=ttest,correctAll=correctAll)
    if(getPosition):
            row_position={};col_position={}
    for path in tq(Dict,desc='Plotting'):
        if not(getPosition):
            fig, axes = plt.subplots(nrows=1, ncols=2, figsize=size)
            ax1 = axes[0];ax2 = axes[1];fig.tight_layout(pad=padd)
        pathDict = Dict[path];genes = __runGenes(pathDict)
        genes = __goodGenes(genes,LFC.index);LFCgenes = LFC.loc[genes].T;LFCgenes.index.name='Cancer Areas'
        if(getPosition):
            row_order, col_order = __getClusterPosition(LFCgenes,more=True)
            row_position[path] = row_order;col_position[path] = col_order
        else:
            LFCgenes = __fixCor(LFCgenes,path,row_position,col_position)
            LFCgenes = __renameLFC(LFCgenes);sb.heatmap(LFCgenes,cmap=cmapSpecial2,ax=ax1,vmin=-1.5,vmax=1.5)
            if(mean):
                ax1.set_title(path+' Log2 FC by Mean')
            else:
                ax1.set_title(path+' Log2 FC by Median')
            genes = __runGenes(pathDict);genes = __goodGenes(genes,pval.index);pvalgenes = pval.loc[genes].T
            pvalgenes.index.name='Cancer Areas';pvalgenes = __fixCor(pvalgenes,path,row_position,col_position)
            pvalgenes = __renameLFC(pvalgenes);sb.heatmap(pvalgenes,cmap=cmapSpecial,vmin=0,vmax=1,ax=ax2)
            if(ttest):
                ax2.set_title(path+ ' P-Value by T-Test')
            else:
                ax2.set_title(path+ ' P-Value by RankSums')
            plt.show();print(path)
    if(getPosition):
        return row_position,col_position
    else:
        return 1
    
def __numLstsIn(elem,SuperLst=[]):
    num = 0
    for lst in SuperLst:
        if(elem in lst):
            num += 1
    return num
def __enoughLsts(elem,SuperLst=[],threshold=3):
    num = __numLstsIn(elem,SuperLst)
    return (num >= threshold)

def __renameLFC(lfc):
    Col = []
    for col in lfc.index:
        col = col.replace('Carcinoma','');col = col.replace('carcinoma','');col = col.replace('Melanoma','');Col.append(col)
    lfc.index = Col
    return lfc
# Gets the mix N row-col pairings from within a dataframe
# 
def __topN(df,num=1,absVal=True,top=True):
    outLst=[];df2 = df.copy()
    if(absVal):
        df2 = abs(df2)
    if top:
        for i in range(num):
            maxSer = df2.max();maxId = maxSer.argmax();maxCol = maxSer.index[maxId];maxRow = df2[maxCol].idxmax()
            df2[maxCol][maxRow] = 0;df2[maxRow][maxCol] = 0
            outLst.append((maxCol,maxRow,maxCol+', '+maxRow))
    else:
        df2 = df2[df2 >= num]
        for col in df2:
            ser = df2[col];ser = ser.dropna()
            for row in ser.index:
                df2[col][row]=np.nan;df2[row][col]=np.nan;outLst.append((col,row,col+', '+row))
    return outLst

__careLst = ['Bladder', 'Breast', 'Colon', 'Liver', 'Lung', 'Melanoma', 'Ovarian', 'Pancreatic', 'Prostate','renal']
def kobePlaceLst(careLst=__careLst,thres=1,cares=True):
    """kobePlaceLst:
                    Input:
                            careLst, a list of cancer areas that every area in the output list 
                                     must be a member of one of these areas, 
                                     and areas outside of careLst will be ignored.
                                default = kobe cancers 'Bladder', 'Breast', 'Colon',
                                                       'Liver', 'Lung', 'Melanoma', 
                                                       'Ovarian', 'Pancreatic', 'Prostate',
                                                       and 'renal' 
                            thres, the number of hot/cold samples a place must include to be in the output
                                default = 1
                            cares, a boolean telling if caresLst should be ignored
                                default = True
                    Action:
                            Looks through all available TPM cancer areas, and creates a list of areas
                            with hot/cold samples > the given threshold, and within the careLst 
                    Output:
                            A list of cancer areas.
    """
    pLst = glob.glob('Genes/TPM/*');inputs = tq(pLst)
    out = Parallel(n_jobs=num_cores)(delayed(__kobePlaceLstHelp)(p,careLst,thres,cares) for p in inputs)
    out = set(out);out.remove('');out = list(out);out.sort()
    return out

def __kobePlaceLstHelp(p,careLst,thres,cares):
    place = p.split('_')[1]
    if(cares):
        value = any(e in place for e in careLst)
    else:
        value = True
    if(value):
        filename = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv';postPure = pd.read_csv(filename,index_col=0,sep='\t')
        if(len(postPure.columns)>0):
            length = len(postPure.columns)/2
            if(length>(thres*2)):
                return place
    return ''

def getBiggestXCancers(num=1,thres=30):
    pLst = glob.glob('Genes/TPM/*')
    inputs = pLst;out = Parallel(n_jobs=num_cores)(delayed(__getBiggestXCancersHelp)(p) for p in inputs);outLst=[]
    if(num>len(pLst)):
        num = len(pLst)
    for i in range(num):
        argMax = out.index(max(out))
        place = out[argMax][1];num = out[argMax][0];outLst.append((place,num));out[argMax] = (0,place)
    return outLst
def __getBiggestXCancersHelp(p): 
    place = p.split('_')[1]
    filename = 'Purity/TPM/v1/TCGA_'+place+'_tpm.tsv';postPure = pd.read_csv(filename,index_col=0,sep='\t')
    return  len(postPure.columns),place

def __getPathDict(date,tpy,placeLst,desktop=False,preRank=False):
    pathLst=[]
    for gsea in glob.glob('GSEA Output/'+date+'/'+tpy+'/*'):
        cancer = __getNameGSEA(gsea)
        if(cancer in placeLst):
            if(desktop):
                fileName=gsea+'/*report*.xls';sep='\t';nCol='NES';fCol='FDR q-val'
            else:
                if preRank:
                    fileName =gsea+'/gseapy.prerank.gene_sets.report.csv'
                else:
                    fileName =gsea+'/gseapy.gsea.phenotype.report.csv'
                sep=',';nCol = 'es';fCol = 'fdr'
            for file in glob.glob(fileName):
                df = pd.read_csv(file,index_col=0,sep=sep)
                for path in df.index:
                    pathLst.append(path)
    pathLst = list(set(pathLst));num=0;pathDict={}
    for path in pathLst:
        pathDict[path]=num;num+=1
    return pathDict

In [7]:
def masterPlotGSEA(dateIn,placeLst,databaseLst='All',title='GSEA',desktop=False,save=False,preRank=False,
                   metric='cosine',fixPos=False,Position={},fontSize=12,rot=45,skipImmune=False,coldNum=0,figSize=(30,20),
                   skipKobePaths=False,numParts=20,reverseOrder=False,dbName='',fdrVal=0.25,
                   noParts=False,removeBad=False,containsColdCluster=(False,False,0)):
    """masterPlotGSEA:
                        Input:
                            dateLst, list of all days in which the GSEA anyalysis you want plotted were done
                                  typically first three letters of the month then the number of the day
                                  ex: jul13, nov5
                                no default
                            placeLst, list of cancer areas to include in the plot
                                no default
                            databaseLst, list of the database to plot,
                                         for all available databases enter 'All'
                                         ex: ['KOBE','KEGG']
                                default = 'All'
                            title, the title of the heatmaps created
                                default = 'GSEA'
                            desktop, boolean telling if output if from GSEA desktop/command line version (True)
                                     or the python wrapper version (False)
                                default = False
                            save, boolean telling if to save the clustermaps (will be in 'Saved Images/ Folder')
                            preRank, boolean telling if GSEA-Prerank was run, or normal GSEA
                            metric, str metric used for clustering
                                default = 'cosine'
                            fixPos, boolean telling if to use Position to order the plots, or use clustering
                            Position, dictionary to be used for positioning clustermap if fixPos == True
                            rot, 
                            
                        Actions:
                            Plots a clustermap of the NES and FDR 
                            (with FDR's positions based on NES's clustering)
                            from all the available GSEA data from the date given
                        Output:
                            On success, it returns the position dictionary to be used by reruns of this function to have
                            different plots with the same positioning
    """
    if(isinstance(dateIn,list)):
        dateLst = dateIn.copy()
    else:
        dateLst = [dateIn]
    if(databaseLst=='All'):
        dbLst=[]
        for gsea in glob.glob('GSEA Output/'+dateLst[0]+'/*'):
            dbLst.append(gsea.split('\\')[-1])
    else:
        dbLst=databaseLst.copy()
    if(skipKobePaths):
        dbLst.remove('KOBE')
    dateNum=1
    for date in dateLst:
        datePos={}
        superDfNes,superDfFdr = __superDfGSEA(date,dbLst=dbLst,placeLst=placeLst,
                                              desktop=desktop,skipImmune=skipImmune,
                                              preRank=preRank,coldNum=coldNum,fdrVal=fdrVal,
                                              containsColdCluster=containsColdCluster)
        superDfNes,superDfFdr,pathLst,catLst = __getPathLst(superDfNes,superDfFdr,removeBad)
        superDfNes.index = pathLst;superDfFdr.index = pathLst
        if coldNum > 0:
            if not containsColdCluster[0]:
                title2 = title+' At least '+str(coldNum)+' Up in Cold Pathways, FDR '+str(fdrVal)
            else:
                title2 = title+' FDR '+str(fdrVal)
        else:
            title2 = title+' All Pathways'
        print('plotting')
        if len(superDfNes.index) > 1:
            if dbName=='c2.all':
                df3,df4 = __C2SplitGSEA(superDfNes,superDfFdr,useLst=True,lst=catLst)
                superDfNes3,superDfFdr3,catLst3=df3;superDfNes4,superDfFdr4,catLst4=df4;dbName=dbName+' '+msigNameDict[dbName]
                if len(superDfNes3.index) > 1:
                    row,col = __helpPlotGSEA(superDfNes3,superDfFdr3,title=title2,db=dbName+' Canonical, ',metric=metric,
                                             save=save,numParts=numParts,reverseOrder=reverseOrder,noParts=noParts,
                                             catLst=catLst3,
                                             figSize=figSize,fixPos=fixPos,pos=Position,fontSize=fontSize,rot=rot)
                if len(superDfNes4.index) > 1:
                    row,col = __helpPlotGSEA(superDfNes4,superDfFdr4,title=title2,
                                             db=dbName+' Chemical and Genetic Perturbations, ',metric=metric,
                                             catLst=catLst4,
                                             save=save,numParts=numParts,reverseOrder=reverseOrder,noParts=noParts,
                                             figSize=figSize,fixPos=fixPos,pos=Position,fontSize=fontSize,rot=rot)
            else:
                if dbName=='c5.all':
                    df3,df4 = __C2SplitGSEA(superDfNes,superDfFdr,useLst=True,lst=catLst,keep=['GO'])
                    superDfNes3,superDfFdr3,catLst3 = df3
                    dbName = dbName+' '+msigNameDict[dbName]+' GO Biological Process, '
                    if len(superDfNes3.index) > 1:
                        row,col = __helpPlotGSEA(superDfNes3,superDfFdr3,title=title2,db=dbName,metric=metric,
                                                 save=save,numParts=numParts,reverseOrder=reverseOrder,
                                                 catLst=catLst3,figSize=figSize,fixPos=fixPos,pos=Position,
                                                 fontSize=fontSize,rot=rot,noParts=noParts)
                else:
                    dbName = dbName+' '+msigNameDict[dbName]+', '
                    row,col = __helpPlotGSEA(superDfNes,superDfFdr,title=title2,db=dbName,metric=metric,save=save,
                                             numParts=numParts,reverseOrder=reverseOrder,noParts=noParts,catLst=catLst,
                                             figSize=figSize,fixPos=fixPos,pos=Position,fontSize=fontSize,rot=rot)
            datePos['All']= (row,col)
    gc.collect()
    return datePos
def __superDfGSEA(date,dbLst,placeLst,desktop,skipImmune,preRank,coldNum,fdrVal,containsColdCluster=(False,False,0)):
    if(len(dbLst)<=0):
        print('Date inputed has no GSEA output')
        raise 
    frstDB=True
    for db in tq(dbLst,desc='superDfGSEA'):
        dfNes, dfFdr = __getGSEAdfs(date,db,placeLst,desktop,skipImmune,preRank=preRank)
        dfNes, dfFdr =__getUpinCold(dfNes,dfFdr,coldNum,coldNum,fdrVal)
        if(frstDB):
            superDfNes = dfNes.copy();superDfFdr = dfFdr.copy();frstDB=False
        else:
            superDfNes = pd.concat([superDfNes,dfNes]);superDfFdr = pd.concat([superDfFdr,dfFdr])
        gc.collect()
    if containsColdCluster[0]:
        coldCancerLst=['Adrenocortical','Breast HR+','K_Lung adenocarcinoma','K_Lung squamous','Thyroid','K_Skin']
        if containsColdCluster[1]:
            lst2=[]
            for place in superDfNes.columns:
                if not place in coldCancerLst:
                    lst2.append(place)
            coldCancerLst = lst2
        df2=superDfNes[coldCancerLst];df2=df2[df2<0]
        df2=countNotNaNs(df2,rev=True);df2=df2[df2>=containsColdCluster[2]];clusterPathways = list(df2.index)
        superDfNes = superDfNes.loc[clusterPathways];superDfFdr = superDfFdr.loc[clusterPathways]
    superDfNes = superDfNes.dropna(how='all').fillna(0);superDfFdr = superDfFdr.dropna(how='all').fillna(1)
    if 'Uterine' in superDfNes.columns and len(placeLst)>20:
        superDfNes = superDfNes.drop(columns=['Uterine']);superDfFdr = superDfFdr.drop(columns=['Uterine'])
    gc.collect()
    return superDfNes, superDfFdr
def __helpPlotGSEA(dfNes,dfFdr,title,db,metric='cosine',fixPos=False,pos={},fontSize=12,rot=45,save=False,catLst=[],
                   figSize=(20,20),noParts=False,rowColors=(None,{}),
                   numParts=20,reverseOrder=False,suptitleName=''):
    gc.collect();parts = math.ceil(len(dfNes.index)/numParts)
    maxx = max(dfNes.max().max(),1);minn = min(dfNes.min().min(),-1);cClus=not(fixPos);rClus=not(fixPos)
    if(fixPos):
        row, col = pos[db];dfNes = __setCluster(dfNes,row,col)
    cg = sb.clustermap(dfNes,cmap=cmapSpecial,col_cluster=cClus,figsize=figSize,vmax=maxx,vmin=minn,
                       row_cluster=rClus,metric=metric,yticklabels=0,xticklabels=1,row_colors=rowColors[0])
    if(fixPos):
        row, col = pos[db]
    else:
        row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
    if noParts: 
        saveLst=[]
        for x,y in zip(cat2,colorLst):
            cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=10, solid_capstyle="butt")
        cg.ax_heatmap.legend(loc=(-0.05,1), title="MSig Category",prop={'size': 40})
        cg.ax_heatmap.set_xlabel('Cancers',size=40);cg.ax_heatmap.set_ylabel('Paths',size=40)
        plt.setp(cg.ax_heatmap.get_xticklabels(),rotation=rot,ha='right',size=30);title2 = db+title+', Metric '+metric+', NES'
        cg.fig.suptitle(title2,size=50);plt.title('NES Values',size=40)
        if save:
            plt.savefig('Saved Images/'+title2+'.pdf',dpi=200,bbox_inches = "tight")
            plt.close();saveLst.append('Saved Images/'+title2+'.pdf')
        else:
            plt.show();print(title2)
        dfFdr = __setCluster(dfFdr,row,col)
        cg = sb.clustermap(dfFdr,cmap=cmapSpecial,col_cluster=False,figsize=figSize,vmax=1,vmin=0,
                           row_cluster=False,metric=metric,yticklabels=0,xticklabels=1)
        cg.ax_heatmap.set_xlabel('Cancers',size=40);cg.ax_heatmap.set_ylabel('Paths',size=40)
        plt.setp(cg.ax_heatmap.get_xticklabels(),rotation=rot,ha='right',size=30)
        title2 = db+title+', Metric '+metric+', FDR';cg.fig.suptitle(title2,size=50);plt.title('FDR Values',size=40)
        if save:
            plt.savefig('Saved Images/'+title2+'.pdf',dpi=200,bbox_inches = "tight");plt.close()
            saveLst.append('Saved Images/'+title2+'.pdf')
        else:
            plt.show();print(title2)
    else:
        plt.close();dfNes = __setCluster(dfNes,row,col);dfFdr = __setCluster(dfFdr,row,col);rowColors2=[]
        for rowCol in rowColors[0]:
            rowLst=[]
            for num in row:
                elem = rowCol[num];rowLst.append(elem)
            rowColors2.append(rowLst)
        if reverseOrder:
            dfNes = dfNes.iloc[::-1];dfFdr = dfFdr.iloc[::-1]
        saveLst=[]
        for n in range(parts):
            partName = str(n+1)
            rows = list(dfNes.index)[n*numParts:(n+1)*numParts];dfNes2 = dfNes.loc[rows];dfFdr2 = dfFdr.loc[rows]
            if rowColors:
                rowColorPart = []
                for rowCol in rowColors2:
                    rowColorPart.append(rowCol[n*numParts:(n+1)*numParts])
            else:
                rowColorPart=None
            if len(dfNes2.index)>0:
                cg = sb.clustermap(dfNes2,cmap=cmapSpecial,col_cluster=False,figsize=figSize,vmax=maxx,vmin=minn,annot=dfFdr2,
                                   row_cluster=False,metric=metric,yticklabels=1,xticklabels=1,annot_kws={"size": 30},
                                   row_colors=rowColorPart)
                for x,y in rowColors[1]:
                    cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=10, solid_capstyle="butt")
                if n == 0:
                    cg.ax_heatmap.legend(loc=(-0.05,1), title="C Groups",prop={'size':40})
                cg.ax_heatmap.set_xlabel('Cancers',size=50);cg.ax_heatmap.set_ylabel('Paths',size=50)
                plt.setp(cg.ax_heatmap.get_xticklabels(),rotation=rot,ha='right',size=35)
                plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=rot,va='bottom',size=35)
                cg.ax_heatmap.set_title(suptitleName+'\nOverlaid text indicates FDR, color indicateas NES',size=50)
                title2 = db+title+', Metric '+metric+', NES and FDR, Part '+partName
                if parts==1:
                    title2 = db+title+', Metric '+metric+', NES and FDR'
                cg.fig.suptitle(title2,size=50);plt.title('NES Values',size=40)
                if save:
                    plt.savefig('Saved Images/'+title2+'.pdf',dpi=200,bbox_inches = "tight")
                    plt.close();saveLst.append('Saved Images/'+title2+'.pdf')
                else:
                    plt.show();print(title2)
    if save:
        gc.collect();__mergePDFs(saveLst,name='Saved Images/'+db+title+', '+metric,erase=True)
    return row, col
def __getPathLst(superDfNesIn,superDfFdrIn,removeBad):
    superDfNes = superDfNesIn.copy();superDfFdr = superDfFdrIn.copy();path2Lst=[];catLst=[]
    for path in superDfNes.index:
        splt = path.split('_');cat = splt[0].replace('MSigDB','')
        cat = cat.split('.all')[0]+'.all';catLst.append(cat);splt=splt[2:];pathName = '_'.join(splt);path2Lst.append(pathName)  
    superDfNes.index = path2Lst;superDfFdr.index = path2Lst;keepLst=superDfNes.index
    if removeBad:
        df = pd.read_csv('MsigDB/c2.cp.v7.2.symbols.gmt.tsv',sep='\t',index_col=0)
        c2can = list(df.columns);c2can = __goodGenes(c2can,superDfNes.index)
        df = pd.read_csv('MsigDB/c5.go.bp.v7.2.symbols.gmt.tsv',sep='\t',index_col=0)
        c5bp = list(df.columns);c5bp = __goodGenes(c5bp,superDfNes.index)
        df = pd.read_csv('MsigDB/h.all.v7.2.symbols.gmt.tsv',sep='\t',index_col=0)
        hall = list(df.columns);hall = __goodGenes(hall,superDfNes.index);keepLst=c2can+c5bp+hall
    superDfNes = superDfNes.loc[keepLst];superDfFdr = superDfFdr.loc[keepLst]
    return superDfNes, superDfFdr, catLst
def __fixOutliers(dfIn,scale=0.1,title='',thres=3,onlyDic=False,dicIn={}):
    prePure = dfIn.copy();totalSize=0;geneLst=[];runGenes = prePure.index
    if(onlyDic):
        runGenes = __totalRunGenes(dicIn);runGenes = __goodGenes(runGenes,prePure.index)
    for gene in runGenes:
        scale2=scale;fixLst=[];thresNum=0
        while(True):
            if(thresNum > thres):
                scale2 = scale2/2
            df = prePure.loc[gene];std35 = (df.mean()+3.5*df.std(),df.mean()-3.5*df.std())
            dfUp = df.loc[df>std35[0]];dfDown = df.loc[df<std35[1]];outSize = len(dfUp)+len(dfDown)
            if(outSize==0):
                break
            for elem in dfUp.index:
                curVal = prePure.loc[gene][elem]
                if(df.std()>1):
                    newVal=df.mean()+3*df.std()+(curVal*scale2)
                else:
                    newVal=df.mean()+3*df.std()+(curVal*scale2*df.std())
                prePure.loc[gene][elem]=newVal;fixLst.append(elem)
            for elem in dfDown.index:
                curVal = prePure.loc[gene][elem]
                if(df.std()>1):
                    newVal = df.mean() - 3*df.std() - (curVal*scale2)
                else:
                    newVal = df.mean() - 3*df.std() - (curVal*scale2*df.std())
                prePure.loc[gene][elem] = newVal;fixLst.append(elem)
            thresNum += 1
        fixLst = __noDupsLst(fixLst);geneLst.append((gene,len(fixLst)))
    return prePure, geneLst
def plotPureCorrConsis(placeLst,meth='spearman'):
    meth='spearman';dfLst=[];frst=True
    for place in tq(placeLst,desc='dfLst'):
        prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
        if(frst):
            genes = prePure.index;frst=False
        else:
            genes = __goodGenes(genes,prePure.index)
        prePure = __pureDf(prePure);dfLst.append((place,prePure))
    genes.insert(0,'Purity Scores')
    for path in tq(overAllDict,desc='Plotting'):
        out = pd.DataFrame();dic = overAllDict[path];genesPath = __runGenes(dic)
        genesPath.insert(0,'Purity Scores');genesPath = __goodGenes(genesPath,genes)
        for dfTup in dfLst:
            name = dfTup[0];df = dfTup[1];df2 = df.loc[genesPath]
            cor = df2.T.corr(method=meth)['Purity Scores'] ;out[name] = list(cor)
        out.index = genesPath;out.columns = getUniqueNames(out.columns)
        out = out.drop('Purity Scores');out.dropna(inplace=True);out = out.T
        sb.clustermap(out,cmap=cmapSpecial,metric='cosine',yticklabels=1,xticklabels=1)
        plt.title(path+' Correlation to Purity, '+meth);plt.show()
        print(path+' Correlation to Purity, '+meth)
    return 1
def __C2SplitGSEA(superDfNes2,superDfFdr2,useLst=False,lst=[],
                  keep=['KEGG','BIOCARTA','PID','REACTOME','ST','WP','SIG','SA']):
    keepLst=[];keepLst2=[];spltLst=[];lst1=[];lst2=[];i=0
    for row in superDfNes2.index:
        splt = row.split('_')[0]
        if splt in keep:
            keepLst.append(row)
            if useLst:
                lst1.append(lst[i])
        else:
            keepLst2.append(row)
            if useLst:
                lst2.append(lst[i])
        i+=1
    superDfNes3 = superDfNes2.loc[keepLst];superDfNes4 = superDfNes2.loc[keepLst2]
    superDfFdr3 = superDfFdr2.loc[keepLst];superDfFdr4 = superDfFdr2.loc[keepLst2]
    return (superDfNes3,superDfFdr3,lst1),(superDfNes4,superDfFdr4,lst2)
msigNameDict={};msigNameDict['h.all']='HallMark Gene Sets';msigNameDict['c2.all']='Curated Gene Sets'
msigNameDict['c3.all']='Regulatory target Gene Sets';msigNameDict['c4.all']='Computational Gene Sets'
msigNameDict['c5.all']='Ontology Gene Sets';msigNameDict['c6.all']='Oncogenic Signature Gene Sets'
msigNameDict['c8.all']='Cell Type Signature Gene Sets';msigNameDict['All']=' Wanted Gene Sets'
def __getOutliers(placeLst,fraction=False):
    frst=True
    for place in placeLst:
        if(fraction):
            prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
            fract = len(prePure.columns)
        with open(place+' Outliers.txt') as f:
            out=[]
            geneLst=[]
            numLst=[]
            for line in f:
                line = line.replace('(','')
                line = line.replace(')','')
                line = line.replace('\n','')
                lines = line.split(',')
                gene = lines[0]
                gene = gene.replace("'",'')
                num = int(lines[1])
                geneLst.append(gene)
                numLst.append(num)
        if(fraction):
            numLst = [number / fract for number in numLst]
        out = pd.DataFrame(numLst,geneLst,columns=[place])
        if(frst):
            df = out.copy()
            frst=False
        else:
            df = df.join(out)
    return df
def plotOutliers(placeLst,fraction=False):
    """plotOutliers:
                    Input:
                        placeLst, a list of cancer area which you want included in the heatmaps
                            no default
                        fraction, boolean telling if to plot percent of outliers in each cancer (True),
                                  or raw total number of outliers in each cancer (False)
                    Actions:
                        plots a heatmap for each superLst pathway of the number of outlier samples 
                        that were moved in each gene in that pathway 
                        (if the same sample was mvoed more than once, it is only counted as 1)
                    Returns:
                        On success return 1.
    """
    df = __getOutliers(placeLst,fraction)
    df.columns = getUniqueNames(df.columns)
    for path in tq(overAllDict,desc='Paths'):
        lst = overAllDict[path]
        genes = __runGenes(lst)
        genes = __goodGenes(genes,df.index)
        df2 = df.loc[genes]
        df2 = np.log2(df2+1)
        df2 = df2.T
        sb.clustermap(df2,cmap=cmapSpecial,row_cluster=False,yticklabels=1,xticklabels=1)
        if(fraction):
            title=path+', percent of Samples Outlier-Shifted for each Gene'
        else:
            title=path+', Log2 of Samples Outlier-Shifted for each Gene'
        plt.title(title)
        plt.show()
        print(title)
    return 1
def __getCorrectionVersion(*args,place,fixOutliers=True):
    dfLst=[];nameLst=[]
    for num in args:
        if(num==0):
            if(fixOutliers):
                df = pd.read_csv('Purity/TPM/Outliers Fixed/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
            else:
                df = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t');__renameTPM(df)
                df = np.log2(df+1);df = pd.DataFrame(stats.zscore(df,axis=1),df.index,columns=df.columns)
                pDf = __pureDf(df);df = df[pDf.columns]
            df.dropna(inplace=True)
        else:
            direct = 'Purity/TPM/v'+str(num)+'/'
            df = pd.read_csv(direct+'TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t');pDf = __pureDf(df);df = df[pDf.columns]
        if(num==0):name = 'Per-Purity'
        else:name = 'Version '+str(num)
        dfLst.append(df);nameLst.append(name)
    return dfLst,nameLst

In [8]:
def __heatDF(dfIn,insert=0,zscore=False):
    df = dfIn.copy();__renameTPM(df)
    hot, cold = __getHotCold(dfIn);candLst=['CD8A','GZMA','GZMB','PRF1']
    df2 = dfIn.loc[candLst];mean = df2.mean()
    if(zscore):mean = pd.Series(stats.zscore(mean),index=mean.index)
    df = df.T;df.insert(insert,'Heat Scores',mean);df = df.T
    return df, hot, cold 
def shareData(palceLst,dicIn,directory,outName=''):
    lst=[]
    for path in dicIn:
        dic = dicIn[path];genes = __runGenes(dic);lst.extend(genes)
    for place in tq(palceLst,desc='Share Data'):
        df = pd.read_csv('Purity/TPM/'+directory+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        genes = __goodGenes(lst,df.index);df = df.loc[genes];df, hot, cold = __heatDF(df,place,len(df.index));hcDeg=[]
        for col in df:
            if(col in hot.columns):hcDeg.append(1)
            else:
                if(col in cold.columns):hcDeg.append(2)
                else:hcDeg.append(0)
        df = df.T;df['HC Designation, 1-Hot, 2-Cold, Neither-0'] = hcDeg
        df = df.T;df.to_csv('Share Data/'+place+outName+'_HeatScoreAndDesignation.csv')
    return 1
def masterPlotPurity(placeLst,*args,dic,allPath=False,onlySkinny=False,noSkinnys=False,
                     size=(30,10),font=1.5,meth='spearman',corrNum=0,regLine=False,regLine2=False,
                     regLineNum=0,regLineNum2=1,corThres=0.1,corLimit=False,fixOutliers=True,skip=[],save=False):
    start=time.time()
    if(save):
        for img in glob.glob('Saved Images/*'):os.remove(img)
    for place in placeLst:
        __masterPlotPurityHelp(place,*args,dic=dic,allPath=allPath,onlySkinny=onlySkinny,noSkinnys=noSkinnys,
                               size=size,font=font,meth=meth,corrNum=corrNum,regLine=regLine,regLine2=regLine2,
                               regLineNum=regLineNum,regLineNum2=regLineNum2,corThres=corThres,
                               corLimit=corLimit,fixOutliers=fixOutliers,skip=skip,save=save)
    end=time.time();total = (end-start)/60
    return total
def __masterPlotPurityHelp(place,*args,dic,allPath=False,onlySkinny=False,noSkinnys=False,
                           size=(30,10),font=1.5,meth='spearman',corrNum=0,regLine=False,
                           regLine2=False,regLineNum=0,regLineNum2=1,corThres=0.1,corLimit=False,
                           fixOutliers=True,skip=[],save=False):
    if not place in skip:
        dfLst, nameLst = __getCorrectionVersion(*args,place=place,fixOutliers=fixOutliers);nameLst2=[]
        for name in nameLst:
            name=name.split('.2')[0]+', Skinny Corrected';nameLst2.append(name)
        __plotPurityCorrection2(dfLst,nameLst2,dicIn=dic,allPath=allPath,onlySkinny=onlySkinny,
                                size=size,font=font,place=place,meth=meth,noSkinnys=noSkinnys,corrNum=corrNum,
                                regLine=regLine,regLine2=regLine2,corThres=corThres,corLimit=corLimit,save=save,
                                regLineNum=regLineNum,regLineNum2=regLineNum2)
    return 1

In [9]:
def __plotPurityCorrection2(dfLst,nameLst,dicIn,allPath=False,onlySkinny=False,noSkinnys=False,
                            size=(15,5),font=1.5,place='',meth='spearman',corrNum=0,diffCorr=False,
                            corrDict={},regLine=True,regLineNum=0,regLine2=False,regLineNum2=1,
                            corThres=0.1,corLimit=False,save=False):
    dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
    dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
    __renameTPM(dfHeat)
    hot,cold = __getHotCold(dfHeat)
    notHotCold = __notHotOrCold(dfHeat,hot,cold)
    inputs = dicIn
    out = Parallel(n_jobs=3)(delayed(__plotPurityCorrection2Help)
                                     (path,hot,cold,notHotCold,dicIn,dfLst,nameLst,allPath,onlySkinny,noSkinnys,size,font,place,
                                      meth,corrNum,diffCorr,corrDict,regLine,regLineNum,
                                      regLine2,regLineNum2,corThres,corLimit,save) for path in inputs)
    return out
def __plotPurityCorrection2Help(path,hot,cold,notHotCold,dicIn,dfLst,nameLst,allPath,onlySkinny,noSkinnys=False,
                                size=(15,5),font=1.5,place='',meth='spearman',corrNum=0,diffCorr=False,
                                corrDict={},regLine=True,regLineNum=0,regLine2=False,regLineNum2=1,
                                corThres=0.1,corLimit=False,save=False):
    sb.set(font_scale=font)
    title=''
    colors = ['white','gray','orange','purple','aqua','yellow','brown','pink','navy','maroon']
    if(allPath):
        fig, axes = plt.subplots(nrows=round(len(dfLst)/2), ncols=2, figsize=size)
        ax1 = axes[0]
        ax2 = axes[1]
        fig.tight_layout(pad=3)
    df2Lst=[]
    dic = dicIn[path]
    genes = __runGenes(dic)
    for df in dfLst:
        genes = __goodGenes(genes,df.index)
        dfPath = df.loc[genes]
        colorNum=0
        movedColor=False
        colorsLen = len(colors)
        pDf = __pureDf(dfPath)
        goodHotCols = __goodGenes(hot.columns,pDf.columns)
        goodColdCols = __goodGenes(cold.columns,pDf.columns)
        goodNCols = __goodGenes(notHotCold.columns,pDf.columns)
        cold = df[goodColdCols]
        hot = df[goodHotCols]
        notHotCold = df[goodNCols]
        hotX = pDf[goodHotCols].loc['Purity Scores']
        hotXInd = __noDupsLst(hotX.index)
        hotX = hotX[hotXInd]
        coldX = pDf[goodColdCols].loc['Purity Scores']
        coldXInd = __noDupsLst(coldX.index)
        coldX = coldX[coldXInd]
        notX = pDf[goodNCols].loc['Purity Scores']
        notXInd = __noDupsLst(notX.index)
        notX = notX[notXInd]
        df1Cor = pDf.T.corr(method=meth)['Purity Scores']
        df2Lst.append((df,pDf,df1Cor,hot,cold,notHotCold,hotX,coldX,notX))
    for gene in genes:
        printName=False
        if not(gene=='Purity Scores'):
            dfNum=0
            for dfTup in df2Lst:
                df = dfTup[0]
                pDf = dfTup[1]
                df1Cor = dfTup[2]
                hot=dfTup[3]
                cold=dfTup[4]
                notHotCold=dfTup[5]
                hotX = dfTup[6]
                coldX = dfTup[7]
                notX = dfTup[8]
                x = pDf.loc['Purity Scores']    
                y = pDf.loc[gene]
                geneCor = df1Cor.loc[gene]
                residual=True
                if(len(y.index)==0 or len(y.values)==0):
                    residual = False
                    break
                if(onlySkinny and not __skinyGene(y) and dfNum==0):
                    residual = False
                    break
                if(noSkinnys and __skinyGene(y) and dfNum==0):
                    residual = False
                    break
                if(corLimit and abs(geneCor)<(corThres) and dfNum==0):
                    residual = False
                    break
                if(dfNum==0):
                    fig, axes = plt.subplots(nrows=round(len(dfLst)/2), ncols=2, figsize=size)
                    fig.tight_layout(pad=3)
                    printName=True
                if(allPath):
                    if(residual):
                        if(max(y)>5 or min(y)<-5):
                            colorVal = colorNum % colorsLen;colorNum += 1
                            movedColor=True;color = colors[colorVal];ax1.plot(x,y,'o',label=gene,color=color)
                        else:
                            hotY = hot.loc[gene];coldY = cold.loc[gene];notY = notHotCold.loc[gene]
                            ax1.plot(hotX,hotY,'o',color='red',label='')
                            ax1.plot(coldX,coldY,'o',color='blue',label='');ax1.plot(notX,notY,'o',color='black',label='')
                        if(max(residual)>5 or min(residual)<-5):
                            if not(movedColor):
                                colorVal = colorNum % colorsLen;colorNum += 1;color = colors[colorVal]
                            movedColor=False;ax2.plot(x,residual,'o',label=gene,color=color)
                        else:
                            hotRes = residual.loc[goodHotCols];coldRes = residual.loc[goodColdCols]
                            notRes = residual.loc[goodNCols];ax2.plot(hotX,hotRes,'o',color='red',label='')
                            ax2.plot(coldX,coldRes,'o',color='blue',label='');ax2.plot(notX,notRes,'o',color='black',label='')
            
                else:
                    if(residual):
                        geneCor = df1Cor.loc[gene];col = int(dfNum/2);row = dfNum%2
                        if((len(dfLst)/2)<=1):ax = axes[row]
                        else:ax = axes[col][row]
                        ax.set_facecolor('white');ax.spines["top"].set_color('black');ax.spines["bottom"].set_color('black')
                        ax.spines["right"].set_color('black');ax.spines["left"].set_color('black')
                        ax.spines["top"].set_lw(4);ax.spines["bottom"].set_lw(4);ax.spines["right"].set_lw(4)
                        ax.spines["left"].set_lw(4);hotY = hot.loc[gene];hotYInd = __noDupsLst(hotY.index)
                        hotY = hotY[hotYInd];coldY = cold.loc[gene];coldYInd = __noDupsLst(coldY.index)
                        coldY = coldY[coldYInd];notY = notHotCold.loc[gene];notYInd = __noDupsLst(notY.index)
                        notY = notY[notYInd];ax.plot(hotX,hotY,'o',label='Hot',color='red')
                        ax.plot(coldX,coldY,'o',label='Cold',color='blue');
                        ax.plot(notX,notY,'o',label='Neither',color='black')
                        if(regLine and regLineNum==dfNum):
                            try:m, b = np.polyfit(x, y, 1)
                            except:m, b = np.polyfit(x, y, 1)
                            reggressionLine = m*x + b;ax.plot(x,reggressionLine,label='Reg Line',color='black')
                        if(regLine2 and regLineNum2==dfNum):
                            minArea = y.min() + 0.4;y2 = y[y > minArea];y2Ind = __noDupsLst(y2.index)
                            y2Ind = __goodGenes(y2Ind,x.index);y2 = y2[y2Ind];y2.dropna(inplace=True);x2 = x.loc[y2.index]
                            try:m, b = np.polyfit(x2, y2, 1)
                            except:m, b = np.polyfit(x2, y2, 1)
                            reggressionLine = m*x + b;ax.plot(x,reggressionLine,label='Reg Line',color='black')
                        ax.set_xlim([min(min(hotX),min(coldX),min(notX))-0.025,1.025]);ax.set_ylim(-2.5,4)
                        ax.set_xlabel('Purity Score');ax.set_ylabel('ZScore')
                        if(dfNum==0):
                            ax.legend(loc=(1,1.1));title = place+', '+path+', '+gene+', '+nameLst[dfNum]
                        else:title = nameLst[dfNum]
                        if __skinyGene(y) and dfNum==0:title = title+', Is Skinny'
                        if(regLine and dfNum == regLineNum):title = title+', Reg Line for v8'
                        if(regLine2 and dfNum == regLineNum2):title = title+', Reg Line for v7'
                        title2 = title
                        if(diffCorr):
                            if(dfNum>1):title2 = title+', Corr to other Version = '+str(round(corrDict[gene],3))
                        ax.set_title(title2)
                dfNum =dfNum+1
            if(printName):
                title = place+', '+path+', '+gene+', '+meth
                if(onlySkinny):
                    title = title+', Only Skinny'
                if(noSkinnys):
                    title = title+', No Skinnys'
                if(save):
                    plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
                plt.show();print(title)
    if(allPath):
        ax1.set_xlabel('Purity Score');ax1.set_ylabel('ZScore');ax1.set_title(place+', '+path+', Pre-Purity')
        ax2.legend(ncol=6,loc=(-1,-0.6));ax2.set_xlabel('Purity Score')
        ax2.set_ylabel('ZScore');ax2.set_title(place+', '+path+', Post-Purity');title = place+', '+path+', All Genes'
        if(save):
            plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
        plt.show();print(title)
    return 1

In [10]:
def __removeBadPlace(df,thres=0.9):
    out=[]
    for gene in df.index:
        df2 = df.loc[gene];zeros = list(df2).count(0);total = len(list(df2));percent = zeros/total
        if(percent <= thres):out.append(gene)
    dfOut = df.loc[out]
    return dfOut

def plotHeatCorr(placeLst,directory='All But Skinny Corrected, Shift Zeros',meth='spearman',geneSets=True
                ,badCorr=False,negCorr=False,Dict={}):
    """
    plotHeatCorr:
                Inputs:
                        placeLst, list of cancer areas you want plotted
                            no default
                        directory, directory where the purity correction version you want plotted is
                            default = 'All But Skinny Corrected, Shift Zeros'
                        meth, the method for correlation to heat
                            default = 'spearman'
                        geneSets, badCorr and negCorr, 3 booleans telling what to highlight on top of the plot
                                                       which genesets the gene came from (geneSets=True)
                                                       if it was skinny (badCorr=True)
                                                       or if it wad negativley correlated with purity (negCorr=True)
                            default = geneSets(True), badCorr(False), negCorr(False)
                        Dict, the dictionary for the genes in the superLst
                            default is overAllDict
    """
    dfLst =[]
    num=0
    colorLst = sb.color_palette("Set1",n_colors=len(placeLst))
    for place in tq(placeLst,desc='dfLst'):
        color = colorLst[num]
        num += 1
        filename = 'Purity/TPM/'+directory+'/TCGA_'+place+'_tpm.tsv'
        prePure = pd.read_csv(filename,index_col=0,sep='\t')
        prePureReal = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
        __renameTPM(prePureReal)
        __renameTPM(prePure)
        df, hot, cold = __heatDF(prePure,place)
        prePure = prePure[df.columns]
        prePureReal = prePureReal[df.columns]
        df = df.loc['Heat Scores']
        dfLst.append((df,prePure,color,prePureReal))
        
    for path in tq(Dict,desc='Plot'):
        colColrs=[]
        badNumLst=[]
        pathDict = Dict[path]
        genes = __runGenes(pathDict)
        for dfTup in dfLst:
            prePure = dfTup[1]
            prePureReal = dfTup[3]
            goodPathGSEA = __goodGenes(genes,prePure.index)
            cor = prePure.loc[goodPathGSEA].T.corr(method=meth)
            color = dfTup[2]
            badLst=[]
            if(badCorr):
                badLst = __getPurityLst(cor,0.5,'Down',0,percent=True)
            if(negCorr):
                badLst=[]
                pathGenes = __goodGenes(genes,prePureReal.index)
                prePure = prePureReal.loc[pathGenes]
                pDf = __pureDf(prePure)
                x = pDf.loc['Purity Scores']
                for gene in pDf.index:
                    if not(gene=='Purity Scores'):
                        y = pDf.loc[gene]
                        m, b = np.polyfit(x, y, 1)
                        if(m<=lessThan):
                            badLst.append(gene)
            badLst = __goodGenes(badLst,genes)
            badDf = __getBads(genes,badLst)
            badNum = len(badLst)
            badNumLst.append(badNum)
            badColor = __getRowColors(badDf,color,'Removed')
            colColrs.append(badColor)
        if(geneSets):
            colColrs=[]
            num=0
            badNumLst=[]
            for db in pathDict:
                color = colorLst[num]
                num +=1
                badLst=[]
                badLst = pathDict[db]
                badDf = __getBads(genes,badLst)
                badNumLst.append(len(badLst))
                badColor = __getRowColors(badDf,color,'Removed')
                colColrs.append(badColor)
        out = pd.DataFrame()
        
        for gene in genes:
            corLst=[]
            if not(gene=='Heat Score'):
                for dfTup in dfLst:
                    df = dfTup[0]
                    df2 = dfTup[1]
                    cor = np.nan
                    if (gene in df2.index):
                        y = df
                        x = df2.loc[gene]
                        if(meth=='spearman'):
                            cor = stats.spearmanr(x, y)[0]
                        if(meth=='pearson'):
                            cor = stats.pearsonr(x, y)[0]
                    corLst.append(cor)
                    
            out[gene] = corLst
        out.index = placeLst
        out.index.name = path
        out = out.T.dropna().T
        cg = sb.clustermap(out,cmap=cmapSpecial,vmin=-1,vmax=1,col_colors=colColrs,
                           yticklabels=1,xticklabels=1)
        if (geneSets):
            placeLst1=[]
            for num in range(len(list(pathDict.keys()))):
                key = list(pathDict.keys())[num]
                badLen = badNumLst[num]
                placeLst1.append(key+' total genes in db: '+str(badLen))
        else:
            placeLst1=[]
            for num in range(len(placeLst)):
                p = placeLst[num]
                l = badNumLst[num]
                p1 = p+' '+str(l)
                placeLst1.append(p1)
        for x,y in zip(placeLst1, colorLst):
            cg.ax_heatmap.plot(0, 0, color=y, label=x, solid_capstyle="butt")
    
        cg.ax_heatmap.set_xlabel(directory)
        cg.ax_heatmap.legend(loc=(1,1.1), title=directory)
        title = "Correlation to Heat, "+path+', '+meth+', Number of genes = '+str(len(out.columns))+', '+directory
        plt.title(title)
        plt.show()
        print(title)   
    return 1

def __getGSEALst(cls):
    f = open(cls)
    n=1
    for line in f:
        if(n==3):
            lst = line.split('\t')
        n += 1
    return lst

def runGSEA(placeLst,useEveryDict=True,outDir='GSEA Output',inDir='GSEA Input',dbLstIn=[],preRank=False,permNum=1000,
            minSize=1,permType='phenotype',method='signal_to_noise',no_plot=True,title='',processes=4,overRide=False,
            version='v7'):
    """runGsea
                Input:
                    palceLst: list of cancer areas you want to run gsea on 
                             ,each place needs to have expression and phenotype files (as per GSEA requirments)
                    outDir: main directory in which to save the results 
                            (program will also create directories for date/db/place in order to save all results seperatly)
                        default = GSEA Output
                    inDir: main directroy where the input files are located (expression/phenotype/gene sets)
                        default = 'GSEA Input/'
                    title: name given to the outputed directory (other than the current date)
    """
    from datetime import date as dateToday
    date = str(dateToday.today())
    outdir1=outDir+'/'+date+title
    if not os.path.exists(outdir1):
        os.makedirs(outdir1)
    if useEveryDict:
        dbLst = list(everyDict['T-effector'].keys())
    else:
        dbLst = dbLstIn.copy()
    for db in tq(dbLst,desc='Running GSEA'):
        outdir2=outdir1+'/'+db
        if not os.path.exists(outdir2):
            os.makedirs(outdir2)
        inputs = placeLst
        out = Parallel(n_jobs=5)(delayed(__helpRunGSEA)(db,inDir,outdir2,place,preRank,processes,minSize,no_plot,
                                                        permNum,permType,method,overRide,version) for place in inputs)
    return 1
def __helpRunGSEA(db,inDir,outdir2,place,preRank,processes,minSize,no_plot,permNum,permType,method,overRide,version='v7'):
    gene_sets=inDir+'/'+'GeneSets/'+db+'_GeneSet.gmt'
    outdir3=outdir2+'/'+place
    if not os.path.exists(outdir3):
        os.makedirs(outdir3)
        new=True
    else:
        new=False
    if new or overRide:
        data=inDir+'/'+place+'/PostPurity/'+place+'GeneExpressionPostPurity.txt'
        cls=inDir+'/'+place+'/PostPurity/'+place+'PostPurityPhenotypeLabels.cls'
        lst = __getGSEALst(cls)
        rnkFile = inDir+'/Ranks/'+place+'/'+version+'.rnk'
        if preRank:
            GSEA.prerank(rnk=rnkFile,gene_sets=gene_sets,outdir=outdir3,processes=processes,
                         min_size=minSize,no_plot=no_plot,permutation_num=permNum)
        else:
            out = GSEA.gsea(data=data,gene_sets=gene_sets,cls=lst,outdir=outdir3,processes=processes,
                            permutation_num=permNum,min_size=minSize,permutation_type=permType,
                            method=method,no_plot=no_plot)
    return 1
def plotRegDiffGSEA(date1,date2,placeLst,desktop=False,skipImmune=False,allPath=False,onlySkinny=False,
                onlyOutliers=False,size=(30,10),font=1.5,meth='spearman',preRank=False,
                noSkinnys=False,corrNum=0,regLine=True,regLineNum=0,regLine2=False,regLineNum2=1,corThres=0.1,save=False):
    dbLst=[]
    for gsea in glob.glob('GSEA Output/'+date1+'/*'):
        dbLst.append(gsea.split('\\')[-1])
    superDfNes1,superDfFdr1 = __superDfGSEA(date1,dbLst,placeLst,desktop=desktop,skipImmune=skipImmune,preRank=preRank)
    superDfNes2,superDfFdr2 = __superDfGSEA(date2,dbLst,placeLst,desktop=desktop,skipImmune=skipImmune,preRank=preRank)
    totalSize = len(superDfNes1.index)*len(superDfNes1.columns)
    print('Total # of Comparisons = '+str(totalSize))
    diffLst=[]
    bigDiffLst=[]
    for idx in superDfNes1.index:
        for col in superDfNes1.columns:
            elem = superDfNes1.loc[idx][col]
            elem2 = superDfNes2.loc[idx][col]
            if(np.sign(elem) != np.sign(elem2)):
                diffLst.append((col.replace('K_',''),idx))
                if(abs(elem-elem2)>=0.3):
                    bigDiffLst.append((col.replace('K_',''),idx))
    pLst=[]
    for test in bigDiffLst:
        place = test[0]
        if not(place in pLst):
            pLst.append(place)
    fullLst={}
    for p in placeLst:
        for i in pLst:
            if(i in p):
                fullLst[i] = p
    dictIn = everyDict.copy()
    numDiffPlaceLst=[]
    numDiffPathLst=[]
    for test in tq(bigDiffLst,desc='plot'):
        place = test[0]
        path = test[1]
        place = fullLst[place]
        path1 = path.split('_')
        pathDict={}
        for path2 in everyDict:
            if(path2 == path1[1]):
                dic2 = dictIn[path2]
                inDict={} 
                for d in dic2:
                    if(d==path1[0]):
                        inDict[d]=dic2[d].copy()
                        pathDict[path] = inDict
        path = test[1]
        numDiffPlaceLst.append(place)
        numDiffPathLst.append(path)
        num1 = date1.split()[1].split('v')[1]
        num2 = date2.split()[1].split('v')[1]
        dfLst, nameLst = __getCorrectionVersion(0,0,num1,num2,place=place,fixOutliers=True)
        __plotPurityCorrection2(dfLst,nameLst,[],pathDict,allPath=allPath,
                                onlySkinny=onlySkinny,onlyOutliers=onlyOutliers,
                                size=size,font=font,place=place,meth=meth,noSkinnys=noSkinnys,corrNum=corrNum,
                                regLine=regLine,regLineNum=regLineNum,regLine2=regLine2,
                                regLineNum2=regLineNum2,corThres=corThres,save=save)
        
    DiffPlace={}
    for place in numDiffPlaceLst:
        num = numDiffPlaceLst.count(place)
        DiffPlace[place] = num
    DiffPath={}   
    for path in numDiffPathLst:
        num = numDiffPathLst.count(path)
        DiffPath[path] = num
    return DiffPlace, DiffPath
def __getSuperLst(dic):
    superLst=[]
    for path in dic:
        lst = __runGenes(dic[path])
        for elem in lst:
            if not elem in superLst:
                superLst.append(elem)
    return superLst
def plotGeneCorr(placeLst,dicIn,version=7,meth='spearman',metric='cosine',fixPos=False,pos={},
               hots=False,colds=False,diff=False):
    outDic={}
    dfLst=[]
    for place in tq(placeLst,desc='dfLst'):
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        if(hots or colds or diff):
            dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
            dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
            __renameTPM(dfHeat)
            hot,cold = __getHotCold(dfHeat)
            hotCols = __goodGenes(hot.columns,df.columns)
            coldCols = __goodGenes(cold.columns,df.columns)
            dfLst.append((df,place,hotCols,coldCols))
        else:
            dfLst.append((df,place))
    for path in tq(dicIn,desc='Gene-Gene Corr'):
        for dfTup in dfLst:
            df = dfTup[0]
            place = dfTup[1]
            if(hots or colds or diff):
                hotCols = dfTup[2]
                coldCols = dfTup[3]
            
            dic2 = dicIn[path]
            genes = __runGenes(dic2)
            genes = __goodGenes(genes,df.index)
            dfPath1 = df.loc[genes]
            dfPath2 = dfPath1.copy()
            if(hots):
                dfPath2 = dfPath2[hotCols]
            if(colds):
                dfPath2 = dfPath2[coldCols]
            corr = dfPath2.T.corr(method=meth)
            if(diff):
                dfPathH = dfPath1[hotCols]
                dfPathC = dfPath1[coldCols]
                corrH = dfPathH.T.corr(method=meth)
                corrC = dfPathC.T.corr(method=meth)
                corr = corrH - corrC
            cClus=not(fixPos)
            rClus=not(fixPos)
            if(fixPos):
                tup = pos[place+'_'+path]
                row = tup[0]
                col = tup[1]
                corr = __setCluster(corr,row,col)
            cg = sb.clustermap(corr,metric=metric,cmap=cmapSpecial,vmin=-1,vmax=1,yticklabels=0,xticklabels=0,
                              col_cluster=cClus,row_cluster=rClus)
            if not(fixPos):
                row = cg.dendrogram_row.reordered_ind
                col = cg.dendrogram_col.reordered_ind
            title ='Gen to gene Corr, '
            if(hots):
                title += 'Only Hots, '
            if(colds):
                title += 'Only Colds, '
            if(diff):
                title += 'Diff of Hot and Cold, '
            plt.title(title+place+', '+path+', '+meth+', '+metric+', version '+str(version))
            plt.show()
            print(title+place+', '+path+', '+meth+', '+metric+', version '+str(version))
            outDic[place+'_'+path] = (row,col)
    return outDic

def plotPairsCorr(placeLst,dicIn,version=7,meth='spearman',Num=10,fontSize=10,save=True):
    dfLst=[]
    for place in placeLst:
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        dfLst.append((df,place))

    for path in tq(dicIn,desc='Pairs Corr'):
        dic2 = dicIn[path]
        genes = __runGenes(dic2)
        for dfTup in dfLst:
            df = dfTup[0]
            placeName = dfTup[1]
        
            genesP = __goodGenes(genes,df.index)
            dfPath = df.loc[genesP]
            corr = dfPath.T.corr(method=meth)
            for i in range(len(corr.index)):
                corr.iloc[i][i] = 0
            topPairs = __topN(corr,Num)
        
            corPLst=[]
            for dfTup2 in dfLst:
                df2 = dfTup2[0]
                genesP = __goodGenes(genes,df2.index)
                dfPath = df2.loc[genesP]
                corr = dfPath.T.corr(method=meth)
                corLst=[]
                nameLst=[]
                for tup in topPairs:
                    g1 = tup[0]
                    g2 = tup[1]
                    Name = tup[2]
                    corV = corr.loc[g1][g2]
                    corLst.append(corV)
                    nameLst.append(Name)
                corPLst.append(corLst)
        
            out = pd.DataFrame(corPLst,placeLst,columns=nameLst)
            cg = sb.heatmap(out,cmap=cmapSpecial,yticklabels=1,xticklabels=1)
            plt.setp(cg.get_xticklabels(),rotation=45,ha='right',size=fontSize)
            plt.title('Top '+str(Num)+' Correlated Gene Pairs, for '+placeName+', '+path+', '+meth+' version '+str(version))
            plt.show()
            print('Top '+str(Num)+' Correlated Gene Pairs, for '+placeName+', '+path+', '+meth+' version '+str(version))
    return 1

def plotKobeGGCorr(placeLst,Dict,version=7,meth='spearman',metric='cosine',save=True,onlyKobe=False,
                 fontSize=12,thres=10,hots=False,colds=False,diff=False,fixPos=False,pos={},Path2Path=True):
    if(save):
        for img in glob.glob('Saved Images/*'):
            os.remove(img)
    frstDic = list(Dict.keys())[0]
    dbLst=[]
    if(onlyKobe):
        for db in Dict[frstDic]:
            dbLst.append(db)
        kobeDict = __getDBList(Dict,'KOBE')
    else:
        kobeDict = Dict.copy()

    dfLst=[]
    for place in tq(placeLst,desc='dfLst'):
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        
        if(hots or colds or diff):
            dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
            dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
            __renameTPM(dfHeat)
            hot,cold = __getHotCold(dfHeat)
            hotCols = __goodGenes(hot.columns,df.columns)
            coldCols = __goodGenes(cold.columns,df.columns)
            dfLst.append((df,place,hotCols,coldCols))
        else:
            dfLst.append((df,place))
    
    kobeLst={}
    for path in kobeDict:
        dic2 = kobeDict[path]
        lst = __runGenes(dic2)
        kobeLst[path]=lst
    if(Path2Path):
        outDic = __plotKobeGGCorr1(kobeLst,dfLst,placeLst,hots,colds,
                                   diff,version,metric,meth,fixPos,pos,thres,fontSize,save,onlyKobe)
    else:
        outDic = __plotKobeGGCorr2(kobeLst,dfLst,placeLst,hots,colds,
                                   diff,version,metric,meth,fixPos,pos,thres,fontSize,save,onlyKobe)
    return outDic

def __plotKobeGGCorr1(kobeLst,dfLst,placeLst,hots,colds,diff,version,metric,meth,fixPos,pos,thres,fontSize,save,onlyKobe):
    outDic={}
    runLst = list(kobeLst.keys())
    for pathNum in tq(range(len(runLst)),desc='Gene Pair Corr'):
        path1 = runLst[pathNum]
        for path2Num in range(pathNum,len(runLst)):
            path2 = runLst[path2Num]
            if path2 != path1:
                lst1 = kobeLst[path1]
                lst2 = kobeLst[path2]
                for dfTup in dfLst:
                    df = dfTup[0]
                    lst1 = __goodGenes(lst1,df.index)
                    lst2 = __goodGenes(lst2,df.index)
                corPLst=[]
                for dfTup in dfLst:
                    df = dfTup[0]
                    if(hots or colds or diff):
                        hotCols = dfTup[2]
                        coldCols = dfTup[3]
                    corrLst=[]
                    nameLst=[]
                    for elem1 in lst1:
                        for elem2 in lst2:
                            ser1 = df.loc[elem1]
                            ser2 = df.loc[elem2]
                            corr = ser1.corr(ser2,method=meth)
                            if(hots):
                                serH1 = ser1[hotCols]
                                serH2 = ser2[hotCols]
                                corrH = serH1.corr(serH2,method=meth)
                                corr = corrH
                            if(colds):
                                serC1 = ser1[coldCols]
                                serC2 = ser2[coldCols]
                                corrC = serC1.corr(serC2,method=meth)
                                corr = corrC
                            corrLst.append(corr)
                            nameLst.append(elem1+', '+elem2)
                    corPLst.append(corrLst)
                out = pd.DataFrame(corPLst,placeLst,columns=nameLst)
                out.index.name = 'Cancer Areas'
                __addKs(out,axis=0,lst=kobePlaceLst1,inplace=True)
                out.index = getUniqueNames(out.index)
                out = out.T
                out.index.name = 'Gene Pairings'
                
                cClus=not(fixPos)
                rClus=not(fixPos)
                if(fixPos):
                    tup = pos[path1+'_'+path2]
                    row = tup[0]
                    col = tup[1]
                    out = __setCluster(out,row,col)
                cg = sb.clustermap(out,cmap=cmapSpecial,xticklabels=1,yticklabels=1,metric=metric,
                                  col_cluster=cClus,row_cluster=rClus)
                if not(fixPos):
                    row = cg.dendrogram_row.reordered_ind
                    col = cg.dendrogram_col.reordered_ind
                
                addOn = len(cg.ax_heatmap.get_yticklabels()) / thres
                newFont = fontSize - addOn
                plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom',size=newFont)
                title = path1+' to '+path2+', '
                if(onlyKobe):
                    title += 'Kobe Database Gene Correlation, '
                else:
                    title += 'Pathway Gene Gene Correlation'
                if(hots):
                    title += 'Only Hot Samples, '
                if(colds):
                    title += 'Only Cold Samples, '
                if(diff):
                    title += 'Diff of Hot Corr and Cold Corr, '
                title = title+meth+', '+metric+', version '+str(version)
                plt.title(title)
                #plt.tight_layout()
                if(save):
                    plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
                plt.show()    
                print(title)
                outDic[path1+'_'+path2] = (row,col)
    return outDic

def __plotKobeGGCorr2(kobeLst,dfLst,placeLst,hots,colds,diff,version,metric,meth,fixPos,pos,thres,fontSize,save,onlyKobe):
    outDic={};runLst = list(kobeLst.keys())
    for pathNum in tq(range(len(runLst)),desc='Gene Pair Corr'):
        path1 = runLst[pathNum];lst1 = kobeLst[path1]
        for dfTup in dfLst:
            df = dfTup[0];lst1 = __goodGenes(lst1,df.index)
        corPLst=[]
        for dfTup in tq(dfLst,desc='places for '+path1):
            df = dfTup[0]
            if(hots or colds or diff):
                hotCols = dfTup[2];coldCols = dfTup[3]
            corrLst=[];nameLst=[]
            for num1 in range(len(lst1)):
                elem1 = lst1[num1]
                for num2 in range(num1,len(lst1)):
                    elem2 = lst1[num2]
                    if not elem1 == elem2:
                        ser1 = df.loc[elem1];ser2 = df.loc[elem2];corr = ser1.corr(ser2,method=meth)
                        if(hots):
                            serH1 = ser1[hotCols];serH2 = ser2[hotCols]
                            corrH = serH1.corr(serH2,method=meth);corr = corrH
                        if(colds):
                            serC1 = ser1[coldCols];serC2 = ser2[coldCols]
                            corrC = serC1.corr(serC2,method=meth);corr = corrC
                        corrLst.append(corr);nameLst.append(elem1+', '+elem2)
            corPLst.append(corrLst)
        out = pd.DataFrame(corPLst,placeLst,columns=nameLst)
        out.index.name = 'Cancer Areas';__addKs(out,axis=0,lst=kobePlaceLst1,inplace=True)
        out.index = getUniqueNames(out.index);out = out.T;out.index.name = 'Gene Pairings';cClus=not(fixPos);rClus=not(fixPos)
        if(fixPos):
            tup = pos[path1+'_'+path2];row = tup[0];col = tup[1];out = __setCluster(out,row,col)
        cg = sb.clustermap(out,cmap=cmapSpecial,xticklabels=1,yticklabels=1,metric=metric,
                          col_cluster=cClus,row_cluster=rClus)
        if not(fixPos):
            row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
        addOn = len(cg.ax_heatmap.get_yticklabels())/thres;newFont = fontSize - addOn
        plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom',size=newFont);title = path1+', '
        if(onlyKobe):title += 'Kobe Database Gene Correlation, '
        else:title += 'Pathway Gene Gene Correlation'
        if(hots):title += 'Only Hot Samples, '
        if(colds):title += 'Only Cold Samples, '
        if(diff):title += 'Diff of Hot Corr and Cold Corr, '
        title = title+meth+', '+metric+', version '+str(version);plt.title(title)
        if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
        plt.show();print(title);outDic[path1] = (row,col)
    return outDic

def plotKobePPCorr(placeLst,Dict,meth='spearman',metric='cosine',ver=7,fixPos=False,pos=(),
                  hots=False,colds=False,diff=False,save=False,repeat=True):
    frstDic = list(Dict.keys())[0]
    dbLst=[]
    for db in Dict[frstDic]:
        dbLst.append(db)
    kobeDict = __getDBList(Dict,'KOBE')

    dfLst=[]
    for place in placeLst:
        df = pd.read_csv('Purity/TPM/v'+str(ver)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        
        if((hots or colds or diff) and not repeat):
            dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
            dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
            __renameTPM(dfHeat)
            hot,cold = __getHotCold(dfHeat)
            hotCols = __goodGenes(hot.columns,df.columns)
            coldCols = __goodGenes(cold.columns,df.columns)
            dfLst.append((df,place,hotCols,coldCols))
        else:
            dfLst.append((df,place))
    
    kobeLst={}
    for path in kobeDict:
        dic2 = kobeDict[path]
        lst = __runGenes(dic2)
        kobeLst[path]=lst

    runLst = list(kobeLst.keys())
    corPLst=[]
    nameLst=[]
    for pathNum in range(len(runLst)):
        path1 = runLst[pathNum]
        for path2Num in range(pathNum,len(runLst)):
            path2 = runLst[path2Num]
            if path2 != path1:
                lst1 = kobeLst[path1]
                lst2 = kobeLst[path2]
                for dfTup in dfLst:
                    df = dfTup[0]
                    lst1 = __goodGenes(lst1,df.index)
                    lst2 = __goodGenes(lst2,df.index)
                
                corrLst=[]
                for dfTup in dfLst:
                    df = dfTup[0]
                    if((hots or colds or diff) and not repeat):
                        hotCols = dfTup[2]
                        coldCols = dfTup[3]    
                    ser1 = df.loc[lst1].mean()
                    ser2 = df.loc[lst2].mean()
                    corr = ser1.corr(ser2,method=meth)
                    if(hots and not repeat):
                        serH1 = ser1[hotCols]
                        serH2 = ser2[hotCols]
                        corrH = serH1.corr(serH2,method=meth)
                        corr = corrH
                    if(colds and not repeat):
                        serC1 = ser1[coldCols];serC2 = ser2[coldCols]
                        corrC = serC1.corr(serC2,method=meth);corr = corrC
                    corrLst.append(corr)
                nameLst.append(path1+', '+path2);corPLst.append(corrLst)
    out = pd.DataFrame(corPLst,nameLst,columns=placeLst);out.index.name='Paths'
    __addKs(out,lst=kobePlaceLst1,inplace=True);out.columns = getUniqueNames(out.columns)
    out = out.T;out.index.name='Cancer Areas';out = out.T;cClus=not(fixPos);rClus=not(fixPos)
    if(fixPos):
        row, col = pos;out = __setCluster(out,row,col)
    cg=sb.clustermap(out,cmap=cmapSpecial,col_cluster=cClus,row_cluster=rClus,metric=metric,yticklabels=1,xticklabels=1)
    if not fixPos:
        row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
    title = 'Kobe Database Pathway Pathway Correlation, '
    if(hots and not repeat):title = title +'Only Hot Samples,'
    if(colds and not repeat):title = title+'Only Cold Samples, '
    title = title+' correction version '+str(ver)+', '+meth+', '+metric
    plt.title(title)
    if(save):
        plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
    plt.show()
    print(title)
    
    if(hots and repeat):
        plotKobePPCorr(placeLst,Dict,meth,metric,ver,fixPos=True,pos=(row,col),
                       hots=True,save=save,repeat=False)
    if(colds and repeat):
        plotKobePPCorr(placeLst,Dict,meth,metric,ver,fixPos=True,pos=(row,col),
                       colds=True,save=save,repeat=False)
    return 1
def plotGeneCorrAll(placeLst,dicIn,version=7,meth='spearman',metric='cosine',
                    hots=False,colds=False,diff=False,fixPos=False,pos={},indivPlaces=False,save=False):
    if(save):
        for img in glob.glob('Saved Images/*'):os.remove(img)
    outDic={}
    frst=True
    for place in tq(placeLst,desc='allDF'):
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        if(hots or colds or diff):
            dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
            dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t');__renameTPM(dfHeat);hot,cold = __getHotCold(dfHeat)
            hotCols = __goodGenes(hot.columns,df.columns);coldCols = __goodGenes(cold.columns,df.columns)
        if(diff):
            dfH = df[hotCols];dfC = df[coldCols];dfH.index.name='Genes';dfC.index.name='Genes'
        else:
            if(hots):df = df[hotCols]
            if(colds):df = df[coldCols]
            df.index.name='Genes'
        if(frst):
            if(diff):
                allDFh = dfH.copy();allDFc = dfC.copy()
            else:allDF = df.copy()
            frst=False
        else:
            if(diff):
                allDFh = allDFh.merge(dfH,on='Genes');allDFh = allDFc.merge(dfH,on='Genes')
            else:allDF = allDF.merge(df,on='Genes')
    if(indivPlaces):
        dfLst=[]
        for place in tq(placeLst,desc='dfLst'):
            df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
            if(hots or colds or diff):
                dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
                dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
                __renameTPM(dfHeat);hot,cold = __getHotCold(dfHeat)
                hotCols = __goodGenes(hot.columns,df.columns);coldCols = __goodGenes(cold.columns,df.columns)
            if(diff):dfH = df[hotCols];dfC = df[coldCols]
            else:
                if(hots):df = df[hotCols]
                if(colds):df = df[coldCols]
            dfLst.append((df,place))
        for path in tq(dicIn,desc='Gene Gene Corr'):    
            dic2 = dicIn[path];genes = __runGenes(dic2);genes = __goodGenes(genes,allDF.index)
            for dfTup in dfLst:
                df = dfTup[0];place = dfTup[1];genes = __goodGenes(genes,df.index);dfPath = df.loc[genes]
                corr = dfPath.T.corr(method=meth);cClus=not(fixPos);rClus=not(fixPos)
                if(fixPos):
                    row,col = pos[path];corr = __setCluster(corr,row,col)
                cg = sb.clustermap(corr,metric=metric,cmap=cmapSpecial,vmin=-1,vmax=1,yticklabels=0,xticklabels=0,
                                   col_cluster=cClus,row_cluster=rClus)
                if not(fixPos):
                    row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
                outDic[path] = (row,col)
                title = path+', '+meth+', '+metric+', version '+str(version)+', '
                title +='Gene to gene Corr '+place+', '
                if(hots):title += 'Only Hot Samples'
                if(colds):title += 'Only Cold Samples'
                plt.title(title)
                if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
                plt.show();print(title)
    else:        
        for path in tq(dicIn,desc='Gene gene Corr'):    
            dic2 = dicIn[path];genes = __runGenes(dic2)
            if(diff):
                genes = __goodGenes(__goodGenes(genes,allDFh.index),allDFc.index)
                dfPathH = allDFh.loc[genes];dfPathC = allDFc.loc[genes]
                corrH = dfPathH.T.corr(method=meth);corrC = dfPathC.T.corr(method=meth)
                corr = corrH - corrC
            else:
                genes = __goodGenes(genes,allDF.index);dfPath = allDF.loc[genes];corr = dfPath.T.corr(method=meth)
            
            cClus=not(fixPos);rClus=not(fixPos)
            if(fixPos):
                row,col = pos[path];corr = __setCluster(corr,row,col)
            cg = sb.clustermap(corr,metric=metric,cmap=cmapSpecial,yticklabels=0,xticklabels=0,
                               col_cluster=cClus,row_cluster=rClus,vmin=-1,vmax=1)
            if not(fixPos):
                row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
            outDic[path] = (row,col)
            title = path+', '+meth+', '+metric+', version '+str(version)+', ';title +='Gene to gene Corr All Cancers'
            if(diff):title += ', Diff of Hot Corr and Cold Corr'
            else:
                if(hots):title += ', Only Hot Samples'
                if(colds):title += ', Only Cold Samples'
            plt.title(title)
            if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
            plt.show();print(title)
    return outDic

def plotExpression(palceLst,Dict,version=7,save=False):
    if(save):
        for img in glob.glob('Saved Images/*'):os.remove(img)
    frstDic = list(Dict.keys())[0]
    dbLst=[]
    for db in Dict[frstDic]:dbLst.append(db)
    kobeDict = __getDBList(Dict,'KOBE')
    for place in tq(placeLst,desc='Plot Expression'):
        dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
        dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t')
        __renameTPM(dfHeat);hot,cold = __getHotCold(dfHeat)
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        hotCols = __goodGenes(hot.columns,df.columns);coldCols = __goodGenes(cold.columns,df.columns)
        hotCold = hotCols + coldCols;colData = __colDataFunc(hotCols,coldCols,groupName='Heat')
        col_palette = dict(zip(colData.Heat.unique(), ["Red","Blue"])) 
        col_colors1 = colData.Heat.map(col_palette);rowDict={};geneLst=[]
        for path in kobeDict:
            dic2 = kobeDict[path];genes = __runGenes(dic2);genes = __goodGenes(genes,df.index)
            for elem in genes:
                if not elem in geneLst:geneLst.append(elem)
            rowDict[path] = genes
        colorLst=['Red','Green','Blue','Black','Orange','Purple','Yellow','Gray','Brown'];num = 0;rowColorLst=[]
        for pathway in rowDict:
            geneLst2 = geneLst.copy()
            for elem in rowDict[pathway]:geneLst2.remove(elem)
            colData = __colDataFunc(rowDict[pathway],geneLst2,groupName=pathway);color = colorLst[num]
            col_palette = dict(zip(colData[pathway].unique(), [color,"White"]));rowColor = colData[pathway].map(col_palette)
            rowColor = rowColor.loc[geneLst];num += 1;rowColorLst.append(rowColor)
        clusDF = df[hotCold].loc[geneLst];clusDF.index.name = 'Genes'
        cg = sb.clustermap(clusDF,col_cluster=False,row_cluster=False, cmap=cmapSpecial,standard_scale=None,
              row_colors=rowColorLst,col_colors=col_colors1,yticklabels=1,xticklabels=0)
        for x,y in zip(list(rowDict.keys()), colorLst):
            cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=10, solid_capstyle="butt")
        cg.ax_heatmap.legend(loc=(-0.5,1), title="Pathway")
        title = place+', Gene Expression after Correction V'+str(version);plt.title(title)
        if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
        plt.show()
    return 1

def plotRegDiffExpr(placeLst,num1,num2,dicIn,allPath=False,onlySkinny=False,noSkinnys=False,onlyOutliers=False,
                    corrNum=2,regLine=True,regLineNum=0,regLine2=True,regLineNum2=1,corThres=0.1,size=(30,10),font=1.5,
                    meth='spearman',save=False):
    if(save):
        for img in glob.glob('Saved Images/*'):os.remove(img)
    for place in tq(placeLst,desc='Plot Diff'):
        dfLst, nameLst = __getCorrectionVersion(0,0,num1,num2,place=place,fixOutliers=True)
        v1 = dfLst[2].T;v2 = dfLst[3].T;corr = v2.corrwith(v1,method=meth)
        genes = list(corr[corr<0.9].index);corr2 = corr[genes][genes];corrDict = corr2.to_dict();dicIn2={}
        for path in dicIn:
            dicLst=[];lst = __runGenes(everyDict[path]);diffDic={}
            for g in genes:
                if g in lst:dicLst.append(g)
            diffDic['Diff'] = dicLst;dicIn2[path]=diffDic
        __plotPurityCorrection2(dfLst,nameLst,[],dicIn2,allPath=allPath,
                                onlySkinny=onlySkinny,onlyOutliers=onlyOutliers,
                                size=size,font=font,place=place,meth=meth,noSkinnys=noSkinnys,corrNum=corrNum,
                                regLine=regLine,regLineNum=regLineNum,regLine2=regLine2,
                                regLineNum2=regLineNum2,corThres=corThres,save=save,diffCorr=True,corrDict=corrDict)
    return 1

def __totalRunGenes(dicIn):
    totalLst=[]
    for path in dicIn:
        lst = __runGenes(dicIn[path])
        for l in lst:
            if not l in totalLst:totalLst.append(l)
    return totalLst

def countSkinnys(placeLst,dicIn):
    countLst=[];percentLst=[];genes = __totalRunGenes(dicIn)
    for place in tq(placeLst,desc='Counting Skinnies'):
        count = 0;dfHeatName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
        dfHeat = pd.read_csv(dfHeatName,index_col=0,sep='\t');__renameTPM(dfHeat);genes2 = __goodGenes(genes,dfHeat.index) 
        for g in genes2:
            y = dfHeat.loc[g]
            if(__skinyGene(y)):count += 1
        percent = round(count / len(genes2),3);percent=round(percent*100,1);countLst.append(count);percentLst.append(percent)
    dfOut = pd.DataFrame([countLst,percentLst],['Skinny Counts','Skinny Percent'],columns=placeLst).T
    newIdx = replaceLst(dfOut.index,'Breast Hormone','Breast HR+');dfOut.index = newIdx
    return dfOut

def replaceLst(lst,repElm,repWith):
    outLst=[]
    for l in lst:
        if(l==repElm):outLst.append(repWith)
        else:outLst.append(l)
    return outLst
def __dfLstFunc(placeLst,version,idxName=None,pan=False):
    dfLst = Parallel(n_jobs=num_cores)(delayed(__dfLstFuncHelp)(place,version,idxName,pan) for place in placeLst)
    return dfLst
def __dfLstFuncHelp(place,version,idxName=None,pan=False):
    df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
    if(idxName):df.index.name=idxName
    if(pan):
        samples = random.sample(range(0, len(df.columns)),100);df = df.T.iloc[samples].T
    return (place,df)
def MergeDicts(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res

def numCancersGG(placeLst,dicIn,skipLst=[],version=7,corThres=0.6,positive=True,fontSize=16):
    dfLst = __dfLstFunc(placeLst,version);dfLst.sort()
    out = Parallel(n_jobs=num_cores)(delayed(__numCancersGGHelp)(path,dfLst=dfLst,dicIn=dicIn,skipLst=skipLst,
                                                                 version=version,corThres=corThres,positive=positive,
                                                                 fontSize=fontSize) for path in dicIn)
    return 1
def __numCancersGGHelp(place,version):
    for path in tq(dicIn,desc='G-G'):
        if not path in skipLst:
            outDict={};genes = __runGenes(dicIn[path])
            for df in dfLst:genes = __goodGenes(genes,df.index)
            for i in tq(range(len(genes)),desc=path):
                g1 = genes[i]
                for j in range(i,len(genes)):
                    g2 = genes[j]
                    if not g1 == g2:
                        num = 0 
                        for df in dfLst:
                            g1E = df.loc[g1];g2E = df.loc[g2]
                            pCor = g1E.corr(g2E,method='pearson');sCor = g1E.corr(g2E,method='spearman');value=False
                            if(positive):value = (pCor>corThres and sCor>corThres)
                            else:value = (pCor<corThres and sCor<corThres)
                            if(value):num += 1
                        ggLst[num] = 1;outDict[g1+'_'+g2] = ggLst
            df = pd.DataFrame.from_dict(outDict, orient='index');plt.figure(figsize=(40,20))
            cg = sb.heatmap(df,cmap='Greys',yticklabels=0,xticklabels=1,cbar=False)
            plt.setp(cg.get_xticklabels(),ha='right',size=fontSize)
            title = 'Version '+str(version)+', '+path+', Number of Cancers above Corr '
            title += str(corThres)+', for both Spearman and Pearson'
            plt.title(title,size=40,pad=10);plt.show();print(title)
    return 1

def GetggCancerCluster(placeLst,dicIn,skipLst=[],version=7,parallel=True,meth='pearson',
                          metric='cosine',save=False,threshold=True,thres=0.3,colorLow='White',
                          colorHigh='Black',vmin=None,vmax=None):
    inputs = tq(dicIn,desc='Getting DFs');outLst=[]
    for path in inputs:
        out = __ggCancerCluster(path,placeLst=placeLst,dicIn=dicIn,skipLst=skipLst,version=version,
                                parallel=parallel,meth=meth,metric=metric,save=save,threshold=threshold,
                                thres=thres,colorLow=colorLow,colorHigh=colorHigh,vmin=vmin,vmax=vmax)
        outLst.append(out)
    return outLst
def __ggCancerCluster(path, placeLst,dicIn,skipLst=[],version=7,parallel=True,
                    meth='pearson',metric='cosine',save=False,threshold=False,thres=0,
                    colorLow='White',colorHigh='Black',vmin=None,vmax=None):
    mpl.rcParams['figure.dpi'] = 150
    dfLst = __dfLstFunc(placeLst,version)
    if(save):
        for img in glob.glob('Saved Images/*'):
            os.remove(img)
    outDict={}
    if not path in skipLst:
        genes = __runGenes(dicIn[path])
        for dfTup in dfLst:
            df = dfTup[1]
            genes = __goodGenes(genes,df.index)
        inputs=range(len(genes))
        if(parallel):
            out = Parallel(n_jobs=num_cores)(delayed(__ggCancerClusterHelp)(i,genes,dfLst,meth) for i in inputs)
            outDict = dict(ChainMap(*out))
        else:
            for i in inputs:
                outDict2 = __ggCancerHeatmapHelp(i,dfLst,meth)
                outDict=MergeDicts(outDict,outDict2)
        df = pd.DataFrame.from_dict(outDict, orient='index',columns=placeLst).T
        newIdx = replaceLst(df.index,'Breast Hormone','Breast HR+')
        df.index = newIdx
        if(threshold):
            cmap = thresBinCmap(df,thres,colorLow=colorLow,colorHigh=colorHigh,vmin=vmin,vmax=vmax)
        else:
            cmap=cmapSpecial
        title = 'Version '+str(version)+', '+path+', '+meth.title()+' Correlation'
        if(threshold):
            title += ', Visual Threshold of '+str(thres)
        return (title,cmap,df)

def __ggCancerClusterHelp(i,genes,dfLst,meth='spearman'):
    g1 = genes[i]
    outDict={}
    for j in range(i,len(genes)):
        g2 = genes[j]
        if not g1 == g2:
            num = 0 
            ggLst=[]
            for dfTup in dfLst:
                palce = dfTup[0]
                df = dfTup[1]
                g1E = df.loc[g1]
                g2E = df.loc[g2]
                corr = g1E.corr(g2E,method=meth)
                ggLst.append(corr)
            outDict[g1+'_'+g2] = ggLst
    return outDict

def thresBinCmap(df,thres=0,colorLow='Blue',colorHigh='Red',vmin=None,vmax=None):
    if not(vmax):maxVal= df.max().max()
    else:maxVal = vmax
    if not(vmin):minVal= df.min().min()
    else:minVal = vmin
    if(thres>maxVal or thres<minVal):
        raise Exception("Threshold is Out of Range")
    newMax = abs(minVal-maxVal);diff = newMax-maxVal;thres= thres+diff;shiftVal = thres/newMax
    my_cmap = mpl.colors.LinearSegmentedColormap.from_list('', 
                                                 [(0,    colorLow),(shiftVal-0.001, colorLow),(shiftVal, colorHigh),
                                                  (1,    colorHigh)])
    return my_cmap
def run_ggCancerCluster(placeLst,dicIn,runLst):
    for run in tq(runLst,desc='Overall'):
        version=run[0];skipLst=run[1];meth=run[2];threshold=run[3];thres=run[4]
        x = master_ggCancerCluster(placeLst,dicIn,skipLst=skipLst,version=version,
                                   meth=meth,threshold=threshold,thres=thres,metric='cosine',vmin=None,vmax=None)
        print('Done with '+str(version)+', '+meth+', '+str(thres))
    return 1

def master_ggCancerCluster(placeLst,dicIn,skipLst=[],version=7,meth='pearson',threshold=True,thres=0.3,
                           metric='cosine',vmin=None,vmax=None,save=False):
    outLst = GetggCancerCluster(placeLst,dicIn,skipLst=skipLst,version=version,
                                meth=meth,threshold=threshold,thres=thres)
    for title,cmap,df in tq(outLst,desc='Plotting'):
        t = plot_ggCancerCluster(df,cmap,title,vmin=vmin,vmax=vmax,metric=metric,save=save)
    return 1

def plot_ggCancerCluster(df,cmap,title,vmin=None,vmax=None,metric='cosine',save=False):
    path = title.split(', ')[1]
    if(len(df.columns)<50000):
        cg = sb.clustermap(df,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=0,metric=metric);cg.fig.suptitle(title,y=1)
        if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
        plt.show();print(title)
    else:print('\nGene-gene Pairs for '+path+' is to large to cluster, will be skipped\n')
    return 1
def getdfAll(dfLst,idxName='Genes'):
    if(len(dfLst)==0):return []
    frst=True
    for place,df in dfLst:
        if(frst):
            dfAll=df;frst=False
        else:dfAll = dfAll.merge(df,on=idxName,how='inner')
    return dfAll
def maxLst(dicIn):
    finalLst=[];finalNum=0
    for lst in dicIn.values():
        if(len(lst)>finalNum):finalLst = lst
    return finalLst

def plotCliques(placeLst,dicIn,version=7,meth='spearman',threshold=False,thres=0.3,minGenes=5,onlyTop=True):
    outDict=getCliques(placeLst,dicIn,version=version,meth=meth,minGenes=minGenes,onlyTop=onlyTop)
    master_ggCancerCluster(placeLst,outDict,version=version,meth=meth,threshold=threshold,thres=thres)
    return 1
def __plotCliqueClusterHelp(df,dfAll):
    hotX=[];hotY=[]
    for sample in df.columns:
        dfSam = dfAll[sample];genes=dfAll.index
        for i in range(len(genes)):
            g1 = genes[i];x = dfSam.loc[g1]
            for j in range(i,len(genes)):
                g2 = genes[j]
                if(g1 != g2):
                    y= dfSam.loc[g2];hotX.append(x);hotY.append(y)
    return hotX,hotY
def plotGiantCliqueCluster(placeLst,dicIn,version=7,meth='spearman',idxName='Genes',save=False):
    if(save):
        for img in glob.glob('Saved Images/*'):os.remove(img)
    outDict=getCliques(placeLst,dicIn,version=version,meth=meth);dfLst=__dfLstFunc(placeLst,version,idxName)
    dfAll=getdfAll(dfLst,idxName);Hot,Cold=__getHotCold(dfAll);Not=__notHotOrCold(dfAll,Hot,Cold)
    for path in tq(outDict,desc='Plot Giant Clique Cluster'):
        genes = __runGenes(outDict[path]);dfPath=dfAll.loc[genes]
        hotX, hotY = __plotCliqueClusterHelp(Hot,dfPath);coldX, coldY = __plotCliqueClusterHelp(Cold,dfPath)
        notX, notY = __plotCliqueClusterHelp(Not,dfPath);num=int(len(hotX)/2)
        for i in range(int(len(hotX)/num)):
            plt.rcParams['figure.dpi'] = 150;mpl.rcParams['figure.dpi'] = 150
            plt.figure(figsize=(80,40));plotX = hotX[i*num:(i+1)*num];plotY = hotY[i*num:(i+1)*num]
            plt.plot(plotX,plotY,'o',color='red',label='Hots');plotX = coldX[i*num:(i+1)*num];plotY = coldY[i*num:(i+1)*num]
            plt.plot(plotX,plotY,'o',color='blue',label='Colds');plotX = notX[i*num:(i+1)*num];plotY = notY[i*num:(i+1)*num]
            plt.plot(plotX,plotY,'o',color='black',label='Neither')
            title="Giant Clique Scatter Plot, Version "+str(version)+', '+path+', '+meth.title()+', Part '+str(i+1)
            plt.title(title,fontSize=100,pad=30);plt.xticks(fontsize=40);plt.yticks(fontsize=40)
            if(save):plt.savefig('Saved Images/'+title+'.pdf',dpi=300,bbox_inches = "tight")
            plt.show()
        title=title.replace(', Part '+str(i+1),'');print(title)
    return 1

def getFixOutliers(placeLst):
    for place in tq(placeLst):
        if not place in skip:
            prePure = pd.read_csv('Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv',index_col=0,sep='\t')
            __renameTPM(prePure);prePure = np.log2(prePure+1);prePure = __removeBadPlace(prePure)
            prePure = pd.DataFrame(stats.zscore(prePure,axis=1),prePure.index,columns=prePure.columns)
            prePure, _ = __fixOutliers(prePure,thres=100,scale=0.1)
            prePure.to_csv('Purity/TPM/Outliers Fixed/TCGA_'+place+'_tpm.tsv',sep='\t')
    return 1

def plotCancersThres(placeLst,dicIn,version=7,meth='spearman',idxName='Genes',minGenes=5,
                     thres=0.3,fontSize=12,sizeThres=10,onlyTop=True):
    outDict={};dfLst = __dfLstFunc(placeLst,version,idxName);inputs=tq(dicIn,desc='Finding Cliques');totalGenes=[]
    for path in inputs:
        genes = __runGenes(dicIn[path])
        for place, df in dfLst:genes = __goodGenes(genes,df.index)
        inputs=dfLst
        out = Parallel(n_jobs=num_cores)(delayed(__getCliquesLst)
                                         (df.loc[genes],meth,thres,minGenes,
                                          bothMeths=True,retPlace=True,place=place,onlyTop=onlyTop) for place,df in inputs)
        pathDic={}
        for lst,thres,place in out:pathDic[place+'_'+str(thres)] = lst
        outDict[path] = pathDic
    for path in tq(outDict,desc='Plotting Cliques'):
        dicPath = outDict[path];totalGenes=[]
        for place in dicPath:
            placeName = place.split('_')[0];lst=dicPath[place];totalGenes.extend(lst)
        geneDic={}
        for gene in totalGenes:
            inLst=[]
            for place in dicPath:
                placeName = place.split('_')[0];lst=dicPath[place]
                if(gene in lst):inLst.append(1)
                else:inLst.append(0)
            geneDic[gene]=inLst
        dfPlot = pd.DataFrame.from_dict(geneDic,orient='index',columns=placeLst)
        dfPlot.columns = getUniqueNames(dfPlot.columns);dfPlot = dfPlot.loc[dfPlot.T.sum()>12]
        if(len(dfPlot.index)>1):
            cg = sb.clustermap(dfPlot,cmap='Greys',yticklabels=1)
            addOn = len(cg.ax_heatmap.get_yticklabels()) / sizeThres;newFont = fontSize - addOn
            plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom',size=newFont)
            plt.setp(cg.ax_heatmap.get_xticklabels(),rotation=45,ha='right')
            title = 'Version '+str(version)+', Threshold '+str(thres)+', '+path
            cg.fig.suptitle(title,y=1);mpl.rcParams['figure.dpi'] = 200;plt.show();print(title)
        else:print('No genes in enough cancers for '+path)
    return 1
def getCliques(placeLst,dicIn,thres=0.3,version=7,meth='spearman',minGenes=5,idxName='Genes',
               onlyTop=True,bothMeths=False,tqTrue=True,pan=False):
    dfLst = __dfLstFunc(placeLst,version,idxName,pan=pan);dfAll = getdfAll(dfLst,idxName)
    if(tqTrue):inputs=tq(dicIn,desc='Finding Cliques')
    else:inputs=dicIn
    out=[]
    for path in inputs:out.append(__getCliquesHelp(dfAll,dicIn,thres,meth,path,minGenes,onlyTop,bothMeths))
    outDict={}
    for path,thres,lst in out:
        dic={'clique':lst};outDict[path]=dic
    return outDict
def __getCliquesHelp(dfAll,dicIn,thres,meth,path,minGenes,onlyTop,bothMeths): 
    genes = __runGenes(dicIn[path]);genes = __goodGenes(genes,dfAll.index);dfPath = dfAll.loc[genes]
    outLst,thres = __getCliquesLst(dfPath,meth,thres,minGenes,onlyTop=onlyTop,bothMeths=bothMeths)
    outLst2=outLst.copy();thres2=thres
    while(len(outLst2)>30):
        outLst=outLst2.copy();thres=thres2;thres2 = round(thres2+0.05,2)
        outLst2,thres2 = __getCliquesLst(dfPath,meth,thres2,minGenes,onlyTop=onlyTop,bothMeths=bothMeths)
    return (path,thres,outLst)

def __getCliquesLst(dfPath,meth,thres,minGenes,bothMeths=False,retPlace=False,place='',onlyTop=True):
    if(bothMeths):
        lst1, thres = __getCliquesLstHelp(dfPath,meth='spearman',thres=thres,minGenes=minGenes,onlyTop=onlyTop)
        lst2, thres = __getCliquesLstHelp(dfPath,meth='pearson',thres=thres,minGenes=minGenes,onlyTop=onlyTop)
        if(onlyTop):finalLst = list(set(lst1) | set(lst2))
        else:finalLst = mergeDic(lst1,lst2)
    else:finalLst, thres = __getCliquesLstHelp(dfPath,meth=meth,thres=thres,minGenes=minGenes,onlyTop=onlyTop)
    if(retPlace):return finalLst, thres, place
    else:return finalLst, thres
    
def __getCliquesLstHelp(dfPath,meth,thres,minGenes,onlyTop=True):
    corr = dfPath.T.corr(method=meth);outClique={};seenLst=[];finalLst=[]
    if not onlyTop:finalDict={}
    finalNum=0
    for gene in corr.index:
        if not gene in seenLst:
            ser = corr.loc[gene];cliqueGenes = ser[ser>thres].index
            clique = corr.loc[cliqueGenes][cliqueGenes];cliqueThres = clique[clique>thres]
            while(cliqueThres.isna().sum().max()>0):
                maxId = cliqueThres.isna().sum().argmax();dropName = cliqueThres.iloc[maxId].name
                cliqueThres = cliqueThres.drop(dropName);cliqueThres = cliqueThres.drop(dropName,axis=1)
            lst = cliqueThres.index
            if(len(lst)>minGenes):
                if(onlyTop):
                    if(len(lst)>finalNum):
                        finalNum = len(lst);finalLst = list(lst)
                else:
                    finalDict[gene] = (list(lst))
            seenLst.extend(list(lst))
    if not onlyTop:
        finalLst=finalDict
    return finalLst,thres

def mergeDic(dic1,dic2):
    out={}
    for p in dic1:
        lst= dic1[p]
        if p in dic2:
            lst2 = dic2[p];lst = list(set(lst) | set(lst2))
        out[p] = lst
    for p in dic2:
        if not p in dic1:
            lst = dic2[p];out[p] = lst
    return out

def getUniversalCliques(placeLst,dicIn,version=7,idxName='Genes',meth='spearman',
                        minGenes=3,thres=0.3):
    outDict={}
    if(True):
        dfLst = __dfLstFunc(placeLst,version,idxName);inputs=tq(dicIn,desc='Finding Cliques');totalGenes=[]
        for path in inputs:
            genes = __runGenes(dicIn[path])
            for place, df in dfLst:genes = __goodGenes(genes,df.index)
            inputs=dfLst
            out = Parallel(n_jobs=num_cores)(delayed(__getCliquesLst)
                                             (df.loc[genes],meth,thres,minGenes,
                                              bothMeths=True,retPlace=True,place=place,onlyTop=False) for place,df in inputs)

            frst=True;plotDic={};excludedDic={}
            for dic, thres, place in out:
                plotDic2={}
                if(frst):
                    plotDic2= dic.copy()
                    for p in dic:
                        for elem in dic[p]:excludedDic[elem] = [1]
                    frst=False
                else:
                    for p in plotDic:
                        if p in dic:
                            lst1 = dic[p];lst2 = plotDic[p];lst = __goodGenes(lst1,lst2)
                            if(len(lst)>1):plotDic2[p]=lst
                plotDic=plotDic2.copy()
            outDict[path]=plotDic
    return outDict
def setupBNLearnPathways(placeLst,dicIn,pooledVersion=7,minGenes=5,runVLst=[],panCancer=False,
                 numCancers=1,pooledClique=False,indivDBCliques=False,indivCancerCliques=False):
    """
    setupBNLearnPathways:
            Inputs:
                placeLst: a list of cancer areas to use within the function, most important if pooledClique=True
                    no default
                dicIn: a dictionary telling which genes should belong to which pathways, 
                       for use in creating the pathway cliques
                    no default, but everyDict is the recommened input
                pooledVersion: the data correction version from which the pathway cliques will be created
                    default: 7 (for v7 of correction)
                minGenes: the minimun number of genes need to be considered a clique, 
                          Warning if number is set to high possiblity that no cliques will be found
                    deafult: 5
                runVLst: a list of the correction versions for which a BNLearn input file will be created,
                         currently supports inputs of 1 thru 8 (for v1-v8) and 'PrePure'
                    deafult: an empty list, running on deafult will create no files
                numCancers: a number from how many of the biggest (in number of samples) cancer areas BNLearn input files
                            will be created. If numCancers > length of placeLst all placeLst cancers will be used.
                    deafult: 1
                pooledClique: boolean telling if to use the "Pooled" method for deifning cliques, 
                              a method where, for each pathway (as defined by dicIn), the largest group of genes 
                              (of at least size minGenes) that all have correlation of at least 0.3 to all other 
                              genes in the group, with correaltion based on samples from all cancers in placeLst
                              with pooledVersion data,
                              is defined as the clique for that pathway.
                              Warning: this method is heavily towards the largest cancer areas.
                    default: False
                indivDBClqieus: boolean telling if to use the individual databases (as defined in dicIn) pathways
                                as the pathway cliques.
                                Will create a clique for each database in dicIn which has at least minGenes in all pathways.
                    deafult: False
                indivCancerCliques: boolean telling if to use the "Cancer Specific" method for defing cliques,
                                    which is very similar to the "Pooled" method, only instead of using all cancers in 
                                    placeLst it creates a clique for each cancer specifically 
                                    (using only that cacners samples)
                    deafult: False
                Warning:
                    If the dafualts for pooledClique, indivDBCliques and indivCancerCliques are all left unchanged no 
                    input files will be created. If more than one of the three are set to True, uncertain what file will be 
                    created.
            Actions:
                Creates BNLearn input files for each of the N biggest cancer areas (N=numcancers). For each cacner area 
                    a file will be created for each data version in runVLst. 
                For each pathway it creates a clique (depending on which method was chosen), from the pathway genes defined in
                    dicIn.
                The BNLearn file will have each of those pathways as a column, along with Purity Scores and Heat Scores, 
                    and all the specific cancers samples as the rows. The data will the the average of that pathway's 
                    clique genes for the given sample, as well as the Purity and Heat Scores for that Sample.
                The file's name will indicate the cancer area, the number of samples within that cancer, the clique 
                    method used, and the correction version used for that file's data.
                The total number of files created will either be, numCancers*length runVLst (if indivDBCliques=False)
                    or numCancers*length runVLst*databases within dicIn (if indivDBCliques=True).
                    
            Returns:
                Should create the input BNLearn files requested in the 'Run BNLearn/Inputs/' directory, 
                returns 1 if no errors have occured 
    """
    if(pooledClique):
        outDict= getCliques(placeLst,dicIn,version=pooledVersion,minGenes=minGenes,onlyTop=True,bothMeths=True,pan=False)
        __setupBNHelp(numCancers=numCancers,runVLst=runVLst,outDict=outDict,title=', Pooled',panCancer=panCancer)
    if (indivDBCliques):
        frstDic = list(dicIn.keys())[0];dbLst=[]
        for db in tq(dicIn[frstDic],desc='Setting up DBs'):
            if not db =='HALLMARK':
                outDict = __getDBList(dicIn,db);__setupBNHelp(numCancers=numCancers,runVLst=runVLst,
                                                              outDict=outDict,title=', '+db,tqTrue=False,panCancer=panCancer)
    if(indivCancerCliques):
        pLst = getBiggestXCancers(numCancers)
        if not panCancer:
            for place,num in tq(pLst,desc='Setting up Cliques'):
                outDict= getCliques([place],dicIn,version=pooledVersion,minGenes=minGenes,
                                    onlyTop=True,bothMeths=True,tqTrue=False,thres=0.4)
                __setupBNHelp(numCancers=numCancers,runVLst=runVLst,outDict=outDict,
                              title=', Cancer Specific',tqTrue=False,allPlaces=False,setPlace=place,panCancer=panCancer)
        else:
            outDict= getCliques(placeLst,dicIn,version=pooledVersion,minGenes=minGenes,pan=True,
                                onlyTop=True,bothMeths=True,tqTrue=True,thres=0.4)
            __setupBNHelp(numCancers=numCancers,runVLst=runVLst,outDict=outDict,
                              title=', Cancer Specific',tqTrue=True,allPlaces=True,setPlace='',panCancer=True)
    return 1
nameDict={}
nameDict['T-effector']='T-Eff';nameDict['IFNG induced']='IFNg';nameDict['Antigen processing machinery']='Antigen Process'
nameDict['Fatty acid']='FA';nameDict['GABA'] = 'GABA';nameDict['Cholesterol synthesis']='CS';nameDict['PI3K-AKT']='PI3K'
def __setupBNHelp(numCancers, runVLst,outDict,title='',tqTrue=True,allPlaces=True,setPlace='',panCancer=False):
    pLst = getBiggestXCancers(numCancers)
    if(tqTrue):inputs=tq(runVLst,desc='Setting Up BNLearn')
    else:inputs = runVLst
    for runVersion in inputs:
        if(panCancer):
            dfLst=[];cancerNum=1
        for place, placeLen in pLst:
            if(allPlaces or place==setPlace):
                if(runVersion!='PrePure'):df = pd.read_csv('Purity/TPM/v'+str(runVersion)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
                else:df = pd.read_csv('Purity/TPM/Outliers Fixed/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
                pathLst=[];serLst=[]
                for path in outDict:
                    pathLst.append(path);pathGenes = __runGenes(outDict[path]);pathGenes = __goodGenes(pathGenes,df.index)
                    dfPath = df.loc[pathGenes];ser = dfPath.mean();serLst.append(ser)
                dfBN = pd.DataFrame(serLst,pathLst);dfBN = __pureDf(dfBN,zscore=False)
                dfBN = __heatDF(dfBN,place,zscore=True)[0];dfBN = dfBN.T;newCols=[]
                for i in dfBN.columns:
                    i2 = i.split('_')
                    for num in range(len(i2)):
                        elem = i2[num]
                        if(elem in nameDict):i2[num] = nameDict[elem]
                    i = ('_').join(i2);newCols.append(i)
                dfBN.columns=newCols;dfBN.dropna(axis=1,inplace=True)
                if(panCancer):
                    placeLen = len(dfBN.index);samples = random.sample(range(0, placeLen), 100)
                    dfBN = dfBN.iloc[samples];dfBN.insert(len(dfBN.columns),'Cancer Area',cancerNum)
                    dfLst.append((place,dfBN));cancerNum+=1
                else:
                    if(runVersion!='PrePure'):
                        dfBN.to_csv('Run BNLearn/Inputs/Pure '+place+', N = '+str(placeLen)+title+', v'+str(runVersion),sep='\t')
                    else:
                        dfBN.to_csv('Run BNLearn/Inputs/Pure '+place+', N = '+str(placeLen)+title+', '+str(runVersion),sep='\t')
            if(panCancer):
                if(len(dfLst)>0):
                    dfAll = getDfPan(dfLst)
                    if(runVersion!='PrePure'):
                        dfAll.to_csv('Run BNLearn/Inputs/Pure Pre-BN Merged'+title+', v'+str(runVersion),sep='\t')
                    else:dfAll.to_csv('Run BNLearn/Inputs/Pure Pre-BN Merged'+title+', '+str(runVersion),sep='\t')     
    return 1
def getDfPan(dfLst):
    frst=True
    for place, df in dfLst:
        if(frst):
            out = df.copy();frst=False
        else:out = pd.concat([out,df])
    return out
def getKobeClique(getKobeDict=False):
    kobeGenes = pd.read_excel('Genelist_desert.xlsx',index_col=0);pathDict={}
    for pathway in kobeGenes['pathway'].unique():
        dictN={};dictN['kobe'] = list(kobeGenes[kobeGenes['pathway']==pathway].index)
        if(pathway in nameDict):pathway = nameDict[pathway]
        pathDict[pathway]= dictN
    if not getKobeDict:
        return pathDict
    else:
        kobe = pathDict;kobeDict={}
        for path in kobe:
            genes = __runGenes(kobe[path])
            for g in genes:
                g = g.replace('-','');kobeDict[g] = path
        kobeDict['Heat'] = 'Heat';kobeDict['Purity'] = 'Purity';kobeDict['CancerArea'] = 'Cancer Area'
        return kobeDict

In [11]:
def plotBNLearnRobustDBs(globDir='Run BNLearn/Outputs/*',metric='cosine',fsX=8,fsY=6,size=(20,20),
                         kobeList=False,onlyN=True,dropNum=5,split=False,
                         merged=True,indivCancers=True,onlyBGE=False,onlyBDE=False,save=False,fixedPos=False,thres=0.7,
                         interPath=False,immuneNonOnly=False,N=4,nonToIm=False,allDir=False,pathways=False,excludePre=False,
                         reverse=False,undirected=False,directed=False,mergedFixedPos=False,posDrop=([],[]),showMergeY=0):
    """
    plotBNLearn:
        Plots the plotBNLearnRobust Heatmap for each individual database in dicIn
            for more info read description of plotBNLearnRobust
    """
    
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=onlyBGE,onlyBDE=onlyBDE)
    dbLs = []
    for graph in graphLst:
        ls = graph.split('\\')[1].split(',')
        if('BN Merged' in ls[0]):
            db = ls[1].strip()
        else:
            db = ls[2].strip()
        dbLs.append(db)
    dbLs = list(set(dbLs))
    i=0
    gL=[]
    for db in tq(dbLs,desc='Plotting DB HMs'):
        if(mergedFixedPos):
            tup = posDrop[i]
            pos = tup[0]
            drop = tup[1]
        else:
            pos=[]
            drop=[]
        i+=1
        dirGlob='Run BNLearn/Outputs/*'+db+'*'
        col, drop = plotBNLearnRobust(metric=metric,fsX=fsX,fsY=fsY,size=size,merged=merged,indivCancers=indivCancers,
                                      onlyBGE=onlyBGE,onlyBDE=onlyBDE,save=save,fixedPos=fixedPos,reverse=reverse,
                                      undirected=undirected,excludePre=excludePre,tqOn=False,
                                      directed=directed,mergedFixedPos=mergedFixedPos,pos=pos,drop=drop,split=split,
                                      kobeList=kobeList,onlyN=onlyN,thres=thres,interPath=interPath,pathways=pathways,
                                      immuneNonOnly=immuneNonOnly,N=N,nonToIm=nonToIm,allDir=allDir,
                                      dirGlob=dirGlob,title=', Only '+db,showMergeY=showMergeY,dropNum=dropNum)
        gL.append((col,drop))
    return gL,dbLs
    
def plotBNLearnRobust(metric='cosine',fsX=8,fsY=8,size=(20,20),split=False,kobeList=False,onlyN=False,thres=0.7,N=4,
                      pathways=False,nonToIm=False,allDir=False,excludePre=False,tqOn=True,
                      merged=True,indivCancers=True,onlyBGE=False,onlyBDE=False,save=False,fixedPos=False,immuneNonOnly=False,
                      reverse=False,undirected=False,directed=False,mergedFixedPos=False,pos=[],drop=[],interPath=False,
                      dirGlob='Run BNLearn/Outputs/*',title='',showMergeY=0,dropImmune=False,dropNum=0):
    """
        plotBNLearnRobust:
            
            Inputs:
                    metric: the mertic by which to cluster the BNLearn results in the heatmaps
                        deafualt: 'cosine'
                    fsX: font size for the x-axis labels in the resulting heatmaps
                        default: 8
                    fsY: font size for the y-axis labels in the resulting heatmaps
                        deafult: 6
                    size: a tuple with the size of the resulting heatmap
                        default: (20,20), works best when width and hight are equal
                    merged: boolean telling if to plot the merged heatmap, using BNLearn output data from all possible cancer
                            areas
                        deafult: True
                    indivCacners: boolean telling if to plot each possible BNLearn output cancer area data individually
                        deafult: True
                    
                    If both merged and indivCacners are True, will first plot merged and then the individual cacners
                    
                    onlyBGE: boolean telling if to inlcude only BNLearn outputs that used the "bge" scoreing method in the
                             heatmaps (if to only plot the 'bge' BNLearn results)
                        deafult: False
                    onlyBDE: boolean telling if to inlcude only BNLearn outputs that used the "bde" scoreing method in the
                             heatmaps (if to only plot the 'bde' BNLearn results)
                        deafult: False
                    save: boolean telling if to save the resulting heatmaps in the 'Saved Images/' directory
                        default: False
                    fixedPos: boolean telling if to make the order the x-axis of the individual cancer heatmap on the same as
                              the order of the merged heatmaps x-axis
                        default: False
                    reverse: boolean telling if to reverse the direction of the pathways in the x-axis (from A->B to B->A),
                             all B->A directed edges will be colored blue-white-red, while all A->B directed and undirected 
                             edges will be grayscaled based on their edge strength.
                        default: False
                    undirected: boolean telling if to use undirected pathways in the x-axis (from A->B to A--B),
                                all A--B directed edges will be colored blue-white-red, while all A->B and B->A directed
                                edges will be grayscaled based on their edge strength
                        deafult: False
                    directed: boolean telling if to use the base x-axis direction, A->B directed edges will be colored 
                              blue-white-red, while all B->A directed and undirected edges will be grayscaled based on their 
                              edge strength.
                        deafult: False
                    
                    If reverse=True, directed atomatically becomes True
                    If undirected=False, both reverse and directed atomatically become False
                    If reverse, directed and undirected are all False, heatmap plots direction agnostic, where all edges
                        are colored blue-white-red, with the input being the highest strength (form any direction) for 
                        each edge.
                    
                    mergedFixedPos, boolean telling if to base the order of the x-axis of the Merged heatmap on the pos input
                        deafult: False
                    pos: List of ints telling the order of elements to order the Merged Heatmap, ints must be within the range 
                             of the length of the Merged Heatmap's x-axis.
                         A run of the plotBNLearnRobust function returns a list of ints that can be used as the pos input,
                             (it is recommended that a previous run of plotBNLearnRobust be the only way you get a pos input)
                        
                        deafult: an empty list (will error if mergedFixedPos=True, and deafult pos is kept)
                
                Actions:
                    The functions takes all the BNLearn outputs in the 'BNLearn/Outputs/' directory, if 
                        onlyBGE or onlyBDE were set to True then the outputs taken would be limited to only 
                        outputs with that scoreing method.
                    Then plots a heatmap of those outputs, mainly the strength of the different possible edges. If heatmap is
                        not direction agnostic, then the strength of the directed edges are blue-white-red while the strength
                        of edges for the other directions are greyscaled. The y-axis ordering is based on the cluster metric
                        given be metric, while the x-axis order is either also clustered or fixed 
                        (if fixedPos or mergedFixedPos = True).
                    The created plots can also be saved, if the option is chosen.
                
                Outputs:
                    On success returns a list of ints detailing the x-axis order for the last heatmap, which can be used in 
                        future calls of this function for the pos input.
                    
    """
    if(reverse):
        directed=True
    if(undirected):
        reverse=False
        directed=False
    if not pathways:
        graphLst = glob.glob(dirGlob)
    else:
        graphLst = globExclude(dirGlob,exclude='Kobe Genes')
    if(excludePre):
        graphLst = __removeLstFunc(graphLst,'PrePure')
    if(len(graphLst)==0):
        return [],[]
    sizeX=(size[0]/20)+1
    sizeY=(size[1]/20)+1
    fsX=fsX*sizeX
    fsY=fsY*sizeY
    graphLst = __onlyBGEorBDE(graphLst,onlyBGE,onlyBDE)
    if(merged):
        graphLst2 = graphLst = __removeLstFunc(graphLst,'Pre-BN')
        if not mergedFixedPos:
            col,drop = __plotBNLearnRobustHelp(graphLst2,metric=metric,fsX=fsX,fsY=fsY,title=', Merged Graph'+title,
                                               size=size,save=save,onlyBGE=onlyBGE,onlyBDE=onlyBDE,split=split,
                                               reverse=reverse,undirected=undirected,directed=directed,fixedPos=False,
                                               showMergeY=showMergeY,dropImmune=dropImmune,dropNum=dropNum,allDir=allDir,
                                               immuneNonOnly=immuneNonOnly,N=N,nonToIm=nonToIm,pathways=pathways,
                                               kobeList=kobeList,onlyN=onlyN,thres=thres,interPath=interPath)
        else:
            col,drop = __plotBNLearnRobustHelp(graphLst2,metric=metric,fsX=fsX,fsY=fsY,title=', Merged Graph'+title,split=split,
                                               size=size,save=save,onlyBGE=onlyBGE,onlyBDE=onlyBDE,drop=drop,allDir=allDir,
                                               kobeList=kobeList,onlyN=onlyN,thres=thres,interPath=interPath,
                                               immuneNonOnly=immuneNonOnly,N=N,nonToIm=nonToIm,pathways=pathways,
                                               reverse=reverse,undirected=undirected,fixedPos=True,pos=pos,directed=directed,
                                               showMergeY=showMergeY,dropImmune=dropImmune,dropNum=dropNum)
    else:
        if fixedPos:
            print("Error: fixedPos=True option must also have merged=True")
            return 0
    if(indivCancers):
        placeLs = []
        for graph in graphLst:
            place = graph.split('\\')[1].split(',')[0].strip()
            placeLs.append(place)
        placeLs = list(set(placeLs))
        if(tqOn):
            inputs = tq(placeLs,desc='Plotting')
        else:
            inputs = placeLs
        for place in inputs:
            graphLst2 = [p for p in graphLst if place in p]
            col,drop = __plotBNLearnRobustHelp(graphLst2,metric=metric,fsX=fsX-2,fsY=fsY,title=', '+place+title,
                                               interPath=interPath,immuneNonOnly=immuneNonOnly,
                                               size=size,save=save,onlyBGE=onlyBGE,onlyBDE=onlyBDE,fixedPos=fixedPos,pos=col,
                                               drop=drop,dropNum=dropNum,split=split,kobeList=kobeList,onlyN=onlyN,N=N,
                                               thres=thres,nonToIm=nonToIm,pathways=pathways,allDir=allDir,
                                               reverse=reverse,undirected=undirected,directed=directed,dropImmune=dropImmune)
    return col, drop

def __plotBNLearnRobustHelp(graphLst,metric,fsX,fsY,title,split,kobeList,onlyN,N,thres,immuneDict,dictIn,
                            size,save,onlyBGE,onlyBDE,fixedPos,pos,drop,interPath,
                            reverse=False,undirected=False,directed=False,showMergeY=0,dropImmune=False,dropNum=0,
                            immuneNonOnly=False,nonToIm=False,pathways=False,allDir=False):
    outDict, nameLst = __BNOutDict(graphLst,reverse=reverse,undirected=undirected,directed=directed,dropImmune=dropImmune,
                                   allDir=allDir)
    df = pd.DataFrame.from_dict(outDict,orient='index',columns=nameLst)
    if(len(df.index)==0):
        return pos,drop
    newIdx,newNames,l1 = __renameBNOut(df)
    df.columns=newIdx
    df =df.T
    df = df.fillna(0)
    if(interPath):
        df = interPathwayOnly(df,immuneDict=immuneDict,dictIn=dictIn,graphviz=False,
                              immuneNonOnly=immuneNonOnly,nonToIm=nonToIm,pathways=pathways,allDir=allDir)
    colors = ['red','blue','green','yellow','purple','black','orange','grey','white',
              'brown','fuchsia','pink','gold','seagreen','salmon','olive','teal','crimson','cyan']
    
    title2 = 'Strength of BNLearn Pathway Relationships'+title
    cClust = not(fixedPos)
    if fixedPos:
        col = pos.copy()
        drop2 = __changeDrop(drop,directed=directed,reverse=reverse,undirected=undirected)
        drop2 = __goodGenes(drop2,df.columns)
        df1 = df.drop(columns=drop2)
        df1 = __setCluster(df1,range(len(df1.index)),col)
    else:
        colColors=None
        colLut='R'
        df1 = df.copy()
        if(kobeList and (len(newNames)>1)):
            pLst=[]
            for row in df1.index:
                place = row.split(',')[0]
                pLst.append(place)
            pLst = list(set(pLst))
            outDict={}
            for place in pLst:
                rows = [p for p in df1.index if place in p]
                place = place.replace('Kobe Genes ','')
                dfRows = df1.loc[rows]
                outDict[place] = dfRows.mean()
            dfOut = pd.DataFrame.from_dict(outDict, orient='index')
            notNans = dfOut[dfOut < thres].isnull().sum(axis=0)
            if(onlyN):
                isNans = notNans[notNans < N]
                notNans = notNans[notNans >= N] 
            notNans = notNans.index
            df1 = dfOut[notNans]
    if(kobeList):
        colColors,colLut = BNLearnColColors(df1,reverse=reverse,undirected=undirected,directed=directed)
    else:
        colColors=None
        colLut='R'
    if(directed or undirected):
        cmap = cmapSpecial4
        vmin = -1
        vmax = 1
    else:
        cmap = cmapSpecial
        vmin=0
        vmax=1
    if(len(newNames)>1):
        fx = df1.sum() != 0
        colLst=[]
        for col in df1.columns:
            if fx[col]:
                colLst.append(col)
        df1 = df1[colLst]
        lut,placeColors = bnLearnRowColors(df1.index.unique(),colors)
        row_colors = df1.index.unique().map(lut)
        cg = sb.clustermap(df1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,col_cluster=cClust,col_colors=colColors,
                           yticklabels=showMergeY,metric=metric,figsize=size,row_colors=row_colors)
        if not fixedPos:
            col = cg.dendrogram_col.reordered_ind
        if((not fixedPos) and (dropNum > 0 or kobeList)):
            plt.close()
            df2 = __setCluster(df1,range(len(df1.index)),col)
            if(kobeList):
                drop = list(isNans)
            else:
                drop = df2.columns[:dropNum]
                drop2 = __changeDrop(drop,directed=directed,reverse=reverse,undirected=undirected)
                drop2 = __goodGenes(drop2,df.columns)
                df1 = df1.drop(columns=drop2)
            lut,placeColors = bnLearnRowColors(df1.index.unique(),colors)
            row_colors = df1.index.unique().map(lut)
            cg = sb.clustermap(df1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,col_colors=colColors,
                               yticklabels=showMergeY,metric=metric,figsize=size,row_colors=row_colors,col_cluster=cClust)
            col = cg.dendrogram_col.reordered_ind
        handles = [Patch(facecolor=placeColors[name]) for name in placeColors]
        leg1 = plt.legend(handles, placeColors, title='Places',bbox_to_anchor=(0.175, 0.75),
                          bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
        if not colLut=='R':
            handles = [Patch(facecolor=colLut[name]) for name in colLut]
            leg2 = plt.legend(handles, colLut, title='',bbox_to_anchor=(1.175, 0.75),
                              bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
    else:
        if(kobeList):
            colColors,colLut = BNLearnColColors(df1,reverse=reverse,undirected=undirected,directed=directed)
        else:
            colColors=None
            colLut='R'
        cg = sb.clustermap(df1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,yticklabels=1,col_cluster=cClust,
                           metric=metric,figsize=size,col_colors=colColors)
        title2 += ', '+l1
        if not colLut=='R':
            handles = [Patch(facecolor=colLut[name]) for name in colLut]
            plt.legend(handles, colLut, title='',
                       bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
        if not(fixedPos):
            plt.close()
            col = cg.dendrogram_col.reordered_ind
            df2 = __setCluster(df1,range(len(df1.index)),col) 
            if(kobeList):
                drop = isNans
            else:
                drop = df2.columns[:dropNum]
                df1 = df1.drop(columns=drop)
            if(kobeList):
                colColors,colLut = BNLearnColColors(df1,reverse=reverse,undirected=undirected,directed=directed)
            else:
                colColors=None
                colLut='R'
            cg = sb.clustermap(df1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,yticklabels=1,col_cluster=cClust,
                               metric=metric,figsize=size,col_colors=colColors)
            if not colLut=='R':
                handles = [Patch(facecolor=colLut[name]) for name in colLut]
                plt.legend(handles, colLut, title='',
                           bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            col = cg.dendrogram_col.reordered_ind        
        
    if(onlyBGE):
        title2 += ', Only BGE'
    if(onlyBDE):
        title2 += ', Only BDE'
    if(kobeList):
        title2 += ', Kobe Gene List, thres = '+str(thres)
    if(undirected):
        title2 ='Undirected, '+title2
    else:
        if(directed):
            if(reverse):
                title2 ='Direction B-A, '+title2
            else:
                title2 ='Direction A-B, '+title2
        else:
            if(allDir):
                title2 = 'All Directions, '+title2
            else:
                title2 = 'Direction Agnostic, '+title2 
    
    if(split):
        plt.close()
        df2 = __setCluster(df,range(len(df.index)),col)
        half = round(len(df2.columns)/2)
        dfP1 = df2.columns[:half]
        dfP2 = df2.columns[half:]
        dfP1 = df2[dfP1]
        dfP2 = df2[dfP2]
        if(len(newNames)>1):
            if(kobeList):
                colColors,colLut = BNLearnColColors(dfP1,reverse=reverse,undirected=undirected,directed=directed)
            else:
                colColors=None
                colLut='R'
            cg = sb.clustermap(dfP1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,col_cluster=False,col_colors=colColors,
                               yticklabels=showMergeY,metric=metric,figsize=size,row_colors=row_colors)
            title3 = title2+', Part 1'
            cg.ax_heatmap.set_xlabel('Pathway Pairings',size=20)
            cg.ax_heatmap.set_ylabel('Input Data',size=20)
            cg.ax_row_dendrogram.set_visible(False)
            cg.ax_col_dendrogram.set_visible(False)
            plt.setp(cg.ax_heatmap.get_xticklabels(),rotation='45',ha='right',size=fsX)
            plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=fsY,rotation='0')
            title3=title3.replace(', ,',', ').replace(' , ','').replace('  ','').replace(',P',', P')
            plt.title(title3,size=25,y=1,x=8)
            handles = [Patch(facecolor=placeColors[name]) for name in placeColors]
            plt.legend(handles, placeColors, title='Places',bbox_to_anchor=(0.175, 0.75),
                       bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            if not colLut=='R':
                handles = [Patch(facecolor=colLut[name]) for name in colLut]
                plt.legend(handles, colLut, title='',
                           bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            
            if(save):
                plt.savefig('Saved Images/'+title3+'.pdf',bbox_inches = "tight")
            else:
                plt.show()
                print(title3)
            plt.close()
            if(kobeList):
                colColors,colLut = BNLearnColColors(dfP2,reverse=reverse,undirected=undirected,directed=directed)
            else:
                colColors=None
                colLut='R'
            cg = sb.clustermap(dfP2,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,col_cluster=False,col_colors=colColors,
                               yticklabels=showMergeY,metric=metric,figsize=size,row_colors=row_colors)
            handles = [Patch(facecolor=placeColors[name]) for name in placeColors]
            plt.legend(handles, placeColors, title='Places',bbox_to_anchor=(0.175, 0.75),
                       bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            title3 = title2+', Part 2'
            title3=title3.replace(', ,',', ').replace(' , ','').replace('  ','').replace(',P',', P')
            cg.ax_heatmap.set_xlabel('Pathway Pairings',size=20)
            cg.ax_heatmap.set_ylabel('Input Data',size=20)
            cg.ax_row_dendrogram.set_visible(False)
            cg.ax_col_dendrogram.set_visible(False)
            plt.setp(cg.ax_heatmap.get_xticklabels(),rotation='45',ha='right',size=fsX)
            plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=fsY,rotation='0')
            plt.title(title3,size=25,y=1,x=8)
            if not colLut=='R':
                handles = [Patch(facecolor=colLut[name]) for name in colLut]
                plt.legend(handles, colLut, title='',
                           bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            if(save):
                plt.savefig('Saved Images/'+title3+'.pdf',bbox_inches = "tight")
            else:
                plt.show()
                print(title3)
            plt.close()
            
        else:
            if(kobeList):
                colColors,colLut = BNLearnColColors(dfP1,reverse=reverse,undirected=undirected,directed=directed)
            else:
                colColors=None
                colLut='R'
            cg = sb.clustermap(dfP1,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,col_cluster=False,yticklabels=1,
                               metric=metric,figsize=size,col_colors=colColors)

            title3 = title2+', Part 1'
            title3=title3.replace(', ,',', ').replace(' , ','').replace('  ','').replace(',P',', P')
            cg.ax_heatmap.set_xlabel('Pathway Pairings',size=20)
            cg.ax_heatmap.set_ylabel('Input Data',size=20)
            cg.ax_row_dendrogram.set_visible(False)
            cg.ax_col_dendrogram.set_visible(False)
            plt.setp(cg.ax_heatmap.get_xticklabels(),rotation='45',ha='right',size=fsX)
            plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=fsY,rotation='0')
            plt.title(title3,size=25,y=1,x=8)
            if not colLut=='R':
                handles = [Patch(facecolor=colLut[name]) for name in colLut]
                plt.legend(handles, colLut, title='',
                           bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            if(save):
                plt.savefig('Saved Images/'+title3+'.pdf',bbox_inches = "tight")
            else:
                plt.show()
                print(title3)
            plt.close()
            
            if(kobeList):
                colColors,colLut = BNLearnColColors(dfP2,reverse=reverse,undirected=undirected,directed=directed)
            else:
                colColors=None
                colLut='R'
            
            cg = sb.clustermap(dfP2,cmap=cmap,vmin=vmin,vmax=vmax,xticklabels=1,yticklabels=1,col_cluster=False,
                               metric=metric,figsize=size,col_colors=colColors)
            title3 = title2+', Part 2'
            title3=title3.replace(', ,',', ').replace(' , ','').replace('  ','').replace(',P',', P')
            cg.ax_heatmap.set_xlabel('Pathway Pairings',size=20)
            cg.ax_heatmap.set_ylabel('Input Data',size=20)
            cg.ax_row_dendrogram.set_visible(False)
            cg.ax_col_dendrogram.set_visible(False)
            plt.setp(cg.ax_heatmap.get_xticklabels(),rotation='45',ha='right',size=fsX)
            plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=fsY,rotation='0')
            plt.title(title3,size=25,y=1,x=8)
            if not colLut=='R':
                handles = [Patch(facecolor=colLut[name]) for name in colLut]
                plt.legend(handles, colLut, title='',
                           bbox_to_anchor=(1.175, 0.75), bbox_transform=plt.gcf().transFigure, loc=0, prop={'size': 15})
            if(save):
                plt.savefig('Saved Images/'+title3+'.pdf',bbox_inches = "tight")
            else:
                plt.show()
                print(title3)
            plt.close()
    else:     
        cg.ax_heatmap.set_xlabel('Pathway Pairings',size=20)
        cg.ax_heatmap.set_ylabel('Input Data',size=20)
        cg.ax_row_dendrogram.set_visible(False)
        cg.ax_col_dendrogram.set_visible(False)

        plt.setp(cg.ax_heatmap.get_xticklabels(),rotation='45',ha='right',size=fsX)
        plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=fsY,rotation='0')
        plt.title(title2,size=25,y=1,x=8)
        if(save):
            plt.savefig('Saved Images/'+title2+'.pdf',bbox_inches = "tight")
        else:
            plt.show()
            print(title2)
        plt.close()
    if fixedPos:
        col = pos.copy()
        drop = drop.copy()
    return col,drop

def __BNOutDict(graphLst,reverse=False,undirected=False,directed=False,dropImmune=False,allDir=False,sub1=True):
    if(allDir):
        outDict, nameLst = __BNOutDict(graphLst,directed=True,dropImmune=dropImmune,allDir=False,sub1=False)
        out1 = __BNOutDict(graphLst,directed=True,reverse=True,dropImmune=dropImmune,allDir=False,sub1=False)[0]
        out2 = __BNOutDict(graphLst,undirected=True,dropImmune=dropImmune,allDir=False,sub1=False)[0]
        outDict = {**outDict, **out1}
        outDict = {**outDict, **out2}
    else:
        immuneLst=['T.Eff','IFNg','Antigen']
        outDict={}
        nameLst=[]
        frst=True
        for graph in graphLst:
            outName = graph.split('\\')[1].replace('.csv','').strip()
            nameLst.append(outName)
            g = pd.read_csv(graph,index_col=0)
            if(frst):
                cols = g['from'].unique()
                for num in range(len(cols)):
                    pair1 = cols[num]
                    pair1 = pair1.replace('.Scores','')
                    for num2 in range(num,len(cols)):
                        pair2 = cols[num2]
                        pair2 = pair2.replace('.Scores','')
                        if(pair1!=pair2 and pair1!='Purity' and pair2!='Purity'):
                            val = (pair1 in immuneLst or pair2 in immuneLst)
                            if not (dropImmune and val):
                                if undirected:
                                    outDict[pair1+' -- '+pair2]=[]
                                else:
                                    if directed:
                                        if reverse:
                                            outDict[pair2+' -> '+pair1]=[]
                                        else:
                                            outDict[pair1+' -> '+pair2]=[]
                                    else:
                                        outDict[pair1+' '+pair2]=[]
                frst=False
            for pair in g.index:
                direct = g.loc[pair]['direction']
                strength = g.loc[pair]['strength'] 
                if(undirected):
                    fromm = g.loc[pair]['from']
                    to = g.loc[pair]['to']
                    g2 = g.loc[g['to'] == fromm]
                    g2 = g2.loc[g2['from']==to]
                    direct2 = g2['direction'].values[0]
                    if((direct < 0.35 or direct2 < 0.35) and sub1):
                        strength = strength-1
                else:
                    if(directed and direct < 0.65 and sub1):
                        strength = strength-1
                if(undirected):
                    pairing = g.loc[pair]['from']+' -- '+g.loc[pair]['to']
                else:
                    if(directed):
                        pairing = g.loc[pair]['from']+' -> '+g.loc[pair]['to']
                    else:
                        pairing = g.loc[pair]['from']+' '+g.loc[pair]['to']
                pairing = pairing.replace('.Scores','')
                if(direct==0):
                    strength = 0.0
                if(pairing in outDict):
                    outDict[pairing].append(strength)
    return outDict, nameLst

def __renameBNOut(dfIn):
    df = dfIn.copy()
    dfInx =[]
    for col in df.columns:
        if not col.split(',')[0] in dfInx: 
            dfInx.append(col.split(',')[0])
    newNames = getUniqueNames(dfInx)
    sepNum = math.ceil(len(df.columns)/len(newNames))
    newIdx=[]
    num=0
    newNamesNum=0
    for col in df.columns:
        if num ==sepNum:
            newNamesNum+=1
        lst = col.split(',')
        l0 = newNames[newNamesNum]
        l1 = lst[1]
        l2 = lst[2]
        newL = [l0,l1,l2]
        newL2 = [l0,l2]
        if(len(lst)>3):
            l3 = lst[3]
            newL.append(l3)
            newL2.append(l3)
        if(len(lst)>4):
            l4 = lst[4]
            newL.append(l4)
            newL2.append(l4)
        if(len(newNames)>1 or (l0 == 'Pre-BN')):
            newIdx.append(','.join(newL))
        else:
            newIdx.append(','.join(newL2))
        if(num==sepNum):
            num=1
        else:
            num +=1
    return newIdx, newNames, l1

def __onlyBGEorBDE(graphLst,onlyBGE=True,onlyBDE=False):
    if(onlyBDE or onlyBGE):
        if(onlyBGE):
            tpy='G'
        if(onlyBDE):
            tpy='MN'
        graphLst2=[]
        for graph in graphLst:
            ls = graph.split('\\')[1]
            if tpy in ls:
                graphLst2.append(graph)
        graphLst=graphLst2
    return graphLst

def renameBNLearnOut():
    graphLst = glob.glob('Run BNLearn/Outputs/*')
    for graph in graphLst:
        start = graph.split('\\')[0]
        ls = graph.split('\\')[1].split(',')
        g = ls[-1]
        if('BGE' in g):
            g = ' G.csv'
        if('BDE' in g):
            g = ' MN.csv'
        graphNew = ','.join([ls[0],ls[1],ls[2],ls[3],g])
        graphNew = '\\'.join([start,graphNew])
        os.rename(graph,graphNew)    
    return 1

def __BNGraphsTinyBegin(globDir,pathways,onlyBGE,onlyBDE):
    if not pathways:
        graphLst = glob.glob(globDir)
    else:
        graphLst = globExclude(globDir,exclude='Kobe Genes')
    graphLst = __onlyBGEorBDE(graphLst,onlyBGE=onlyBGE,onlyBDE=onlyBDE)
    return graphLst

def makeBNGraphsDBs(thres=0.5,onlyBGE=False,onlyBDE=False,merged=False,dirThres=0.65,below=False,thresFloor=0.3,
                    undirected=False,dropImmune=False,globDir='Run BNLearn/Outputs/*',dropPure=False,colored=False,
                    pathways=False,title='',excludePre=False,nonToIm=False,colorDict={},useDict={},
                    kobeColor=False,onlyConnected=True,erase=False,interPath=False,immuneNonOnly=False,mergedAll=False):
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=onlyBGE,onlyBDE=onlyBDE)
    dbLs = []
    for graph in graphLst:
        ls = graph.split('\\')[1].split(',')
        if('BN Merged' in ls[0]):
            db = ls[1].strip()
        else:
            db = ls[2].strip()
        dbLs.append(db)
    dbLs = list(set(dbLs))
    for db in dbLs:
        globDir2 = globDir+db+'*'
        title2 = 'DataBase is '+db+title+', '
        placeLs = makeBNGraphs(thres=thres,onlyBGE=onlyBGE,onlyBDE=onlyBDE,merged=merged,dirThres=dirThres,below=below,
                               title=title2,thresFloor=thresFloor,undirected=undirected,dropImmune=dropImmune,
                               globDir=globDir2,dropPure=dropPure,colored=colored,pathways=pathways,kobeColor=kobeColor,
                               onlyConnected=onlyConnected,erase=erase,interPath=interPath,immuneNonOnly=immuneNonOnly,
                               mergedAll=mergedAll,excludePre=excludePre,nonToIm=nonToIm,
                               colorDict=colorDict,useDict=useDict)
    if(mergedAll):
        placeLs.append('Post-BN Merged')
    return placeLs, dbLs

def makeBNGraphs(thres=0.5,onlyBGE=False,onlyBDE=False,merged=False,dirThres=0.65,below=False,thresFloor=0.3,
                 undirected=False,dropImmune=False,globDir='Run BNLearn/Outputs/*',dropPure=False,colored=False,
                 pathways=False,title='',excludePre=False,agnostic=False,nonToIm=False,
                 colorDict={},useDict={},blackList=False,blackListFile='',dictIn={},
                 kobeColor=False,onlyConnected=True,erase=False,interPath=False,immuneNonOnly=False,mergedAll=False,
                 allowPaths=False,depthOfPaths=1,rmvLen=False):
    """
    makeGraphs:
        Inputs:
            thres: the edge strength threshold that must be passed for an edge to be inlcuded in the graph
                default: 0.5
            onlyBGE and onlyBDE, booleans telling if to only create plots for BNLearn outputs that used 
                    'bge'/'bde' scoreing methods 
                deafult: False for both
            merged: boolean telling if to create a single graph for each cancer 
                    (using the average edge and direction strengths of all outputs related to that cacner), or plot each
                    output individually
                default: False
            dirThres: an edge in the graph will be deemed as directed if that edge's direction strength is 
                      greater than dirThres, if both directions of an edge are below dirThres that edge will be 
                      considered as undirected.
                deafult: 0.65
            undirected: boolean telling if to inlcude undirected edges in the graph (as defined by dirThres), or
                        ignore them and only plot directed edges
                default: False
        Actions:
            Using all the outputs from BNLearn (all files in the 'Run BNLearn/Outputs/' directory), this function creates a
                graphviz image from that output. Inlcuding all nodes from the output, and all edges that have a 
                strength/confidence > thres and the direction strength > dirThres. 
                If undirected=True will also inlcude edges where no direction is > dirThres, as undirected edges.
                Inlcudes a label on all edges with the edge strength and the direction strength (in parenthesis), 
                undirected edges only have the edge strength for a label.
            All the created graphs will be saved in the 'Graphs/' directory, if graphviz is intalled correctly will save pdfs
                of the graphviz image, else will be saved as a graphviz file that can be graphed with other graphviz software
                (http://graphviz.it/#/new)
            Output:
                On sucess returns 1.
    """
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=onlyBGE,onlyBDE=onlyBDE);placeLs = []
    if(excludePre):graphLst = __removeLstFunc(graphLst,'PrePure')
    for graph in graphLst:place = graph.split('\\')[1].split(',')[0].strip();placeLs.append(place)
    placeLs = list(set(placeLs));title2 = title+'All Methods'
    if(onlyBGE):title2 = title+'Only BGE'
    if(onlyBDE):title2 = title+'Only BDE'
    if(interPath):
        title2 += ', Only InterPathway Edges'
        if(immuneNonOnly):
            if nonToIm:title2 += ' Non-Immune Edges'
            else:title2 += ' Immune-Non Edges'
    if(immuneNonOnly):title += ', '
    title2 +=', Confidence '+str(thres)
    if not mergedAll:
        for place in tq(placeLs,desc='Making BN Graphs'):
            graphLst2 = [p for p in graphLst if place in p]
            helpMakeGraphs(graphLst=graphLst2,merged=merged,thres=thres,title=title2,undirected=undirected,interPath=interPath,
                           dirThres=dirThres,dropImmune=dropImmune,dropPure=dropPure,colored=colored,kobeColor=kobeColor,
                           agnostic=agnostic,nonToIm=nonToIm,pathways=pathways,blackList=blackList,
                           blackListFile=blackListFile,dictIn=dictIn,allowPaths=allowPaths,depthOfPaths=depthOfPaths,
                           colorDictIn=colorDict,useDict=useDict,mergedAll=False,rmvLen=rmvLen,
                           below=below,thresFloor=thresFloor,onlyConnected=onlyConnected,immuneNonOnly=immuneNonOnly)
    else:
        if(agnostic):
            __BNGraphsAgnosMerged(placeLs=placeLs,graphLst=graphLst,dropImmune=dropImmune,dropPure=dropPure,
                                  agnostic=agnostic,dirThres=dirThres,nonToIm=nonToIm,pathways=pathways,
                                  thres=thres,thresFloor=thresFloor,interPath=interPath,immuneNonOnly=immuneNonOnly,
                                  below=below,undirected=undirected,colored=colored,title=title,
                                  onlyBDE=onlyBDE,onlyBGE=onlyBGE)
        else:
            graphLst = __removeLstFunc(graphLst,'Pre-BN')
            helpMakeGraphs(graphLst=graphLst,merged=merged,thres=thres,title=title2,undirected=undirected,interPath=interPath,
                           dirThres=dirThres,dropImmune=dropImmune,dropPure=dropPure,colored=colored,kobeColor=kobeColor,
                           agnostic=agnostic,nonToIm=nonToIm,pathways=pathways,mergedAll=True,
                           colorDictIn=colorDict,useDict=useDict,blackList=blackList,rmvLen=rmvLen,
                           blackListFile=blackListFile,dictIn=dictIn,allowPaths=allowPaths,depthOfPaths=depthOfPaths,
                           below=below,thresFloor=thresFloor,onlyConnected=onlyConnected,immuneNonOnly=immuneNonOnly)
            
        makeBNGraphs(thres=thres,onlyBGE=onlyBGE,onlyBDE=onlyBDE,merged=merged,dirThres=dirThres,below=below,title=title,
                     thresFloor=thresFloor,undirected=undirected,dropImmune=dropImmune,globDir=globDir,dropPure=dropPure,
                     colored=colored,kobeColor=kobeColor,onlyConnected=onlyConnected,erase=erase,interPath=interPath,
                     excludePre=excludePre,useDict=useDict,colorDict=colorDict,
                     immuneNonOnly=immuneNonOnly,mergedAll=False,pathways=pathways,agnostic=agnostic,nonToIm=nonToIm)
    if(erase):
        for file in glob.glob('Graphs/*'):
            if not '.pdf' in file:os.remove(file)
    return placeLs

def helpMakeGraphs(graphLst,merged,thres,title,undirected,below,thresFloor,mergedAll,
                   dirThres,dropImmune,colored,dropPure,interPath,immuneNonOnly,dictIn,
                   kobeColor,onlyConnected,agnostic,nonToIm,pathways,blackList,blackListFile,
                   colorDictIn,useDict,allowPaths,depthOfPaths,rmvLen):
    if not merged:
        for graph in graphLst:
            outName = graph.split('\\')[1].replace('.csv','').strip()
            g = __getMergedG([graph],immuneDict=colorDict,dictIn=useDict,dropImmune=dropImmune,
                             dropPure=dropPure,agnostic=agnostic,thres=thres,nonToIm=nonToIm,below=False,
                             thresFloor=thresFloor,interPath=interPath,immuneNonOnly=immuneNonOnly,dirThres=dirThres,
                             pathways=pathways,allowPaths=allowPaths,depthOfPaths=depthOfPaths,rmvLen=rmvLen)
            outName = outName+', '+title
            __helpMakeGraphsCreate(g[0],thres,outName,undirected=undirected,kobeColor=kobeColor,onlyConnected=onlyConnected,
                                   immuneNonOnly=immuneNonOnly,inputsLst=g[2],colorDictIn=colorDictIn,
                                   useDict=useDict,blackList=blackList,blackListFile=blackListFile,
                                   dirThres=dirThres,colored=colored,below=below,thresFloor=thresFloor,interPath=interPath)
    else:
        g = __getMergedG(graphLst,immuneDict=colorImmuneDict,dictIn=colorDictIn,dropImmune=dropImmune,
                         dropPure=dropPure,agnostic=agnostic,dirThres=dirThres,nonToIm=nonToIm,pathways=pathways,
                         allowPaths=allowPaths,depthOfPaths=depthOfPaths,rmvLen=rmvLen,
                         thres=thres,thresFloor=thresFloor,interPath=interPath,immuneNonOnly=immuneNonOnly)
        g1P = g[1].replace('Kobe Genes ','');outName = g1P+' Average'
        if(g1P==''):return 0
        if(g1P!='Pre-BN Merged'):
            df = pd.read_csv('Purity/TPM/v1/TCGA_'+g1P+'_tpm.tsv',sep='\t',index_col=0)
            placeLen = str(len(df.columns));outName += ', N = '+placeLen
        if(mergedAll):outName = 'Post-BN Merged Average'
        if(agnostic):outName += ', Gene Agnostic'
        else:
            if(kobeColor):outName += ', Genes'
            else:outName += ', Pathways'
        if(below):outName += ', Confidence Range '+str(thresFloor)+' to '+str(thres)
        else:outName=outName+', Confidence Range '+str(thres)+' to '+str(1)
        outName += ', Direction '+str(dirThres)+', '+title
        if not agnostic:
            __helpMakeGraphsCreate(g[0],thres=thres,kobeColor=kobeColor,onlyConnected=onlyConnected,immuneNonOnly=immuneNonOnly,
                                   outName=outName,below=below,thresFloor=thresFloor,interPath=interPath,colorsLst=colorsLst,
                                   colorDictIn=colorDictIn,colorImmuneDict=colorImmuneDict,blackList=blackList,
                                   blackListFile=blackListFile,
                                   undirected=undirected,dirThres=dirThres,colored=colored,inputsLst=g[2])
        else:
            ____helpMakeGraphsCreateAgnos(g[0],outName=outName,undirected=undirected,colored=colored)
    return 1
def agnosticLists(direct,undirect,frm,to,frmLst,toLst,numLst,dirLst):
    g2 = direct[direct['from']==frm];g2 = g2[g2['to']==to];num = len(g2.index)
    frmLst.append(frm);toLst.append(to);numLst.append(num);dirLst.append(1);g2 = undirect[undirect['from']==frm]
    g2 = g2[g2['to']==to];num = len(g2.index);frmLst.append(frm);toLst.append(to);numLst.append(num);dirLst.append(0)
    return 1
def ____helpMakeGraphsCreateAgnos(df,outName,undirected=False,colored=True):
    Nodes = df['from'].unique();Nodes2 = df['to'].unique();Nodes = list(set(list(Nodes)+list(Nodes2)))
    direct = df[df['Directed']==1];undirect = df[df['Directed']==0];dot = Digraph()
    dot.attr(width="900pt", height="18000pt",fixedsize='true')
    if(colored):
        topLs=[];n1 = direct['from'].unique();n2 = direct['to'].unique()
        u1 = list(undirect['from'].unique());u2 = list(undirect['to'].unique());u1.extend(n1)
        u1 = list(set(u1));u2.extend(n2);u2 = list(set(u2));topLs=[]
        for row in u1:
            if not row in u2:topLs.append(row)
        colorPathDict={};kobe = getKobeClique();kobe2={}
        for path in kobe:
            lst = __runGenes(kobe[path]);kobe2[path]=lst
        immuneLst=['T-Eff','IFNg','Antigen Process'];colorImmune=['red','indianred1','crimson']
        colorLst=list(range(1,7));j=0;i=0
        for path in kobe2:
            lst = kobe2[path]
            if path in immuneLst:color = colorImmune[j];j+=1
            else:color = colorLst[i];i+=1
            colorPathDict[path] = color
        colorPathDict['Heat']= 'orangered';colorPathDict['Purity'] = 'grey';colorPathDict['Cancer Area'] = 'aliceblue'
    colorLst=[]    
    for node in Nodes:
        node = node.replace('Scores','');node = node.replace('.','')
        if(colored):
            color= colorPathDict[node]
            if(type(color)==str):dot.node(node,style='filled',color=color);colorLst.append(color)
            else:dot.node(node,style='filled',colorscheme='pastel26',color=str(color));colorLst.append(color)
        else:dot.node(node)
        dot.node(node+'3',style='invis');dot.node(node+'4',style='invis')
    for i in direct.index:
        if colored:color='blue'
        else:color ='blue'
        if(type(color)==str):
            dot.edge(direct['from'][i],direct['to'][i],label=str(direct['Number of Connections'][i]),color=color)
        else:
            dot.edge(direct['from'][i],direct['to'][i],label=str(direct['Number of Connections'][i]),
                     colorscheme='pastel26',color=str(color))
        mid = direct['from'][i]+'3';mid2 = direct['from'][i]+'4'
        dot.edge(direct['from'][i],mid,style='invis')
        dot.edge(mid,mid2,style='invis');dot.edge(mid2,direct['to'][i],style='invis')
    if(undirected):
        for i in undirect.index:
            dot.edge(undirect['from'][i],undirect['to'][i],label=str(undirect['Number of Connections'][i]),
                     arrowhead='none')
            mid = undirect['from'][i]+'3';mid2 = undirect['from'][i]+'4'
            dot.edge(undirect['from'][i],mid,style='invis')
            dot.edge(mid,mid2,style='invis');dot.edge(mid2,undirect['to'][i],style='invis')
    if(colored):
        if(len(Nodes)>0):
            legend = Digraph(name='clusterLegend');legend.attr(label='Legend')
            for path in colorPathDict:
                color = colorPathDict[path]
                if(color in colorLst):
                    if(type(color)==str):legend.node(path+'1',style='filled',color=color,label='')
                    else:legend.node(path+'1',style='filled',colorscheme='pastel26',color=str(color),label='')
                    legend.node(path+'2',label=path,shape="plaintext");legend.edge(path+'1',path+'2',style='invis')
            dot.subgraph(legend);n=0;addOn = max((len(topLs)//len(colorPathDict)),1)
            if(len(topLs)>0):
                for path in colorPathDict:
                    color = colorPathDict[path]
                    if(color in colorLst):
                        node = topLs[n];dot.edge(path+'2',node,style='invis')
                        if(n<len(topLs)-addOn):n+=addOn
                        else:break
    outName = outName.replace(', ,',",");dot.attr(label=outName);dot.attr(fontsize='20')
    outName = outName.replace(', ,',',')
    try:dot.render('Graphs/'+outName,view=False)
    except:x=1
    return 1

def __getMergedG(graphLst,thres,thresFloor,dirThres,dropImmune,dropPure,agnostic,pathways,immuneDict,dictIn,
                 below,interPath,immuneNonOnly,nonToIm,allowPaths=False,depthOfPaths=1,rmvLen=False):
    if allowPaths:interPath=False;immuneNonOnly=False;nonToIm=False
    immuneLst=['T.Eff','IFNg','Antigen'];dfDirLst=[];dfStrLst=[];inputsLst=[]
    for graph in graphLst:
        inputName = graph.replace('Outputs','Inputs').replace(', G.csv','').replace(', MN.csv','');inputsLst.append(inputName)
        outName = graph.split('\\')[1].replace('.csv','').strip().split(',')[0]
        df = pd.read_csv(graph,index_col=0);df.index = df['from']+'_'+df['to'];df.index.name='Index'
        df=df.drop(columns=['from','to']);
        if dropPure or dropImmune:
            dropLst=[]
            for idx in df.index:
                if(('Purity' in idx) and dropPure):dropLst.append(idx)
                pair1 = idx.split('_')[0];pair2 = idx.split('_')[1]
                if( dropImmune and (pair1 in immuneLst or pair2 in immuneLst)):dropLst.append(idx)
            df = df.drop(dropLst)
        dfDirLst.append(('',df.drop(columns=['strength'])));dfStrLst.append(('',df.drop(columns=['direction'])))
    dfAllDir = getdfAll(dfDirLst,idxName='Index');dfAllStr = getdfAll(dfStrLst,idxName='Index')
    if type(dfAllStr) == list or type(dfAllDir) == list:
        g=pd.DataFrame();return(g,'',[])
    strength = dfAllStr.T.mean();idxLst=[]
    for idx in strength.index:
        idx = idx.replace('.Scores','').replace('.','');idxLst.append(idx)
    strength.index=idxLst;direction = dfAllDir.T.mean();direction.index=idxLst;toLst=[];fromLst=[]
    for idx in strength.index:
        elem = idx.split('_');fromLst.append(elem[0]);toLst.append(elem[1])
    g = pd.DataFrame([fromLst,toLst,strength,direction],['from','to','strength','direction']).T
    if not(below):g = g[g['strength']>=thres]
    else:g = g[g['strength']<thres];g = g[g['strength']>=thresFloor]
    if(interPath):
        g = interPathwayOnly(g,immuneDict=immuneDict,dictIn=dictIn,immuneNonOnly=immuneNonOnly,nonToIm=nonToIm,
                             pathways=pathways)
    newToLst=[];newFrmLst=[]
    
    #for row in g.index:
    #    print(g.loc[row])
    #    newFrmLst.append(conversionDict[g.loc[row]['from']]);newToLst.append(conversionDict[g.loc[row]['to']])
    #g['from']=newFrmLst;g['to']=newToLst
    if allowPaths:
        g=__getNonToImmPath(depthOfPaths=depthOfPaths,immuneDict=immuneDict,g=g,dictIn=dictIn,immuneNonOnly=immuneNonOnly,
                            nonToIm=nonToIm,rmvLen=rmvLen)
    if(agnostic):
        direct = g[g['direction']>=dirThres];dropLst=[]
        for pair in g.index:
            direct1 = g.loc[pair]['direction'];fromm = g.loc[pair]['from'];to = g.loc[pair]['to'];g2 = g.loc[g['to'] == fromm]
            if(len(g2.index)>0):
                g2 = g2.loc[g2['from']==to];direct2 = g2['direction'].values[0]
                if not pair in dropLst:dropLst.append(g2.index[0])
                if(direct1>dirThres or direct2>dirThres):dropLst.append(pair)
        undirect = g.drop(dropLst);kobe = getKobeClique(True);__agnosKobe(direct);__agnosKobe(undirect)
        ls = list(set(kobe.values()));dirLst=[];numLst=[];frmLst=[];toLst=[]
        for i in range(len(ls)):
            for j in range(i+1,len(ls)):
                to = ls[i];frm = ls[j]
                agnosticLists(direct,undirect,frm,to,frmLst,toLst,numLst,dirLst);to = ls[j];frm = ls[i]
                agnosticLists(direct,undirect,frm,to,frmLst,toLst,numLst,dirLst)
        df = pd.DataFrame([frmLst,toLst,numLst,dirLst]);df.index = ['from','to','Number of Connections','Directed']
        df =df.T;df = df[df['Number of Connections']>0];g = df.dropna()
    return (g,outName,inputsLst)
def __agnosKobe(df):
    kobe = getKobeClique(True)
    for row in df.index:
        frm = df.loc[row]['from'];too = df.loc[row]['to'];frmPath = kobe[frm];tooPath = kobe[too]
        df.loc[row]['from'] = frmPath;df.loc[row]['to'] = tooPath
    return 1
def __getCorrInputsLst(inputsLst,clique=False):
    conversionDict={'Heat':'Heat'};conversionDict['Heat Scores']='HeatScores';frst=True
    for fileName in inputsLst:
        place=fileName.split(',')[0]
        if clique:
            place=fileName.split(',')[1];lst=place.split(' N ');place=','.join([lst[0],' N '+lst[1]])
            dfNonImm=pd.read_csv('Cliques/Overlapping Cliques, Non-Immune,'+place+'.csv')
            dfImm=pd.read_csv('Cliques/Overlapping Cliques, Immune,'+place+'.csv')
            for i in dfImm.columns:
                conversionDict['Immune'+str(i)]='Immune'+str(i)+'_'+str(len(dfImm[i].dropna()))
                conversionDict['Immune '+str(i)]='Immune'+str(i)+'_'+str(len(dfImm[i].dropna()))
            for i in dfNonImm.columns:
                conversionDict['NonImmune'+str(i)]='NonImmune'+str(i)+'_'+str(len(dfNonImm[i].dropna()))
                conversionDict['Non Immune '+str(i)]='NonImmune'+str(i)+'_'+str(len(dfNonImm[i].dropna()))
        # Sanatize to avoid removal
        fileName=fileName.replace('Glioblastoma','xGlioblastoma') 
        # Remove Extra info
        fileName=fileName.replace(', G','').replace(', NM','').replace(', Sparse','').replace('.csv','')
        # Safe to bring back G
        fileName=fileName.replace('xG','G')
        inpt = pd.read_csv(fileName,sep='\t',index_col=0)
        if frst:
            spearTotal=inpt.corr(method='spearman');pearTotal=inpt.corr(method='pearson');frst=False
        else:
            spear=inpt.corr(method='spearman');pear=inpt.corr(method='pearson');spearTotal += spear;pearTotal += pear
    spearTotal /= len(inputsLst);pearTotal /= len(inputsLst);corr=(spearTotal+pearTotal)/2;newLst=[]
    for i in corr.index:newLst.append(i.replace('-','.').replace(' ','.'))
    corr.index=newLst;corr.columns=newLst
    return corr

def __getNode2(node,antigen=True,scores=False,corr=True,rmvLen=False):
    node2 = node
    if corr:
        if '-' in node: node2=node.replace('-','.')
        if(node=='HLAA'):node2 = 'HLA.A'
        if('MT' in node):
            splt = node.split('MT')[-1]
            node2='MT.'+splt
    if(antigen and (node=='PI3K' or node=='AKT')):node2 = 'PI3K/AKT'
    if(node=='TEff'):node2='T-Eff'
    if(antigen and node=='Antigen'):node2='Antigen.Process'
    if scores and 'Purity' in node:node2 = 'Purity.Scores'
    if scores and 'Heat' in node:node2 = 'Heat.Scores'
    if 'Cancer' in node:node2 = 'Cancer.Area'
    if rmvLen:node2 = node.split('_')[0]
    return node2

def __helpMakeGraphsCreate(g,thres,outName,undirected,dirThres,kobeColor,immuneNonOnly,colorDictIn,useDict,
                           colored,onlyConnected,below,thresFloor,interPath,inputsLst,blackList,blackListFile):
    setupOut = __setupHelpMakeGraphsCreate(g=g,undirected=undirected,dirThres=dirThres,colored=colored,
                                           blackList=blackList,blackListFile=blackListFile,
                                           colorDictIn=colorDictIn,useDict=useDict,onlyConnected=onlyConnected)
    Nodes,direct,undirect,colorDict,colorDictPath,topLs = setupOut
    numNodes = len(Nodes)/10;DPI = str(1000);FS = (round(numNodes*10)+20);NodeFS = str(0.75*FS);FS=str(FS)
    numNodes = numNodes/2
    if(numNodes)>0.5:RankSep = str(numNodes*3)
    else:RankSep = str(0.5)
    outName = outName.replace(', ,',',')
    dot = Digraph();dot.attr(ranksep=RankSep,dpi=DPI,fontsize=FS,label=outName,size='10,15!',fixedsize='true');colorLst=[]
    for node in Nodes:
        node = node.replace('Scores','');node = node.replace('.','')
        if(colored):
            color= colorDict[node]
            if(color[0]==0):dot.node(node,style='filled',color=color[1],fontsize=NodeFS);colorLst.append(color)
            else:dot.node(node,style='filled',colorscheme=color[1],color=str(color[0]),fontsize=NodeFS);colorLst.append(color)
        else:dot.node(node)
    corr = __getCorrInputsLst(inputsLst);corr2 = corr.copy()
    for i in direct.index:
        frm=direct['from'][i];to=direct['to'][i];frm2 = __getNode2(frm,antigen=False,scores=True)
        to2 = __getNode2(to,antigen=False,scores=True);corrVal=corr2.loc[frm2][to2]
        color = str(__corrValColor(corrVal,rg=True))
        if corrVal > 0:
            dot.edge(frm,to,label=str(round(direct['strength'][i],2))+
                     ' ('+str(round(direct['direction'][i],2))+')',color=color,fontsize=NodeFS)
        else:
            dot.edge(frm,to,label=str(round(direct['strength'][i],2))+
                     ' ('+str(round(direct['direction'][i],2))+')',color=color,fontsize=NodeFS,arrowhead='tee')
    if(undirected):
        for i in undirect.index:
            frm=undirect['from'][i];to=undirect['to'][i]
            frm2 = __getNode2(frm,antigen=False,scores=True);to2 = __getNode2(to,antigen=False,scores=True)
            corrVal=corr2.loc[frm2][to2];color = str(__corrValColor(corrVal,rg=True))
            dot.edge(frm,to,label=str(round(undirect['strength'][i],2)),
                     arrowhead='none',color=color,style='dashed',fontsize=NodeFS)
    if(colored):
        legend = Digraph(name='clusterLegend');legend.attr(label='Legend')
        for path in colorDictPath:
            color = colorDictPath[path]
            if(color[0]==0):legend.node(path+'1',style='filled',color=color[1],label='')
            else:legend.node(path+'1',style='filled',colorscheme=color[1],color=str(color[0]),label='')
            legend.node(path+'2',label=path,shape="plaintext",fontsize=str(float(NodeFS)*0.75))
            legend.edge(path+'1',path+'2',style='invis')
        dot.subgraph(legend);n=0;addOn = max((len(topLs)//len(colorDictPath)),1)
        if len(topLs)>0:
            for path in colorDictPath:
                node = topLs[n];dot.edge(path+'2',node,style='invis')
                if(n<len(topLs)-addOn):n+=addOn
                else:break
    try:dot.render('Graphs/'+outName,view=False)
    except:x=1
    return 1
def __setupHelpMakeGraphsCreate(g,undirected,dirThres,colored,onlyConnected,colorDictIn,useDict,
                                blackList,blackListFile):
    if blackList:
        dropNumLst=[];blDf = pd.read_csv(blackListFile,index_col=0)
        for gene in blDf.index:
            lst = list(blDf.loc[gene].dropna())
            l1 = list(g[g['from'] == gene].index);l2 = list(g[g['to'] == gene].index);l3=l1+l2
            for i in l3:
                if ((g.loc[i]['from'] == gene and g.loc[i]['to'] in lst) or 
                    (g.loc[i]['to'] == gene and g.loc[i]['from'] in lst)):dropNumLst.append(i)
        dropNumLst = list(set(dropNumLst));newIdx=[]
        for i in g.index:
            if not i in dropNumLst:newIdx.append(i)
        g=g.loc[newIdx]
    Nodes = g['from'].unique()
    Nodes2 = g['to'].unique();Nodes = list(set(list(Nodes)+list(Nodes2)));direct = g[g['direction']>=dirThres]
    n1 = direct['from'].unique();n2 = direct['to'].unique();topLs=[]
    for row in n1:
        if not row in n2:topLs.append(row)
    undirect=0
    if(undirected):
        dropLst=[]
        for pair in g.index:
            save=False;direct1 = g.loc[pair]['direction'];strength = g.loc[pair]['strength'] 
            fromm = g.loc[pair]['from'];to = g.loc[pair]['to'];g2 = g.loc[g['to'] == fromm];g2 = g2.loc[g2['from']==to]
            if(g2.index>0):
                direct2 = g2['direction'].values[0]
                if not pair in dropLst and (direct1<dirThres and direct2<dirThres):save=True
            if not save:dropLst.append(pair)
        undirect = g.drop(dropLst)
        for row in direct.index:
            pairF = direct.loc[row]['from'];pairT = direct.loc[row]['to']
        u1 = list(undirect['from'].unique());u2 = list(undirect['to'].unique())
        u1.extend(n1);u1 = list(set(u1));u2.extend(n2);u2 = list(set(u2));topLs=[]
        for row in u1:
            if not row in u2:topLs.append(row)
    if(onlyConnected):    
        n1 = list(direct['from'].unique());n2 = list(direct['to'].unique());Nodes = list(set(n1+n2))
        if(undirected):
            n3 = list(undirect['from'].unique());n4 = list(undirect['to'].unique());Nodes = list(set(n1+n2+n3+n4))
    colorDictOut={};colorDictPath={}
    if(colored):
        colorDictOut={}
        for node in Nodes:
            path = useDict[node]
            color = colorDict[path]
            colorDictOut[node]=color
            if not path in colorDictPath:
                colorDictPath[path]=color
    return Nodes,direct,undirect,colorDictOut,colorDictPath,topLs

def interPathwayOnly(dfIn,immuneDict,dictIn,immuneNonOnly,nonToIm,pathways=False,graphviz=True,rmvLen=False):
    gDF = dfIn.copy();kobeDict=dictIn.copy();keepLst=[];specialLst=['Heat','Cancer Area','Purity']
    if(graphviz):
        for row in gDF.index:
            val2=True;ser = gDF.loc[row];frm = __getNode2(ser['from'],rmvLen=rmvLen);too=__getNode2(ser['to'],rmvLen=rmvLen)
            if(pathways):node1Path = frm;node2Path = too
            else:
                if frm in specialLst or too in specialLst:
                    if frm in specialLst:node1Path = frm
                    else:
                        if frm in kobeDict:node1Path = kobeDict[frm]
                        else:val2=False
                    if too in specialLst:node2Path = too
                    else:
                        if too in kobeDict:node1Path = kobeDict[too]
                        else:val2=False
                else:
                    if frm in kobeDict and too in kobeDict:node1Path=kobeDict[frm];node2Path=kobeDict[too]
                    else:val2=False
            val=True
            if(immuneNonOnly and nonToIm):
                direct = gDF.loc[row]['direction']
                if direct<0.35:val=False
            if(val and val2):
                if(__immuneNonPaths(node1Path=node1Path,node2Path=node2Path,immuneNonOnly=immuneNonOnly,
                                    nonToIm=nonToIm,immuneDict=immuneDict)):keepLst.append(row)
        gDF=gDF.loc[keepLst]
    else:
        for col in gDF.columns:
            val=True;colL = col.split()
            if(allDir and len(colL)<2):val = False
            if len(colL)>2:node1 = colL[0];node2 = colL[2]
            else:node1 = colL[0];node2 = colL[1]
            if(pathways):node1Path = node1.replace('.','');node2Path = node2.replace('.','')
            else:node1Path = kobeDict[node1.replace('.','')];node2Path = kobeDict[node2.replace('.','')]
            if(node1Path=='Heat'):val=False
            if(val):
                if(__immuneNonPaths(node1Path=node1Path,node2Path=node2Path,immuneNonOnly=immuneNonOnly,
                                    nonToIm=nonToIm,graphviz=False,immuneDict=immuneDict)):keepLst.append(col)
        gDF = gDF[keepLst]
    return gDF

def __immuneNonPaths(node1Path,node2Path,immuneDict,immuneNonOnly=False,nonToIm=False,graphviz=True):
    if(node1Path==node2Path):return False
    if(immuneNonOnly):
        if nonToIm:
            if((node2Path in immuneDict) and (not node1Path in immuneDict)):return True
            else:return False
        else:
            if(node1Path in immuneDict):
                if(node2Path in immuneDict):return False
                else:return True
            else:
                if(node2Path in immuneDict):return True
                else:return False
    else:return True
def __changeDrop(dropLst,directed=False,reverse=False,undirected=False):
    if not directed and not reverse and not undirected:return dropLst
    else:
        outLst=[]
        for elem in dropLst:
            if(undirected):outLst.append(elem.replace(' ',' -- '))
            if(directed):
                if(reverse):
                    pairs = elem.split(' ');pair1 = pairs[0];pair2 = pairs[1];outLst.append(pair2+' -> '+pair1)
                else:outLst.append(elem.replace(' ',' -> '))
        return outLst 

def mergeGraphs(directory='/Users/USER/Downloads/',title='Average BNLearn',erase=False):
    """Merges together all BN plots within a given directory"""
    images = glob.glob(directory+'*graphviz*png*');from PIL import Image;pdfLst=[]
    for image in images:
        image1 = Image.open(image)
        im1 = image1.convert('RGB');imageName = image.split('.')[0];im1.save(imageName+'.pdf');pdfLst.append(imageName+'.pdf')
    __mergePDFs(pdfLst,title)
    for pdf in pdfLst:os.remove(pdf)
    if erase:
        for image in images:os.remove(image)
    return 1
def setupBNLearnGenes(pLst,runVLst=[],title='',panCancer=False,onlyKobeGenes=True,genesIn=[],blackList=False,
                      blDict={},blPlaceLst=[],blCutoff=5,blTitle='',tqOn=False,areaTitle='Kobe Genes'):
    if onlyKobeGenes:
        kobe = getKobeClique();geneLst=[]
        for path in kobe:
            genes = __runGenes(kobe[path]);geneLst.extend(genes)
    else:geneLst = genesIn.copy()
    for runVersion in runVLst:
        if panCancer or blackList:dfLst=[];dfLstBl=[];cancerNum=1
        inputs=pLst
        if tqOn:inputs=tq(pLst,desc=str(runVersion))
        for place in inputs:
            if(runVersion!='PrePure'):
                df = pd.read_csv('Purity/TPM/v'+str(runVersion)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
            else:df = pd.read_csv('Purity/TPM/Outliers Fixed/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
            placeLen = len(df.columns);gL = __goodGenes(geneLst,df.index); df = df.loc[gL]
            
            
            if blackList:dfLstBl.append((place,df))
            if panCancer :
                df = df.T;sampleLen = min(100,placeLen);samples = random.sample(range(0, placeLen), sampleLen)
                df = df.iloc[samples].T;df = __pureDf(df,zscore=True);df = __heatDF(df,place,zscore=True)[0];df =df.T
                df.insert(len(df.columns),'Cancer Area',float(cancerNum));df.index.name='Genes';df =df.T
                dfLst.append((place,df));cancerNum +=1
            else:
                df = __pureDf(df,zscore=True);df = __heatDF(df,place,zscore=True)[0];df=df.T
                if(runVersion!='PrePure'):
                    df.to_csv('Run BNLearn/Inputs/'+areaTitle+', '+place+', N = '+str(placeLen)+title+', v'+str(runVersion),
                              sep='\t')
                else:
                    df.to_csv('Run BNLearn/Inputs/'+areaTitle+', '+place+', N = '+str(placeLen)+title+', '+str(runVersion),
                              sep='\t')
        if panCancer:
            dfAll = getdfAll(dfLst);dfAll = dfAll.T
            if(runVersion!='PrePure'):
                title2= 'Run BNLearn/Inputs/'+areaTitle+', Pre-BN Merged'+title+', v'+str(runVersion)
                dfAll.to_csv(title2,sep='\t')
            else:
                title2= 'Run BNLearn/Inputs/'+areaTitle+', Pre-BN Merged'+title+', '+str(runVersion)
                dfAll.to_csv(title2,sep='\t')
        if blackList:
            __setupBNLearnBlackList(blDict=blDict,blPlaceLst=blPlaceLst,cutoff=blCutoff,title=blTitle+', v'+str(runVersion),
                                    dfLst=dfLstBl)        
    return 1

def __setupBNLearnBlackList(blDict,dfLst,blPlaceLst,cutoff=5,title=''):
    keys = list(blDict.keys());placeBlDict={}
    for place, df in tq(dfLst):
        blackLstDict={}
        if place in blPlaceLst:
            for i in range(len(keys)):
                pathway1 = keys[i];blLst=[]
                for j in range(len(keys)):
                    pathway2 = keys[j]
                    if not pathway1 == pathway2:
                        genes1 = blDict[pathway1];genes1=__goodGenes(genes1,df.index)
                        genes2 = blDict[pathway2];genes2=__goodGenes(genes2,df.index);bl=True
                        for g1 in genes1:
                            for g2 in genes2:
                                a=df.loc[g1];b=df.loc[g2];cor=stats.spearmanr(a, b)[0];cor+=stats.pearsonr(a,b)[0];cor/=2
                                if cor > 0.3:
                                    bl=False;break
                        if bl:
                            if not (pathway1,pathway2) in blLst:blLst.append((pathway1,pathway2))
                blackLstDict[pathway1]=blLst
            placeBlDict[place] = blackLstDict
    blPaths={}
    for place in placeBlDict:
        dict2 = placeBlDict[place]
        for a in dict2:
            for b in dict2[a]:
                if b in blPaths:blPaths[b] += 1
                else:blPaths[b] = 1
    blDf = pd.DataFrame.from_dict(blPaths,orient='index',columns=['Number'])
    blDf = blDf[blDf['Number']>cutoff];pairDf = __getPairDf(blDf);pairDf.to_csv('Run BNLearn/BlackList'+title+'.csv')
    return 1

def __getPairDf(df):
    pairDict={}
    for pair in df.index:
        path1 = pair[0].strip();path2=pair[1].strip()
        for gene1 in blDict[path1]:
            lst = blDict[path2].copy()
            if gene1 in pairDict:pairDict[gene1].extend(lst)
            else:pairDict[gene1]=lst
        gc.collect()
    maxLen=0
    for g in pairDict:
        if len(pairDict[g])>maxLen:maxLen = len(pairDict[g])
    for g in pairDict:
        lst = pairDict[g].copy()
        for i in range(maxLen-len(pairDict[g])):lst.append('')
        pairDict[g] = lst
    pairDf = pd.DataFrame.from_dict(pairDict, orient='index')
    return pairDf
def shiftLst(lst,num):
    lst2=[lst[num]];i=0
    for elem in lst:
        if i != num:lst2.append(elem)
        i += 1
    return lst2
def plotHistograms(pLst,size=(60,20),save=False,show=True):
    import math
    dfLst=[];fsT = round(size[0]/2)+5;fsL = round(size[0]/2);legSize = round(size[0]/2)
    for place in tq(pLst,desc='dfLst'):
        df = pd.read_csv('Purity/TPM/v'+str(7)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        df2 = pd.read_csv('Purity/TPM/v'+str(8)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        df = df.loc[gL];df2 = df2.loc[gL];dfLst.append((df,df2,place))
    for gene in tq(gL,desc='Plotting Histograms'):
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=size);axNum=0;height = 0 
        for dfTup in dfLst:
            ax = axes[axNum];df = dfTup[0];df2 = dfTup[1]
            place = dfTup[2];dfG1 = df.loc[gene];dfG2 = df2.loc[gene]
            weights = np.ones_like(dfG1)/float(len(dfG1));ax.hist(dfG1,label='v7',bins=100,weights=weights,alpha=0.5)
            ax.hist(dfG2,label='v8',bins=100,weights=weights,alpha=0.5);ax.legend(prop={'size': legSize})
            ax.set_title(gene+', '+place,fontsize=fsT);ax.set_xlabel('ZScore of Log2 TPM Gene Expression',fontsize=fsL)
            ax.set_ylabel('Percent of data',fontsize=fsL);axNum += 1;bot, top = ax.get_ylim()
            if(top > height):height = top
        for ax in axes:ax.set_ylim(0,height)
        if(save):
            plt.savefig('Saved Images/'+gene+', Histogram.pdf',bbox_inches = "tight");plt.close()
        if(show):plt.show()
    return 1

def BNLearnColColors(df,reverse=False,undirected=False,directed=False):
    colors = ['green','yellow','purple','black','orange','grey','white',
              'brown','fuchsia','pink','gold','seagreen','salmon','olive','teal','crimson','cyan']
    kobe = getKobeClique(True);lutCol = dict(zip(set(kobe.values()), colors));col1=[];col2=[];sep=' '
    if(undirected):sep= ' -- '
    else:
        if(directed or reverse):sep=' -> '
    for row in df:
        split = row.split(sep);frst = split[0].replace('.','');snd = split[1].replace('.','')
        path1 = kobe[frst];path2 = kobe[snd];col1.append(lutCol[path1]);col2.append(lutCol[path2])
    colColors=[col1,col2]
    return colColors, lutCol

def orderShiftLst(files,orderLst=[]):
    for order in orderLst:
        n=0
        for file in files:
            if order in file:take=n
            n += 1
        files = shiftLst(files,take)
    return files

def globExclude(globDir,exclude):
    files = glob.glob(globDir);files2=[]
    for file in files:
        if not exclude in file:files2.append(file)
    return files2

def bnLearnRowColors(ls,colors):
    places=[]
    for elem in ls:places.append(elem.split(',')[0])
    places = list(set(places));n=0
    if(len(places)>len(colors)):sys.exit('ERROR, Not Enough Colors for Rows')
    place_colors={}
    for place in places:color = colors[n];place_colors[place]=color;n+=1
    row_colors={}
    for elem in ls:place = elem.split(',')[0];color = place_colors[place];row_colors[elem] = color
    return row_colors, place_colors

def mergeBNGraphsAgnos(thresLst,orderLst,dirThres=0.65,merged=True,undirected=True,excludePre=True,onlyBGE=True,erase=True,
                       mergedAll=True,onlyBDE=False,interPath=True,immuneNonOnly=True,colored=True,nonToIm=False):
    for thres in tq(thresLst,desc='Overall'):
        pl = makeBNGraphs(globDir='Run BNLearn/Outputs/*Kobe Genes*',pathways=False,thres=thres,dirThres=dirThres,
                          merged=merged,undirected=undirected,excludePre=excludePre,onlyBGE=onlyBGE,erase=erase,
                          colored=colored,nonToIm=nonToIm,
                          agnostic=True,mergedAll=mergedAll,onlyBDE=onlyBDE,interPath=interPath,immuneNonOnly=immuneNonOnly)
        files = glob.glob('Graphs/*');files = orderShiftLst(files,orderLst);nameL = files[3].split('\\')[1].split(',')
        if(len(nameL)>5):name = ",".join(['Graphviz Graphs',nameL[2],nameL[3],nameL[4],nameL[5]]).strip()
        else:name = ",".join(['Graphviz Graphs',nameL[1],nameL[2],nameL[4]]).strip()
        name = name.replace('.pdf','');name = 'Saved PDFs/Agnostic/'+name;__mergePDFs(files,name,erase=True)
    return 1

def __corrValColor(corVal,minn=-1,sepLen=12,rg=False):
    if not rg:
        sep = 2/sepLen;val=True
        if(corVal==0):val=False
        if(corVal>0 and corVal-sep<=0):val=False
        if(corVal<0 and corVal+sep>=0):val=False
        corVal2 = corVal;color = 0
        while(corVal2-minn >= sep):
            color +=1;corVal2 += minn*sep
        if val:
            if corVal < 0:icolor += 1
            else:color -= 1
    else:
        if abs(corVal) <= 0.2:color='black'
        else:
            if corVal > 0:color = 'green'
            else:color='red'
    return color

In [12]:
def __BNGraphsAgnosMerged(placeLs,graphLst,dropImmune,dropPure,agnostic,dirThres,nonToIm,pathways,thres,thresFloor,
                          interPath,below,undirected,colored,title,onlyBDE,onlyBGE,immuneNonOnly):
    gLst=[]
    for place in placeLs:
        graphLst2 = [p for p in graphLst if place in p]
        g = __getMergedG(graphLst2,dropImmune=False,dropPure=False,agnostic=agnostic,dirThres=0.65,nonToIm=nonToIm,
                         pathways=pathways,thres=thres,thresFloor=0.3,interPath=interPath,
                         immuneNonOnly=immuneNonOnly,below=False)[0]
        if(len(g.index)>0):
            gLst.append(g)
    maxId = 0
    for g in gLst:
        ls = max(list(g.index))
        if ls > maxId:
            maxId = ls
    maxId += 1
    frmLst=[]
    toLst=[]
    dirLst=[]
    canLst=[]
    iLst=[]
    for i in range(maxId):
        canNum=0
        frst=True
        for g in gLst:
            if i in g.index:
                canNum +=1
                if(frst):
                    frmLst.append(g.loc[i]['from'])
                    toLst.append(g.loc[i]['to'])
                    dirLst.append(g.loc[i]['Directed'])
                    frst=False
        if not frst:
            canNum = str(canNum)+'/'+str(len(gLst));canLst.append(canNum);iLst.append(i)
    df = pd.DataFrame([frmLst,toLst,canLst,dirLst]);df.index = ['from','to','Number of Connections','Directed']
    df =df.T;df.index = iLst;outName = 'Post-BN Merged Average, Gene Agnostic'
    if(below):
        outName += ', Confidence Range '+str(thresFloor)+' to '+str(thres)
    else:
        outName=outName+', Confidence Range '+str(thres)+' to '+str(1)
    outName += ', Direction '+str(dirThres)
    if onlyBDE:outName += ', Only BDE'
    if onlyBGE:outName += ', Only BGE'
    if interPath:
        outName += ', Only InterPathway'
        if immuneNonOnly:addOn = ' Immune-Non Edges'
        if nonToIm:addOn = ' Non-Immune Edges'
        outName += addOn
    outName += title
    ____helpMakeGraphsCreateAgnos(df,outName=outName,undirected=undirected,colored=colored)
    return 1
def __hotColdDfLstHelp(place,vNum,extra):
    fileName = 'Genes/TPM/TCGA_'+place+'_tpm.fullIDs.remapped.tsv'
    df = pd.read_csv(fileName,index_col=0,sep='\t');__renameTPM(df);Hot,Cold = __getHotCold(df,extra=extra)
    if vNum != 'PrePure':fileName = 'Purity/TPM/v'+str(vNum)+'/TCGA_'+place+'_tpm.tsv'
    df = pd.read_csv(fileName,index_col=0,sep='\t');hotCols = __goodGenes(Hot.columns,df.columns);Hot = df[hotCols]
    coldCols = __goodGenes(Cold.columns,df.columns);Cold = df[coldCols]
    return (place,hotCols,coldCols,df)
def getTsvFromMsig():
    for file in glob.glob('MsigDB/*.gmt'):
        f = open(file)
        outDict={}
        for line in f:
            line = line.replace('\n','');ls = line.split('\t');name = ls[0];lst = ls[2:];outDict[name] = [lst]
        df = pd.DataFrame.from_dict(outDict);df.index=['Genes'];df.to_csv(file+'.tsv',sep='\t')
    return 1
def getFullDict(outputLst=True,useLst=[]):
    fullDict=[]
    for file in glob.glob('MsigDB/*.gmt'):
        outDict={}
        fName=file.split('\\')[1].replace('.v7.2.symbols.gmt','')
        f = open(file)
        for line in f:
            line = line.replace('\n','')
            ls = line.split('\t')
            name = ls[0]
            lst = ls[2:]
            outDict[name] = lst
        fullDict.append((fName,outDict))
    if outputLst:
        return fullDict
    else:
        outDict={}
        for name,dic in fullDict:
            if name in useLst:
                outDict.update(dic)
        return outDict

In [13]:
def __getBNnodes(pathways,place,thres):
    if pathways:
        globDir='Run BNLearn/Outputs/*'
    else:
        globDir='Run BNLearn/Outputs/*Kobe Genes*'
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=True,onlyBDE=False)
    graphLst = __removeLst(graphLst,'PrePure')
    graphLst = [p for p in graphLst if place in p]
    g = __getMergedG(graphLst,dropImmune=False,dropPure=False,agnostic=False,dirThres=0.65,nonToIm=False,
                     pathways=pathways,thres=thres,thresFloor=0,interPath=False,immuneNonOnly=False)[0]
    Nodes = __setupHelpMakeGraphsCreate(g=g,undirected=True,dirThres=0.65,colored=False,onlyConnected=True,colorsLst=[],
                                        colorDictIn={},colorImmuneDict={},blackList=False,blackListFile='')[0]
    return Nodes
def makeBNScatterPlots(placeL,date,pathways=True,spotCheck=False,thres=0.3):
    kobeDict=getKobeClique(True)
    if pathways:
        globDir='BN Learn/Outputs/*Kobe Genes*'
    else:
        globDir='BN Learn/Outputs/*'
    imLst = ['T-Eff','IFNg','Antigen Process']
    for place in tq(placeL,desc='Plotting'):
        nodes = __getBNnodes(pathways,place,thres=thres)
        if pathways:
            if not os.path.exists('Saved PDFs/'+place+', Correlation Scatter Plots, '+date):
                os.mkdir('Saved PDFs/'+place+', Correlation Scatter Plots, '+date)
            lst = glob.glob('Run BNLearn/Inputs/*'+place+"*")
            lst = __removeLstFunc(lst,'Kobe Genes')
            typLst=[]
            for i in lst:
                typ = i.split(',')[-2].strip()
                typLst.append(typ)
            typLst=list(set(typLst))
        else:
            lst = glob.glob('Run BNLearn/Inputs/*Kobe Genes*'+place+"*")
            typLst = ['Kobe Genes']
        for typ in typLst:
            lst2 = [p for p in lst if typ in p]
            dfLst=[]
            cols=[]
            frst=True
            for l in lst2:
                lname = l.split(',')[-1].strip()
                df = pd.read_csv(l,sep='\t',index_col=0)
                if frst:
                    cols = df.columns
                    frst=False
                else:
                    cols = __goodGenes(cols,df.columns)
                dfLst.append((lname,df))
            saveLst=[]
            num =len(cols)
            for i in tq(range(num),desc=''):
                for j in range(i,num):
                    col1 = cols[i]
                    col2 = cols[j]
                    coL1 = col1.replace(' Scores','').replace('A-A','AA')
                    coL2 = col2.replace(' Scores','').replace('A-A','AA')
                    if col2 != col1 and coL1 in nodes and coL2 in nodes:
                        frstDF=True
                        for dfTup in dfLst:
                            lname = dfTup[0]
                            df = dfTup[1]
                            spear = df.corr(method='spearman').loc[col1][col2]
                            pear = df.corr(method='pearson').loc[col1][col2]
                            if lname != 'PrePure':
                                if frstDF:
                                    corr = (spear+pear)/2
                                    n=1
                                    frstDF=False
                                else:
                                    corr += ((spear+pear)/2)
                                    n+=1
                            plt.scatter(df[col1],df[col2],label=lname)
                        corr /= n
                        corr = round(corr,2)
                        if not pathways:
                            path1 = kobeDict[coL1]
                            path2 = kobeDict[coL2]
                        val=True
                        if spotCheck:
                            val = __BNSpotCheck(path1=path1,path2=path2,corr=corr,
                                                kobeDict=kobeDict)
                        if val:
                            plt.xlabel(col1,fontsize=5)
                            plt.ylabel(col2,fontsize=5)
                            if pathways:
                                title = 'Database is '+typ+', Correlation between '+col1+' and '+col2+', '
                                title += place+', Correlation is '+str(corr)
                            else:
                                title = 'Database is '+typ+', Correlation between '+col1+'('+path1.replace('/','-')+') and '
                                title += col2+'('+path2.replace('/','-')+'), '+place+', Correlation is '+str(corr)
                            plt.title(title,fontsize=6)
                            plt.legend(fontsize=6)
                            plt.savefig(directory+title+'.pdf')
                            saveLst.append(directory+title+'.pdf')
                        plt.close()
            if pathways:
                __mergePDFs(saveLst,
                            'Saved PDFs/'+place+', Correlation Scatter Plots, '+date+'/Database is '+
                            typ+', Correlation Scatter Plots',erase=True)
            else:
                __mergePDFs(saveLst,
                            'Saved PDFs/'+place+', Database is '+
                            typ+', Correlation Scatter Plots',erase=True)
    return 1

def __BNSpotCheck(path1,path2,corr,kobeDict):
    imLst = ['T-Eff','IFNg','Antigen Process']
    val = True
    if path1 in imLst and path2 in imLst and (corr>-0.2):
        val=False
    else:
        if path1 not in imLst and path2 not in imLst and (corr>-0.2):
            val=False
    if path1 in imLst and path2 not in imLst and (corr<0.2):
        val=False
    if path1 not in imLst and path2 in imLst and (corr<0.2):
        val=False
    if path1=='Heat' or path2=='Heat' or path1=='Purity' or path2=='Purity':
        val=False
    return val

def countNotNaNs(df,rev=False):
    out=pd.Series()
    for row in df.index:
        ser = df.loc[row]
        num = np.count_nonzero(np.isnan(ser))
        if rev:
            num = len(ser) - num
        out[row]=num
    return out

def __getBothSupers(superDfNesIn,superDfFdrIn):
    superDfNes=superDfNesIn.copy();superDfFdr=superDfFdrIn.copy()
    superDfNes.index.name='Pathways';superDfFdr.index.name='Pathways'
    both = superDfNes.merge(superDfFdr,how='outer',on='Pathways')
    colLst=[]
    for col in both.columns:
        if not(col=='MSigDB Category_x' or col=='MSigDB Category_y'):
            colN = col.replace('_x','_NES').replace('_y','_FDR')
        else:
            colN = col.replace('_x','')
        colLst.append(colN)
    both.columns = colLst
    both = both.drop(columns=['MSigDB Category_y'])

    sLst=[]
    for col in both.columns:
        if not col=='MSigDB Category':
            colN = col.replace('_NES','').replace('_FDR','')
            sLst.append(colN)
    sLst2=[]
    for elem in sLst:
        if not elem in sLst2:
            sLst2.append(elem)
    sLst2.reverse()
    finLst=[]
    for elem in sLst2:
        finLst.append(elem+'_FDR')
        finLst.append(elem+'_NES')
    finLst.append('MSigDB Category')
    newCols = orderShiftLst(both.columns,finLst)
    both = both[newCols]
    return both

def __getUpinCold(superDfNes,superDfFdr,coldNum,fdrNum,fdrVal):
    coldUps = superDfNes[superDfNes<0];numColds = countNotNaNs(coldUps,rev=True);numColds2 = numColds[numColds >= coldNum]
    superDfNes2 = superDfNes.loc[numColds2.index];superDfFdr2 = superDfFdr.loc[numColds2.index]
    keepLst=[]
    for row in superDfNes2.index:
        count = 0
        for col in superDfNes2.columns:
            ser = superDfNes2.loc[row][col]
            if ser < 0:
                serF = superDfFdr2.loc[row][col]
                if serF < fdrVal:
                    count +=1
                    if count >= fdrNum:
                        break
        if count >=fdrNum:
            keepLst.append(row)
    superDfNes2 = superDfNes2.loc[keepLst]
    superDfFdr2 = superDfFdr2.loc[keepLst]
    return superDfNes2,superDfFdr2

def getGSEAxCellFile(ver,dateIn,placeLst,coldNum,dbLstIn='All',fdrVal=0.25,dbName='All',direct='',removeBad=False):
    date = dateIn+', v'+str(ver)
    if dbLstIn == 'All':
        dbLst=[]
        for gsea in glob.glob('GSEA Output/'+date+'/*'):
            dbLst.append(gsea.split('\\')[-1])
    else:
        dbLst = dbLstIn.copy()
    superDfNes,superDfFdr = __superDfGSEA(date,dbLst=dbLst,placeLst=placeLst,
                                          desktop=False,skipImmune=False,
                                          preRank=True,coldNum=coldNum,fdrVal=fdrVal)
    superDfNes,superDfFdr,pathLst,catLst = __getPathLst(superDfNes,superDfFdr,removeBad=False)
    superDfNes.insert(0,'MSigDB Category',catLst)
    superDfFdr.insert(0,'MSigDB Category',catLst)
    superDfNes.index.name='Pathways';superDfFdr.index.name='Pathways'
    if dbName=='c2.all':
        dbName = dbName+' '+msigNameDict[dbName]
        df1,df2 = __C2SplitGSEA(superDfNes,superDfFdr,useLst=True,lst=catLst)
        superDfNes1,superDfFdr1,catLst1 = df1
        superDfNes2,superDfFdr2,catLst2 = df2
        
        directory = dbName+' Canonical'+direct
        if not os.path.exists(directory):
            os.makedirs(directory)
        directory+='/'
        superDfNes1.to_excel(directory+dbName+' Canonical, GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+
                             ' Up in Cold'+', Results, NES.xlsx')
        superDfFdr1.to_excel(directory+dbName+' Canonical, GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+
                             ' Up in Cold'+', Results, FDR.xlsx')
        both = __getBothSupers(superDfNes1,superDfFdr1)
        both.to_excel(directory+dbName+' Canonical, GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+
                      ' Up in Cold, Results, Both NES and FDR.xlsx')
        
        directory = dbName+' Chemical and Genetic Perturbations'+direct
        if not os.path.exists(directory):
            os.makedirs(directory)
        directory+='/'
        superDfNes2.to_excel(directory+dbName+' Chemical and Genetic Perturbations, GSEA Agnostic v'+str(ver)+
                             ', At least '+str(coldNum)+' Up in Cold'+', Results, NES.xlsx')
        superDfFdr2.to_excel(directory+dbName+' Chemical and Genetic Perturbations, GSEA Agnostic v'+str(ver)+
                             ', At least '+str(coldNum)+' Up in Cold'+', Results, FDR.xlsx')
        both = __getBothSupers(superDfNes2,superDfFdr2)
        both.to_excel(directory+dbName+' Chemical and Genetic Perturbations, GSEA Agnostic v'+str(ver)+
                      ', At least '+str(coldNum)+' Up in Cold, Results, Both NES and FDR.xlsx')
        
    else:
        if dbName=='c5.all':
            df1,df4 = __C2SplitGSEA(superDfNes,superDfFdr,useLst=True,lst=catLst,keep=['GO'])
            superDfNes1,superDfFdr1,catLst1 = df1
            dbName = dbName+' '+msigNameDict[dbName]
            directory = dbName+' GO Biological Process'+direct
            if not os.path.exists(directory):
                os.makedirs(directory)
            directory+='/'
            superDfNes1.to_excel(directory+dbName+' GO Biological Process, GSEA Agnostic v'+str(ver)+
                                 ', At least '+str(coldNum)+
                                 ' Up in Cold'+', Results, NES.xlsx')
            superDfFdr1.to_excel(directory+dbName+' GO Biological Process, GSEA Agnostic v'+str(ver)+
                                 ', At least '+str(coldNum)+
                                 ' Up in Cold'+', Results, FDR.xlsx')
            both = __getBothSupers(superDfNes1,superDfFdr1)
            both.to_excel(directory+dbName+' GO Biological Process, GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+
                          ' Up in Cold, Results, Both NES and FDR.xlsx')
        else:
            dbName = dbName+' '+msigNameDict[dbName]
            directory = dbName
            directory = dbName+direct
            if not os.path.exists(directory):
                os.makedirs(directory)
            directory+='/'

            superDfNes.to_excel(directory+dbName+', GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+' Up in Cold'+
                                ', Results, NES.xlsx')
            superDfFdr.to_excel(directory+dbName+', GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+' Up in Cold'+
                                ', Results, FDR.xlsx')
            both = __getBothSupers(superDfNes,superDfFdr)
            both.to_excel(directory+dbName+', GSEA Agnostic v'+str(ver)+', At least '+str(coldNum)+
                          ' Up in Cold, Results, Both NES and FDR.xlsx')
    return 1

def getMSigCategories(v,coldNum,placeLst,fdrVal=0.1,containsColdCluster=(False,False,0),
                      desktop=False,skipImmune=False,preRank=True):
    date='2020-11-13, v'+str(v)
    dbLst=[]
    for gsea in glob.glob('GSEA Output/'+date+'/*'):
        dbLst.append(gsea.split('\\')[-1])

    superDfNes,superDfFdr = __superDfGSEA(date,dbLst=dbLst,placeLst=placeLst,
                                          desktop=desktop,skipImmune=skipImmune,
                                          preRank=preRank,coldNum=coldNum,fdrVal=fdrVal,
                                          containsColdCluster=containsColdCluster)
    fullDict = getFullDict()
    fullLst=[]
    for name,dic in fullDict:
        for path in dic:
            fullLst.extend(dic[path])
    fullLst=list(set(fullLst))
    nDict={}
    for name, dic in fullDict:
        lst2=[]
        for row in superDfNes.index:
            group = row.split('_')[0].replace('MSigDB','')
            if group == name:
                lst2.append(row)
        df2 = superDfNes.loc[lst2]
        df2,df3,pathLst,catLst = __getPathLst(df2,df2,removeBad=False)
        pathways = df2.index
        lst=[]
        for path in dic:
            if path in pathways:
                lst.extend(dic[path])
        lst = list(set(lst))
        for i in range(len(fullLst)-len(lst)):
            lst.append('')
        nDict[name]=lst
    df = pd.DataFrame.from_dict(nDict)
    df.index = fullLst
    title = 'Saved Images/V'+str(v)+', fdr '+str(fdrVal)+' Up in '+str(coldNum)
    title+= ' Cancers'+title+', GSEA Agnostic, Msig Gene Lists.tsv'
    if containsColdCluster[0]:
        title = 'Saved Images/V'+str(v)+', fdr '+str(fdrVal)+', Up in at least '
        title += str(containsColdCluster[2])
        if containsColdCluster[1]:
            title += ' Other-Cluster Cancer Areas'
        else:
            title += ' Cold-Cluster Cancer Areas'
        title += ', GSEA Agnostic, Msig Gene Lists.tsv'
    df.to_csv(title,sep='\t')
    gc.collect()
    return 1
def all0s(ser):
    out=True
    for elem in ser:
        elem = float(elem)
        if elem != 0.0:
            out=False
            break
    return out

In [14]:
def clusterTTest(ver,pvalCutoff):
    superDfNes = pd.read_csv('SuperDfNes_c2_c5_hall_v'+str(ver)+'.tsv',sep='\t',index_col=0)
    p4=['Adrenocortical','Breast HR+','K_Lung adenocarcinoma','K_Lung squamous','Thyroid','K_Skin'];p5=[]
    for place in superDfNes.columns:
        if not place in p4:
            p5.append(place)
    p6 = p4+p5
    coldClus=superDfNes[p4]
    othrClus=superDfNes[p5]
    bothClus=superDfNes[p6]

    dfIn = coldClus
    outDf = __helpClusterTTest(dfIn,pvalCutoff)
    outDf.to_excel('C1 coldCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx')

    dfIn = othrClus
    outDf = __helpClusterTTest(dfIn,pvalCutoff)
    outDf.to_excel('C2 otherCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx')

    dfIn = bothClus
    outDf = __helpClusterTTest(dfIn,pvalCutoff)
    outDf.to_excel('C Union bothCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx')
    return 1

def __helpClusterTTest(dfIn,pvalCutoff):
    df=dfIn.copy()
    keepLst=[]
    for path in df.index:
        ser = df.loc[path]
        if not all0s(ser):
            keepLst.append(path)
    df = df.loc[keepLst]
    outDict={}
    for path in df.index:
        ser = df.loc[path]
        stat = stats.ttest_1samp(ser,0)
        outDict[path]=stat
    dfOut = pd.DataFrame.from_dict(outDict)
    dfOut.index = ['TStat','Pval']
    dfOut=dfOut.T
    dfOut=dfOut[dfOut['TStat'] < 0]
    dfOut=dfOut[dfOut['Pval'] < pvalCutoff]
    return dfOut
def __helpCondensePaths(inDict,pathLst,path1,cutoff,condenseLst,inplace=True,doneLst=[]):
    path1Genes = inDict[path1]
    if inplace:
        condenseLst2 = condenseLst
    else:
        condenseLst2 = condenseLst.copy()
    for j in range(len(pathLst)):
        path2 = pathLst[j]
        if path2 == path1:
            condenseLst2.append(path2)
        else:
            if(not path2 in doneLst):
                path2Genes = inDict[path2]
                if percentInLst(path1Genes,path2Genes) >= cutoff or percentInLst(path2Genes,path1Genes) >= cutoff:
                    condenseLst2.append(path2)
    return condenseLst2
def maxLenDict(dic):
    maxLen=0
    for i in dic:
        num = len(dic[i])
        if num > maxLen:
            maxLen = num
    return maxLen
def percentInLst(l1,l2):
    n = len(list(set(l1).intersection(l2)))
    percent = n/len(l1)
    return percent

def __testCondense(inDict,cutoffLst,testPath,fullDict,doneLst=[]):
    testDict={}
    for cutoff in cutoffLst:
        outDict={}
        pathLst = list(inDict.keys())
        if testPath:
            path1 = testPath
        condenseLst=[]
        __helpCondensePaths(inDict=fullDict,pathLst=pathLst,path1=path1,cutoff=cutoff,
                            condenseLst=condenseLst,doneLst=doneLst)
        if len(condenseLst)>1:
            outDict[path1] = condenseLst
        if testPath in outDict.keys():
            testLst = outDict[testPath]
            testDict['Overlap Percentage '+str(cutoff)]=testLst
        else:
            testDict['Overlap Percentage '+str(cutoff)] = [path1]
    return testDict
def __helpTestCondense(testDict):
    maxLen = maxLenDict(testDict)
    testDict2={}
    for i in testDict:
        lst = testDict[i].copy()
        length = len(lst)
        for j in range(maxLen-len(lst)):
            lst.append('')
        testDict2[i+', # of Pathways is '+str(length)]=lst
    df = pd.DataFrame.from_dict(testDict2)
    return df
def getColNum(col):
    splt = col.split()
    num=0
    for i in splt:
        i=i.replace(',','')
        try:
            num=float(i)
            break
        except ValueError:
            pass
    return num
def condensePathways(cutoffLst,useLst,testPath,useName):
    fullDict = getFullDict(outputLst=False,useLst=useLst)
    inDict = fullDict.copy()
    testPathName = testPath
    testDict = __testCondense(inDict=inDict,cutoffLst=cutoffLst,testPath=testPath)
    df = __helpTestCondense(testDict)
    name = testPathName+', '+useName+', Condensing Pathways Test, ' 
    name+=' condensed group at different overlap percentages.xlsx'
    df.to_excel(name)
    return 1
metabolicLst = ('Metabloic',['METABOLISM','METABOLIC','CATABOLISM','CATABOLIC','GLUCOSE','GLUCONEOGENESIS','GLYCOLYSIS'])
lipidLst = ('Lipid',['LIPID','FATTY_ACID','CHOLESTEROL'])
prolifLst = ('Prolifiration',['ELONGATION','INITIATION','TRANSLATION','TRANSCRIPTION'])
mitochanLst = ('Mitochandrial',['ELECTRON_TRANSPORT','MITOCH'])
testLstLists = [metabolicLst,lipidLst,prolifLst,mitochanLst]

def plotUpinClusterClumps(dateIn,coldNum,placeLst,metric='cosine',save=False,numParts=30,reverseOrder=False,noParts=False,
                          fixPos=False,Position={},fontSize=12,rot=45,figSize=(60,40),containsColdCluster=(True,False,4),
                          cutoff=0.2,plot=False,excel=False,rowColors=None,replace=False):
    ver = dateIn.split('v')[1]
    gc.collect()
    if containsColdCluster[1]:
        pathName = 'Up in at least '+str(coldNum)+' Other-Cluster Cancer Areas, V'+ver
    else:
        pathName = 'Up in at least '+str(coldNum)+' Cold-Cluster Cancer Areas, V'+ver
    lstDict={}
    for file in glob.glob('Lists/GSEA Pathways/*'):
        lst = readLst(file)
        splt = file.split(',')
        fName =','.join([splt[1],splt[2]]).replace(' U','U')
        lstDict[fName]=lst
    pathways = lstDict[pathName]
    plotClumps(pathways=pathways,dateIn=dateIn,coldNum=coldNum,metric=metric,save=save,numParts=numParts,
               reverseOrder=reverseOrder,noParts=noParts,fixPos=fixPos,placeLst=placeLst,Position=Position,fontSize=fontSize,
               rot=rot,figSize=figSize,containsColdCluster=containsColdCluster,cutoff=cutoff,rowColors=rowColors,
               plot=plot,excel=excel,pathName=pathName,replace=replace)
    return 1
def plotClumps(pathways,dateIn,coldNum,placeLst,metric='cosine',save=False,numParts=30,reverseOrder=False,noParts=False,
               fixPos=False,colors=[],cGroups=(False,[]),replace=False,testLstLists=testLstLists,
               Position={},fontSize=12,rot=45,figSize=(60,40),containsColdCluster=(True,False,4),cutoff=0.2,
               plot=False,excel=False,pathName='',geneSpecific=False,excelGeneSpecific=False):
    useLst = ['c2.cp','c5.go.bp','h.all']
    fullDict = getFullDict(outputLst=False,useLst=useLst)
    inDict = fullDict.copy()
    hallLst=[]
    smallerDict={}
    for path in pathways:
        if path in fullDict.keys():
            smallerDict[path] = fullDict[path]
    inDict = smallerDict.copy()

    for i in fullDict.keys():
        if 'HALLMARK' in i:
            hallLst.append(i)
    clumps={}
    doneLst=[]
    for testPath in hallLst:
        testPathName = testPath
        testDict = __testCondense(inDict=inDict,cutoffLst=[cutoff],testPath=testPath,doneLst=doneLst,fullDict=fullDict)
        lst = testDict['Overlap Percentage '+str(cutoff)]
        if len(lst)>1:
            clumps[testPathName] = lst
        doneLst.extend(lst)
    for name,test_list in testLstLists:
        lst=[]
        for path in pathways:
            res = any(elem for elem in test_list if(elem in path))
            if res:
                lst.append(path)
        clumps[name]=lst
    notHallLst=[]
    for pathway in inDict:
        if not pathway in doneLst:
            notHallLst.append(pathway)
    clumps['Uncategorized'] = notHallLst
    dbLst=[]
    for gsea in glob.glob('GSEA Output/'+dateIn+'/*'):
        dbLst.append(gsea.split('\\')[-1])
    df1,df2=__superDfGSEA(dateIn,dbLst,placeLst,desktop=False,skipImmune=False,preRank=True,coldNum=coldNum,
                          fdrVal=0.1,containsColdCluster=containsColdCluster)    
    df1,df2,catLst = __getPathLst(df1,df2,removeBad=False)
    ls = df1.index.copy()
    for hallClump in tq(clumps,desc='Working on Clumps'):
        gc.collect()
        lst = clumps[hallClump]
        pathways = __goodGenes(ls,lst)
        if len(pathways)>1:
            df3 = df1.loc[pathways]
            df4 = df2.loc[pathways]
            title=pathName;db='Gene Sets Overlapping with '+hallClump+', '
            if geneSpecific or excelGeneSpecific:
                clumpCountDict = countSpecificUpinColdGenes(ver=ver,inDict=inDict,placeLst=placeLst,
                                                            pvalCutoff=pvalCutoffGenes,pathways=pathways)
            if plot:
                title2 = 'Saved Images/Gene Sets Overlapping with '+hallClump+', '+title+', '+metric+'.pdf'
                title2=title2.replace(', ,',',')
                if replace or not os.path.exists(title2):
                    suptitleName = hallClump.replace('HALLMARK_','').replace('_',' ')
                    if hallClump != 'Uncategorized':
                        suptitleName += ' Associated' 
                    suptitleName+= ' Gene Lists'
                    if cGroups[0]:
                        c1 = cGroups[1][0];c2 = cGroups[1][1];c3 = cGroups[1][2]
                        rowColors = getRowColorsCs(pathways=list(df3.index),c1=c1,c2=c2,c3=c3,colors=colors)
                        names=zip(['C All','C2','C1'],colors)
                        rowColors=(rowColors,names)
                    else:
                        rowColors=(None,'')
                    row,col = __helpPlotGSEA(df3,df4,title=title,db=db,metric=metric,suptitleName=suptitleName,
                                             rowColors=rowColors,
                                             save=save,numParts=numParts,reverseOrder=reverseOrder,noParts=noParts,
                                             catLst=[],figSize=figSize,fixPos=fixPos,pos=Position,fontSize=fontSize,rot=rot)
            if excel:
                title2 = 'Excel/Gene Sets Overlapping with '+hallClump+', '+title+', '+metric
                df3.to_excel(title2+', NES.xlsx');df4.to_excel(title2+', FDR.xlsx')
                both = __getBothSupers(df3,df4);both.to_excel(title2+', Both NES and FDR.xlsx')
            if excelGeneSpecific:
                title2 = 'Excel/Gene Sets Overlapping with '+hallClump+', Counts of Specific Genes up in Cold' 
                title2+=', '+title+', '+metric
                countDf= pd.DataFrame.from_dict(clumpCountDict);order = list([countDf.columns[-1]])+list(countDf.columns[:-1])
                countDf = countDf[order];countDf.to_excel(title2+'.xlsx')
    return 1
def __helpMergePDFs(directory,name,moveLast=False,erase=False,move2=False):
    if os.path.exists(name+'.pdf'):
        os.remove(name+'.pdf')
    images = glob.glob(directory)
    if moveLast:
        if move2:
            half = int(len(images)/2)
            l2 = images[:half]
            l3 = images[half:]
            l2 = l2[1:]+[l2[0]]
            l3 = l3[1:]+[l3[0]]
            images = l2+l3
        else:
            images = [images[-1]]+images[:-1]
    __mergePDFs(images,name,erase=erase)
    return 1

def getRowColorsCs(pathways,c1,c2,c3,colors):
    rowColors=[]
    for cPath,color in [(c1,colors[0]),(c2,colors[1]),(c3,colors[2])]:
        rowLst=[]
        for path in pathways:
            if path in cPath:
                rowLst.append(color)
            else:
                rowLst.append('White')
        rowColors.append(rowLst)
    return rowColors
def countSpecificUpinColdGenes(ver,placeLstIn,inDict,pvalCutoff,pathways,acceptLst=['C1','C2','C All'],
                               count=True,useGenes=False,genesIn=[],tqOn=False,excludeGenes=[]):
    superDfNes = pd.read_csv('SuperDfNes_c2_c5_hall_v'+str(ver)+'.tsv',sep='\t',index_col=0)
    p4=['Adrenocortical','Breast HR+','K_Lung adenocarcinoma','K_Lung squamous','Thyroid','K_Skin'];p5=[]
    for place in superDfNes.columns:
        if not place in p4:p5.append(place)
    p6 = p4+p5;shortNamePlaceLst = list(superDfNes.columns);shortNamePlaceLst = shortNamePlaceLst[1:]+[shortNamePlaceLst[0]]
    CareasLst = [(p4,'C1'),(p5,'C2'),(p6,'C All')];dfLst=[];frst=True
    for place in placeLstIn:
        df = pd.read_csv('Purity/TPM/v'+str(ver)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        if frst:allGenes = list(df.index);frst=False
        else:allGenes.extend(list(df.index))
        if len(df.index)>1:Hot,Cold = __getHotCold(df);dfLst.append((place,Hot,Cold))
    allGenes = list(set(allGenes));clumpCountDict={}
    if count:
        totalCountDict={}
        for areas,name in CareasLst:
            if name in acceptLst:totalCountDict[name]=0
        totalCountDict['Total # Genes in Clump']=0
    for path in pathways:
        gc.collect()
        if path in inDict.keys() or useGenes:
            if useGenes:
                if genesIn =='All':genes = allGenes.copy()
                else:genes = genesIn.copy()
            else:genes = inDict[path]
            dfLstPath=[];pathwayDict={};inputs=genes
            if tqOn:inputs=tq(genes)
            for gene in inputs:
                geneLst=[]
                for place,Hot,Cold in dfLst:
                    if gene in Hot.index and gene in Cold.index:
                        a = Hot.loc[gene];b = Cold.loc[gene];tStat,pval = stats.ttest_ind(a,b);geneLst.append(tStat)
                    else:geneLst.append(0)
                pathwayDict[gene] = geneLst
            pathwayDf = pd.DataFrame.from_dict(pathwayDict, orient='index',columns=shortNamePlaceLst);pathwayCountDict={}
            for areas,name in CareasLst:
                if name in acceptLst:
                    if count:countN=0
                    else:countN=[]
                    pathwayDfC = pathwayDf[areas]
                    for gene in pathwayDfC.index:
                        ser = pathwayDfC.loc[gene];stat,pval = stats.ttest_1samp(ser,0)
                        if stat < 0 and pval <= pvalCutoff:
                            if count:countN +=1;totalCountDict[name] += 1
                            else:countN.append(gene)
                    pathwayCountDict[name] = countN
            if count:
                pathwayCountDict['Total # Genes In Pathway']=len(genes);totalCountDict['Total # Genes in Clump']+=len(genes)
            clumpCountDict[path] = pathwayCountDict
    if count:clumpCountDict['Total Count'] = totalCountDict
    return clumpCountDict
def plotCGroupClumps(ver,placeLst,pvalCutoff=0.05,coldNum=1,metric='cosine',save=False,numParts=30,noParts=False,
                     reverseOrder=False,replace=False,merge=False,pvalCutoffGenes=0.05,
                     fixPos=False,Position={},fontSize=12,rot=45,figSize=(60,40),containsColdCluster=(False,False,0),
                     clumpCutoff=0.2,plot=False,excel=False,colors=['Orange','Purple','Green'],
                     geneSpecific=False,excelGeneSpecific=False):
    clusterTTest(ver,pvalCutoff)
    c1 = pd.read_excel('C1 coldCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx',
                       sep='\t',index_col=0);c1Paths = list(c1.index)
    c2 = pd.read_excel('C2 otherCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx',
                       sep='\t',index_col=0);c2Paths = list(c2.index)
    c3 = pd.read_excel('C Union bothCluster cancers, 1 sample ttest up in cold, v'+str(ver)+', pval '+str(pvalCutoff)+'.xlsx',
                       sep='\t',index_col=0);c3Paths = list(c3.index)
    pathways = list(set(c1Paths+c2Paths+c3Paths))
    pathName = 'Up in C Groups per 1 sample Ttest, v'+str(ver)+', pval '+str(pvalCutoff);dateIn = '2020-11-13, v'+str(ver)
    plotClumps(pathways,dateIn=dateIn,coldNum=coldNum,metric=metric,save=save,numParts=numParts,replace=replace,
               reverseOrder=reverseOrder,noParts=noParts,fixPos=fixPos,Position=Position,fontSize=fontSize,rot=rot,
               figSize=figSize,containsColdCluster=containsColdCluster,cutoff=clumpCutoff,plot=plot,excel=excel,
               placeLst=placeLst,pathName=pathName,colors=colors,cGroups=(True,[c3Paths,c2Paths,c1Paths]),
               geneSpecific=geneSpecific,excelGeneSpecific=excelGeneSpecific)
    if merge:
        files= glob.glob('Saved Images/*Gene Sets*v'+str(ver)+'*')
        name='Saved Images\\Gene Sets Overlapping, Up in C Groups per 1 sample Ttest, v'+str(ver)
        name+= ', pval '+str(pvalCutoff)+', '+metric;__mergePDFs(files,name,erase=True)
    return 1
def getListofGenes(ver,pvalLst,genesIn,title):
    outDict={}
    for pval in tq(pvalLst,desc='Getting Genes'):
        output = countSpecificUpinColdGenes(ver=ver,placeLstIn=placeLst3,inDict={},pvalCutoff=pval,pathways=['All Genes'],
                                            acceptLst=['C1'],tqOn=False,
                                            count=False,useGenes=True,genesIn=genesIn) 
        countDf= pd.DataFrame.from_dict(output);genes = countDf['All Genes']['C1'];outDict[pval] = genes
    maxLen = len(outDict[0.05])
    for pv in outDict:
        lst = outDict[pv]
        for i in range(maxLen-len(lst)):lst.append('')
        lst.insert(0,str(len(lst)));outDict[pv]=lst
    df = pd.DataFrame.from_dict(outDict);df.to_excel(title+'.xlsx')
    return 1
def __helpGetBLDict(dfIn,col,blDict={},blDict2={},usePathwayName=True,useGenes=False,genesIn=[]):
    df = dfIn.copy();useLst = ['c2.cp','h.all'];fullDict = getFullDict(outputLst=False,useLst=useLst)
    for cat in list(df[col].unique()):
        df3 = df[df[col]==cat];cat = cat.strip();lst=[]
        if usePathwayName:
            for ind in df3.index:
                if ind in fullDict.keys():pathLst = fullDict[ind];lst.extend(pathLst)
        else:lst =  list(df3.index)
        for elem in lst:blDict2[elem]=cat
        lst=list(set(lst));blDict[cat] = lst.copy()
    if useGenes:
        blDict2_b={};blDict_b={}
        for path in blDict:blDict_b[path]=[]
        for gene in genesIn:
            if gene in blDict2:path = blDict2[gene];blDict_b[path].append(gene);blDict2_b[gene]=path
        blDict = blDict_b.copy();blDict2 = blDict2_b.copy()
    return blDict,blDict2

In [15]:
def getInteractTypeOrder(interactType):
    lst=interactType.split('=')
    for i in lst:
        if '+' in i:
            return (True,False)
        if '-' in i:
            return (False,True)
def BNLearnHMs(thres,globDir,titleIn,genesIn,dictIn,immuneDict,metric='cosine',size=(30,20),excludePre=True,pathways=False,
               onlyBGE=False,onlyBDE=False,mergedAll=True,indivCancers=False,interPath=False,immuneNonOnly=False,
               nonToIm=False,dirThres=0.65,cancersNum=1,excel=False,ylabels=0):
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=onlyBGE,onlyBDE=onlyBDE);placeLs = []
    if(excludePre):graphLst = __removeLstFunc(graphLst,'PrePure')
    for graph in graphLst:
        place = graph.split('\\')[1].split(',')[0].strip();placeLs.append(place)
    placeLs = list(set(placeLs))
    if mergedAll:
        graphLst = __removeLstFunc(graphLst,'Pre-BN')
        __BNLearnHMsHelp(graphLst,genesIn=genesIn,immuneDict=immuneDict,dropImmune=False,excel=excel,metric=metric,
                         ylabels=ylabels,
                         dropPure=False,thres=thres,dirThres=dirThres,pathways=pathways,interPath=interPath,size=size,
                         immuneNonOnly=immuneNonOnly,nonToIm=nonToIm,dictIn=dictIn,cancersNum=cancersNum,titleIn=titleIn)
    if indivCancers:
        for place in tq(placeLs,desc='Making BN Graphs'):
            graphLst2 = [p for p in graphLst if place in p]
            __BNLearnHMsHelp(graphLst2,genesIn=genesIn,immuneDict=immuneDict,dropImmune=False,excel=excel,ylabels=ylabels,
                             dropPure=False,thres=thres,dirThres=dirThres,pathways=pathways,interPath=interPath,size=size,
                             immuneNonOnly=immuneNonOnly,nonToIm=nonToIm,dictIn=dictIn,cancersNum=cancersNum,titleIn=titleIn)
    return 1
def __BNLearnHMsHelp(graphLst,genesIn,immuneDict,dropImmune,dropPure,thres,dirThres,pathways,interPath,immuneNonOnly,
                     nonToIm,dictIn,cancersNum,titleIn,excel,size,metric,ylabels):
    dfDict={};dfDict2={};dfDict3={};dfTypeDict={}
    for graph in graphLst:
        place = graph.split(',')[0].split('\\')[-1].replace('Kobe Genes ','');placeDict={}
        for gene in genesIn:
            for gene2 in genesIn:
                if not gene==gene2:placeDict[(gene,gene2)]=[]
        dfDict[place]=placeDict.copy();dfDict2[place]=placeDict.copy();dfDict3[place]=placeDict.copy()
        dfTypeDict[place]=placeDict.copy()
    inputs=graphLst
    out = Parallel(n_jobs=num_cores)(delayed(__parallelBNLearnHMs)(graph,immuneDict,thres,dictIn,interPath,immuneNonOnly,
                                                                   nonToIm,dirThres,pathways) 
                                     for graph in inputs)
    for dfPlace,place,corr in out:
        for frm,to in dfDict[place]:
            corrVal=np.nan
            if frm in corr.columns and to in corr.columns:
                corrVal=corr.loc[frm][to];df2=dfPlace[dfPlace['from']==frm];df2=df2[df2['to']==to]
                if len(df2.index)>0:
                    inVal=1;inVal2=round(corrVal,4);inValType = df2.loc[df2.index[0]]['Type']
                    if corrVal<0:inVal=-1
                else:
                    inVal=np.nan;inVal2=np.nan;inValType=np.nan
            else:
                inVal=np.nan;inVal2=np.nan;inValType=np.nan
            dfDict[place][(frm,to)]=corrVal;dfDict2[place][(frm,to)]=inVal;dfDict3[place][(frm,to)]=inVal2
            dfTypeDict[place][(frm,to)]=inValType
    df = pd.DataFrame.from_dict(dfDict);df2 = pd.DataFrame.from_dict(dfDict2);df3 = pd.DataFrame.from_dict(dfDict3)
    dfType = pd.DataFrame.from_dict(dfTypeDict)
    df2.dropna(how='all',inplace=True);df=df.loc[df2.index];df2=df2.fillna(0);df3=df3.loc[df2.index];
    dfType=dfType.loc[df2.index];dfType.dropna(axis=1,how='all',inplace=True);idxLst=[]
    for i,j in dfType.index:
        for col in dfType.columns:
            typ=dfType.loc[(i,j)][col] 
            if typ == 'Directed' or typ== 'Undirected':
                if typ== 'Undirected':
                    print(typ)
                idxTyp=dfType.loc[(i,j)][col]
                break
        if idxTyp=='Directed':idx = i+' -> '+j
        else:idx=i+' -- '+j
        idxLst.append(idx)
    del(dfType);del(dfTypeDict);gc.collect();df.index = idxLst;df2.index=idxLst;df3.index=idxLst
    absDf2 = abs(df2);absDf2=absDf2.T;absNum = absDf2.sum() >= cancersNum;idxLst=list(absNum[absNum].index)
    df=df.loc[idxLst];df2=df2.loc[idxLst];df3=df3.loc[idxLst];df=df.dropna();df3=df3.loc[df.index]
    if len(df.index)>1:
        if excel:
            path1Lst=[];path2Lst=[]
            for i in df4.index:
                splt=i.split();frm=splt[0];to=splt[-1];path1=dictIn[frm];path2=dictIn[to]
                path1Lst.append(path1);path2Lst.append(path2)
            df4.insert(0,'To Gene Pathway',path2Lst);df4.insert(0,'From Gene Pathway',path1Lst)
            title = 'Immune-Non Pairings, Confidence '+str(thres)+', edge in at least '
            title += str(cancersNum)+' cancer areas'
            title += ', undirected and directed.xlsx';df4.to_excel(title)
        maxx=df.max().max();minn=df.min().min()
        maxx = max(abs(maxx),abs(minn));minn=maxx*-1
        cg = sb.clustermap(df,cmap=cmapSpecial,vmax=maxx,vmin=minn,xticklabels=1,yticklabels=ylabels,metric=metric,
                           annot=df3,annot_kws={"size":10},figsize=size)
        plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom')
        title=titleIn+', Confidence '+str(thres)+', metirc '+metric+', edge in at least in '
        title+= str(cancersNum)+' cancers, only BGE, v7';cg.fig.suptitle(title,size='x-large')
        for t in cg.ax_heatmap.texts:
            string = t.get_text()
            if string == 'nan':t.set_text('')
        plt.savefig('Graphs/'+title+'.png',bbox_inches = "tight",dpi=300);plt.close()
    return 1

def __parallelBNLearnHMs(graph,immuneDict,thres,dictIn,interPath,immuneNonOnly,nonToIm,dirThres,pathways):
    g = __getMergedG([graph],immuneDict=immuneDict,dropImmune=False,dropPure=False,agnostic=False,
                     thres=thres,nonToIm=nonToIm,dictIn=dictIn,
                     thresFloor=0.1,interPath=interPath,immuneNonOnly=immuneNonOnly,dirThres=dirThres,
                     pathways=pathways,below=False)
    inputsLst=g[2];corr=__getCorrInputsLst(inputsLst)
    setupOut = __setupHelpMakeGraphsCreate(g=g[0],undirected=True,dirThres=dirThres,colored=False,colorsLst=[],
                                           blackList=False,blackListFile='',
                                           colorDictIn={},colorImmuneDict={},onlyConnected=True)
    Nodes,direct,undirect,colorDict,colorPathDict,topLs=setupOut
    dfPlace = pd.concat([direct,undirect]);typeLst=[]
    for i in range(len(direct.index)):typeLst.append('Directed')
    for i in range(len(undirect.index)):typeLst.append('Undirected')
    dfPlace['Type']=typeLst;place = graph.split(',')[0].split('\\')[-1].replace('Kobe Genes ','')
    return (dfPlace,place,corr)
def recursiveHelpPathFinder(i,targetLst,direct,doneLst,corr,allowNegCorr=True,allowPosCorr=True,numNegs=0):
    ser=direct.loc[i]
    if ser['to'] in targetLst:
        correlation = corr.loc[ser['to']][ser['from']]
        if correlation < 0:numNegs+=1
        isOdd=numNegs%2!=0
        if ((allowPosCorr and correlation >= 0) or (allowNegCorr and isOdd)):return [ser['to'],ser['from']], [i]
        else:return False, []
    else:
        to=direct.loc[i]['to'];direct2=direct[direct['from']==to]
        for newI in direct2.index:
            if not newI in doneLst:
                frm= direct.loc[i]['from'];to=direct.loc[i]['to'];correlation = corr.loc[to][frm]
                if correlation < 0:numNegs+=1
                path, lst = recursiveHelpPathFinder(i=newI,targetLst=targetLst,direct=direct,doneLst=doneLst,
                                                    corr=corr,allowNegCorr=allowNegCorr,allowPosCorr=allowPosCorr,
                                                    numNegs=numNegs)
                if path:
                    if not to in path:path.append(to)
                    path.append(frm);lst.append(i)
                    return path,lst
        return False,[]
# Import modules
import http.client
import json
##### Function to run X2K
### Input: a Python list of gene symbols
### Output: a dictionary containing the results of X2K, ChEA, G2N, KEA.

def run_X2K(input_genes, options={},useFile=False,file=''):
    if useFile:
        print('Will do')
    else:
        # Open HTTP connection
        conn = http.client.HTTPConnection("amp.pharm.mssm.edu")

        # Set default options
        default_options = {'text-genes': '\n'.join(input_genes),
                           'included_organisms': 'both',
                           'TF-target gene background database used for enrichment': 'ChEA & ENCODE Consensus',
                           'sort transcription factors by': 'p-value',
                           'min_network_size': 10,
                           'number of top TFs': 10,
                           'path_length': 2,
                           'min_number_of_articles_supporting_interaction': 0,
                           'max_number_of_interactions_per_protein': 200,
                           'max_number_of_interactions_per_article': 100,
                           'enable_BioGRID': True,
                           'enable_IntAct': True,
                           'enable_MINT': True,
                           'enable_ppid': True,
                           'enable_Stelzl': True,
                           'kinase interactions to include': 'kea 2018',
                           'sort kinases by': 'p-value'}

        # Update options
        for key, value in options.items():
            if key in default_options.keys() and key != 'text-genes':
                default_options.update({key: value})
        # Get payload
        boundary = "----WebKitFormBoundary7MA4YWxkTrZu0gW"
        payload = ''.join(
            ['--' + boundary + '\r\nContent-Disposition: form-data; name=\"{key}\"\r\n\r\n{value}\r\n'.format(**locals())
             for key, value in default_options.items()]) + '--' + boundary + '--'
        # Get Headers
        headers = {
            'content-type': "multipart/form-data; boundary=" + boundary,
            'cache-control': "no-cache",
        }
        # Initialize connection
        # Get response
        try:
            conn.request("POST", "/X2K/api", payload, headers)
            res=conn.getresponse()
            # Read response
            data = res.read().decode('utf-8')
            # Convert to dictionary
            x2k_results = {key: json.loads(value) if key != 'input' else value for key, value in json.loads(data).items()}
            # Clean results
            x2k_results['ChEA'] = x2k_results['ChEA']['tfs'];x2k_results['G2N'] = x2k_results['G2N']['network']
            x2k_results['KEA'] = x2k_results['KEA']['kinases'];x2k_results['X2K'] = x2k_results['X2K']['network']
            return x2k_results
        except:
            return run_X2K(input_genes, options,useFile,file)

def parallelGraphDict(gene,graphDict,dfPlace,meanDict,thres):
    ser=dfPlace[gene]
    out = Parallel(n_jobs=7)(delayed(__parallelGraphDictHelper)(gene=gene,n=n,ser=ser,meanDict=meanDict,thres=thres,
                                                                graphDict=graphDict,outDict={}) for n in graphDict)
    outDict={}
    for d in out:
        outDict.update(d)
    return (gene,outDict)
def __parallelGraphDictHelper(gene,n,ser,meanDict,thres,graphDict,outDict):
    ls = graphDict[n]
    if gene in ls:
        outDict[n]=True
    else:
        mean = meanDict[n]
        corP=ser.corr(mean,method='pearson');corS=ser.corr(mean,method='spearman');corr=(corP+corS)/2
        if corr >= thres:
            outDict[n]=True
        else:
            outDict[n]=False
    return outDict
def __helpMergeDfsCliques(df,dfPlace,title,frst=True,colLst=[],dfOut=[[],[]]):
    colLst2=colLst.copy()
    for col in df.columns:
        clique = list(df[col].dropna());df2=dfPlace[clique].T;df2Mean=df2.mean()
        df2Mean.index.name='Samples';df2Mean.name='Name'
        if frst:
            dfOut = df2Mean.to_frame();dfOut.index.name='Samples';frst=False
        else:
            dfOut=dfOut.merge(df2Mean,on='Samples')
        colLst2.append(title+' '+str(col))
    return (dfOut,colLst2)
def __getNonToImmPath(depthOfPaths,immuneDict,g,dictIn,immuneNonOnly,nonToIm,rmvLen):
    useImmDict=immuneDict.copy();outLst=[];useG=g.copy()
    for n in range(depthOfPaths):
        keep=interPathwayOnly(useG,immuneDict=useImmDict,dictIn=dictIn,immuneNonOnly=immuneNonOnly,
                              nonToIm=nonToIm,rmvLen=rmvLen,pathways=True)
        outLst.extend(keep.index);keepLst=list(set(list(keep['from'])))
        for clique in keepLst:useImmDict[clique.split('_')[0]]=['red']
        useGLst=[]
        for i in useG.index:
            if useG.loc[i]['to'] in keepLst:useGLst.append(i)
        useG=g.loc[useGLst]
    outLst=list(set(outLst));gOut = g.loc[outLst]
    return gOut
def removeCommonTFs(immTFIn,nonImmLst,removeTFs=[]):
    immTF=immTFIn.copy();outLst=[];immDoneLst=[];outLst2=[]
    if len(immTF)>1:
        immSet=set(immTF['name'])
        for i in range(len(nonImmLst)):
            nonImmTF = nonImmLst[i];nonImmDoneLst=[]
            if len(nonImmTF)>1:
                nonImmSet=set(nonImmTF['name']);commonTFs= list(immSet.intersection(nonImmSet));commonTFs.extend(removeTFs)
                for tf in commonTFs:
                    immIdxLst=list(immTF[immTF['name']==tf].index);immDoneLst.extend(immIdxLst)
                    immIdxLst=list(nonImmTF[nonImmTF['name']==tf].index);nonImmDoneLst.extend(immIdxLst)
                nonImmTF2=nonImmTF.loc[nonImmDoneLst]#nonImm2 = getTF2(dfTF,removeTFs)
                nonImmTF.drop(nonImmDoneLst,inplace=True)#;nonImmTF=nonImmTF[nonImmTF['type']=='tf']
                outLst.append(nonImmTF);outLst2.append(nonImmTF2)
        immDoneLst=list(set(immDoneLst))
        immTF2=immTF.loc[immDoneLst];immTF.drop(immDoneLst,inplace=True)#;immTF=immTF[immTF['type']=='tf']
        outLst.reverse();outLst.append(immTF);outLst2.reverse();outLst2.append(immTF2)
    return outLst,outLst2
def fixSourceTargetTFs(source,target,tfs=True,dataframe=True):
    if tfs:
        source=source[source['type']=='tf'];target=target[target['type']=='tf']
    if dataframe:
        sourceTFs=set(source['name']);targetTFs=set(target['name']);common=sourceTFs.intersection(targetTFs)
        sourceTFs=list(sourceTFs);sourceTFs = [i.split('_')[0] for i in sourceTFs if not i in targetTFs]
        targetTFs =list(targetTFs);targetTFs = [i.split('_')[0] for i in targetTFs if not i in sourceTFs]
    else:
        sourceTFs=source.copy();targetTFs=target.copy()
    sourceTFs=','.join(sourceTFs);targetTFs=','.join(targetTFs)
    return sourceTFs,targetTFs
def masterOmniPath(globDir='Run BNLearn/Outputs/*Clique*',pathways=False,onlyBGE=False,onlyBDE=False,excludePre=True,
                   mergedAll=False,thres=0.75,interPath=True,immuneNonOnly=True,nonToIm=True,dirThres=0.65,
                   allowPosCorr=False,allowNegCorr=False,useFile=False,file=''):
    directory='TF Anaylsis/Outputs/'
    if allowNegCorr and not allowPosCorr:directory+='Negative Correlation, '
    else:directory+='Any Correlation, '
    if onlyBGE or onlyBDE:
        if onlyBGE:directory+='BGE, '
        else:directory+='BDE, '
    directory+='Confidence '+str(thres)+', NonToImmune Paths'
    if not os.path.exists(directory):os.makedirs(directory)
    graphLst = __BNGraphsTinyBegin(globDir=globDir,pathways=pathways,onlyBGE=onlyBGE,onlyBDE=onlyBDE);placeLs = []
    if(excludePre):graphLst = __removeLstFunc(graphLst,'PrePure')
    for graph in graphLst:place = graph.split('\\')[1].split(',')[0].strip();placeLs.append(place)
    placeLs=list(set(placeLs))
    if mergedAll:graphLst=__removeLstFunc(graphLst,'Pre-BN')
    immuneDict={};dictIn={}
    for i in range(58):
        immuneDict['Immune'+str(i)]=['Red'];dictIn['NonImmune'+str(i)]='NonImmune'+str(i)
        dictIn['Immune'+str(i)]='Immune'+str(i);colorDictIn['NonImmune'+str(i)]=[0];colorDictIn['Immune '+str(i)]=[2]
    immuneDict['Heat']='red';colorDictIn['Heat']=[1];colorsLst=(('set312',12),('pastel28',8),('pastel19',9));colorsLen=29
    for graph in graphLst:
        g = __getMergedG([graph],immuneDict=immuneDict,dropImmune=False,dropPure=False,agnostic=False,
                         thres=thres,nonToIm=True,dictIn=dictIn,
                         thresFloor=0.1,interPath=True,immuneNonOnly=True,dirThres=dirThres,
                         pathways=pathways,below=False,allowPaths=True)
        inputsLst=g[2];corr=__getCorrInputsLst(inputsLst)
        setupOut = __setupHelpMakeGraphsCreate(g=g[0],undirected=True,dirThres=dirThres,colored=False,colorsLst=[],
                                               blackList=False,blackListFile='',
                                               colorDictIn={},colorImmuneDict={},onlyConnected=True)
        Nodes,direct,undirect,colorDict,colorPathDict,topLs=setupOut
        corr = __getCorrInputsLst(inputsLst);targetLst=[]
        for i in list(set(direct['to'])):
            if not 'Non' in i:targetLst.append(i)
        doneLst=[];pathLst=[]
        for i in direct.index:
            if not i in doneLst and 'Non' in direct.loc[i]['from']:
                path,lst = recursiveHelpPathFinder(direct=direct,i=i,doneLst=doneLst,targetLst=targetLst,
                                                   corr=corr,allowPosCorr=allowPosCorr,allowNegCorr=allowNegCorr)
                if path:
                    doneLst.extend(lst);pathLst.append(path)
        __masterOmniPathHelper(pathLst=pathLst,directory=directory,useFile=useFile,file=file,corr=corr)
    return 1
def runOmniPath(geneFrom,geneTo):
    try:
        OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&source_target=AND&sources='+geneFrom
        OmniPathURL+='&targets='+geneTo+'&fields=sources,references'
        __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
    except:
        return runOmniPath(geneFrom,geneTo)
    return df
def __OmniPathHelperFunc(page):
    totalLst=[];n=0
    for line in page.split('\\n'):
        cols=line.split('\\t');totalLst.append(cols)
    if len(totalLst)>2:
        for ls in totalLst:
            if len(ls)>1:n+=1
        totalLst=totalLst[:n]
        try:
            df=pd.DataFrame(totalLst);df.columns=df.loc[0];df=df.T;df.drop(columns=[0],inplace=True);df=df.T
            return df
        except:
            return False
    else: 
        return False
def getTFs(imm,useFile,file):
    splt=imm.split('Immune');tpy=splt[0];num=splt[1]
    immFile='TF Anaylsis/Inputs\\'+tpy+'Immune Clique '+num+' Gene List.txt'
    input_genes=readLst(immFile);x2k_results = run_X2K(input_genes,useFile=useFile,file=file)
    x2k_tf=pd.DataFrame(x2k_results['X2K']['nodes'])
    if len(x2k_tf.index)>0:
        nameLst=[]
        for name in list(x2k_tf['name']):
            name=name.split('_')[0];nameLst.append(name)
        x2k_tf['name']=nameLst
    return x2k_tf
def getTFsNonImmLst(nonImmLst,useFile,file):
    output=[]
    for nonImm in nonImmLst:
        df=getTFs(nonImm,useFile,file);output.append(df)
    return output
def allLst(lst):
    for elem in lst:
        if not elem:return False
    return True
def getGeneFromUniport(geneID):
    try:
        url='https://www.uniprot.org/uniprot/'+geneID;__r = requests.get(url);page = str(__r.content)
        gene=page.split(' -')[0].split('>')[-1]
        return gene
    except:
        return getGeneFromUniport(geneID)
            
def __masterOmniPathHelper(pathLst,directory,useFile,file,corr,cutoff=20,tfs=True):
    for path in tq(pathLst,desc='Paths'):
        frst=True;imm = path[0];nonImmLst=path[1:]
        x2k_tfImm=getTFs(imm,useFile,file);x2k_tfNonImmLst = getTFsNonImmLst(nonImmLst,useFile,file)
        counts = pd.read_excel('Avi Results Counts.xlsx',index_col=0)
        removeTFs=list(counts[counts['Number of Cliques']>cutoff].index)
        lst,lst2=removeCommonTFs(x2k_tfImm,x2k_tfNonImmLst,removeTFs)
        pathName=path.copy();pathName.reverse()
        for i in range(len(lst)-1):
            source=lst[i];target=lst[i+1]
            sourceTFs,targetTFs=fixSourceTargetTFs(source,target,tfs=tfs);df=runOmniPath(sourceTFs,targetTFs)
            if type(df)==pd.core.frame.DataFrame:
                df['Clique Connection']=str(pathName[i])+' -> '+str(pathName[i+1])
                df=checkConsistencyOfTFs(df,corr)
                if frst:dfAll=df.copy();frst=False
                else:dfAll = pd.concat([dfAll,df],ignore_index=True)
        pathName=', '.join(pathName);sourceLst=[];targetLst=[]
        if frst:print('Warning: No matches for any of the inputed TFs')
        else:
            dfAll=addSourcePathways(dfAll);dfAll=keepTFPaths(dfAll)
            if len(dfAll.index)>0:dfAll.to_excel(directory+'/'+pathName+'.xlsx')
    return 1
def checkConsistencyOfTFs(df,corr):
    keepLst=[]
    for i in df.index:
        connect=df.loc[i]['Clique Connection'];cliq1,cliq2=connect.split(' -> ');corVal = corr.loc[cliq1][cliq2]
        fits=False
        if corVal<0:
            if int(df.loc[i]['is_inhibition']):fits=True
        else:
            if int(df.loc[i]['is_stimulation']):fits=True
        if fits:keepLst.append(i)
    df2=df.loc[keepLst]
    return df2

def keepTFPaths(df):
    keepLst=[]
    for i in df.index:
        targ=df.loc[i]['target']
        for j in df.index:
            if targ == df.loc[j]['source'] and j>i:keepLst.extend([i,j])
    keepLst=list(set(keepLst));keepLst.sort();df2=df.loc[keepLst]
    return df2

In [16]:
def __fixGeneNamesUni(df):
    sourceLst=[];targetLst=[]
    lst=list(df.columns);lst=[lst[0].replace("b'",'')]+lst[1:];df.columns=lst
    genes=list(set(list(df['source'])+list(df['target'])));geneDict={}
    for gene in genes:
        geneDict[gene]=getGeneFromUniport(gene)
    for i in df["source"]:sourceLst.append(geneDict[i])
    for i in df["target"]:targetLst.append(geneDict[i])
    df["source"]=sourceLst;df["target"]=targetLst
    return df
def addSourcePathways(df,useLst=['c5.go.bp']):
    df=__fixGeneNamesUni(df);fullDict=getFullDict(outputLst=False,useLst=useLst)
    fullDict2={};immuneLst= ['T-effector', 'IFNG induced', 'Antigen processing machinery']
    for path in everyDict:
        if path in immuneLst:
            lst=__runGenes(everyDict[path])
            for elem in lst:
                if elem in fullDict2:fullDict2[elem].append(path)
                else:fullDict2[elem]=[path]
    blDictGenes={}
    for path in fullDict:
        lst=fullDict[path]
        if len(lst)<100:
            for elem in lst:
                if elem in blDictGenes:blDictGenes[elem].append(path)
                else:blDictGenes[elem]=[path]
    kobeGenes = pd.read_excel('Genelist_desert.xlsx',index_col=0)
    df2 = pd.read_csv('Pathways4model.csv',index_col=0);df2=df2.dropna()
    genes=list(kobeGenes.index);genes = list(set(genes))
    blDict, blDict2 = __helpGetBLDict(df2,'MANUAL CATEGORIES ',useGenes=True,genesIn=genes)
    blDict, blDict2 = __helpGetBLDict(kobeGenes,'pathway',blDict,blDict2,usePathwayName=False,useGenes=True,genesIn=genes)
    for path in everyDict:
        lst=__runGenes(everyDict[path]);path=path.replace('-A','/A')
        if path in blDict:blDict[path].extend(lst)
        else:blDict[path]=lst
    blDict2={}
    for path in blDict:
        lst = blDict[path]
        for elem in lst:
            if elem in blDict2:blDict2[elem].append(path)
            else:blDict2[elem]=[path]
    pathLst=[];numLst=[]
    for gene in list(df['target']):
        if gene in blDict2:
            lst=blDict2[gene];num=len(lst)
            pathLst.append(lst);numLst.append(num)
        else:
            pathLst.append([]);numLst.append(0)
    df.insert(2,'Pathways with Target',numLst);df.insert(2,'# of Pathways with Target',pathLst);path31Lst=[]
    for gene in list(df['source']):
        if gene in blDict2:
            lst=blDict2[gene];path31Lst.append(lst)
        else:path31Lst.append(False)
    blDictGenes2={}
    for path in fullDict2:
        lst=fullDict2[path]
        for elem in lst:
            if elem in blDictGenes2:blDictGenes2[elem].append(path)
            else:blDictGenes2[elem]=[path]
    sourceLst=list(df['source']);sourceGenesLst=[]
    for elem in sourceLst:
        if elem in blDictGenes:sourceGenes=blDictGenes[elem]
        else:sourceGenes=[]
        sourceGenesLst.append(sourceGenes)
    surceLenLst=[]
    for elem in sourceGenesLst:surceLenLst.append(len(elem))
    df.insert(2,'Pathways with Source',sourceGenesLst);df.insert(2,'# of Pathways with Source',surceLenLst)
    df.insert(2,'Source in 31 Special Pathways',path31Lst)
    return df
def multiInsert(dfIn,lst):
    df=dfIn.copy()
    for i,name,elem in lst:df.insert(i,name,elem)
    return df
def __parallelRunAvi(imm,immDf,nonDf,place,useFile=False,file='',outDict={},tfs=True):
    try:
        val=False;immNum=imm.split('_')[0];immTyp=immNum.split('une')[0]+'une';immNum=immNum.split('une')[-1]
        if 'Non' in immTyp:
            if immNum in nonDf.columns:
                genes=list(nonDf[immNum].dropna());val=True
        else:
            if immNum in immDf.columns:
                genes=list(immDf[immNum].dropna());val=True
        if val:
            try:
                if tfs:
                    imm_tf = pd.read_excel('TF Anaylsis/Avi Results/TFs/'+place+' '+imm+'.xlsx',index_col=0)
                else:
                    imm_tf = pd.read_excel('TF Anaylsis/Avi Results/Kinase/'+place+' '+imm+'.xlsx',index_col=0)
            except FileNotFoundError:
                imm_results = run_X2K(genes,useFile=useFile,file=file)
                if tfs:
                    imm_tf=pd.DataFrame(imm_results['X2K']['nodes'])
                    imm_tf.to_excel('TF Anaylsis/Avi Results/TFs/'+place+' '+imm+'.xlsx')
                else:
                    imm_tf=pd.DataFrame(imm_results['KEA'])
                    imm_tf.to_excel('TF Anaylsis/Avi Results/Kinase/'+place+' '+imm+'.xlsx')
        else:imm_tf=pd.DataFrame()
        outDict[imm]=imm_tf
        return outDict
    except TimeoutError:
        return __parallelRunAvi(imm=imm,place=place,immDf=immDf,nonDf=nonDf,useFile=useFile,
                                file=file,outDict=outDict,tfs=tfs)
    return (immTyp,imm,imm_tf,nonImm)

In [17]:
def runOmniParallel(imm,nonImm,dic,tfs):
    imm_tf=dic[imm];non_tf=dic[nonImm]
    if len(imm_tf.index)>1 and len(non_tf.index)>1:
        sourceTFs,targetTFs=fixSourceTargetTFs(imm_tf,non_tf,tfs);omniRes=runOmniPath(sourceTFs,targetTFs)
        if type(omniRes)==pd.core.frame.DataFrame:
            if len(omniRes.index)>1:
                omniRes['Clique Connection']=nonImm+' -> '+imm;omniRes.columns=['source']+list(omniRes.columns[1:])
                omniRes=checkConsistencyOfTFs(omniRes,corr);omniRes=addSourcePathways(omniRes)
                return omniRes
    return pd.DataFrame()
def getCliqueDf(file,immOrNon,ver='v7'):
    fName=file.split('\\')[-1].replace(', '+ver,'')
    lst=fName.split(' N ');fName=','.join([lst[0],' N '+lst[1]])
    fName=fName.replace('Clique Overlapping','Overlapping Cliques, '+immOrNon)
    df=pd.read_csv('Cliques/'+fName+'.csv',index_col=0)
    return df
def createCliques(placeLst,version=7,thres=0.6,title='Overlapping Cliques'):
    kobeGenes = pd.read_excel('Genelist_desert.xlsx',index_col=0)
    df2 = pd.read_csv('Pathways4model.csv',index_col=0);df2=df2.dropna()
    useLst = ['c2.cp','c5.go.bp','h.all'];fullDict = getFullDict(outputLst=False,useLst=useLst);dict2={}
    for i in df2.index:
        if i in fullDict:
            lst = fullDict[i];dict2[i]=lst
    genes=list(kobeGenes.index)
    for col in dict2:
        lst = dict2[col];genes.extend(lst)
    genes = list(set(genes))
    blDict, blDict2 = __helpGetBLDict(df2,'MANUAL CATEGORIES ',useGenes=True,genesIn=genes)
    blDict, blDict2 = __helpGetBLDict(kobeGenes,'pathway',blDict,blDict2,usePathwayName=False,useGenes=True,genesIn=genes)
    removeLst=[];immuneLst= ['T-effector', 'IFNG induced', 'Antigen processing machinery']
    for path in everyDict:
        if path in immuneLst:
            lst = __runGenes(everyDict[path])
            if path in blDict:blDict[path].extend(lst)
            else:blDict[path]=lst
    immDict2={};nonImmDict2={}
    for path in blDict:
        lst=blDict[path]
        if path in immuneLst:
            for elem in lst:immDict2[elem]=path
        else:
            for elem in lst:nonImmDict2[elem]=path
    immSet=set(immDict2.keys());nonSet=set(nonImmDict2.keys());removeLst=[]
    for i in immDict2.keys():
        if i in nonSet:removeLst.append(i)
    for i in removeLst:del immDict2[i]
    removeLst=[]
    for i in nonImmDict2.keys():
        if i in immSet:removeLst.append(i)
    for i in removeLst:del nonImmDict2[i]
    for place in tq(placeLst,desc='Creating Cliques'):
        dfPlace = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        numSamples=len(dfPlace.columns)
        titleImm=title+', Immune';titleNon=title+', Non-Immune'
        __createCliquesHelper(dfPlace,immDict2,version=version,thres=thres,numSamples=numSamples,title=titleImm,place=place)
        __createCliquesHelper(dfPlace,nonImmDict2,version=version,thres=thres,numSamples=numSamples,title=titleNon,place=place)
    for place in tq(placeLst,desc='Saving'):
        dfPlace = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        numSam=len(dfPlace.columns)
        dfImm=pd.read_csv('Cliques/Overlapping Cliques, Immune, '+place+', N = '+str(numSam)+'.csv',index_col=0)
        dfNonImm=pd.read_csv('Cliques/Overlapping Cliques, Non-Immune, '+place+', N = '+str(numSam)+'.csv',index_col=0)
        dfPlace=dfPlace.T
        dfOut,colLst = __helpMergeDfsCliques(dfImm,dfPlace,title='Immune',colLst=[])
        dfOut,colLst = __helpMergeDfsCliques(dfNonImm,dfPlace,title='Non Immune',frst=False,colLst=colLst,dfOut=dfOut)
        dfOut.columns = colLst;dfOut=dfOut.T;dfOut=__heatDF(dfOut,place=place)[0];dfOut=dfOut.T
        dfOut.to_csv('Run BNLearn/Inputs/Clique Overlapping, '+place+' N = '+str(numSam)+', v'+str(version),sep='\t')
    return 1
def __createCliquesHelper(dfPlace,blDict2,version,thres,title,numSamples,place):
    geneLst = list(blDict2.keys());geneLst=list(set(geneLst));geneLst=__goodGenes(geneLst,dfPlace.index)
    dfPlace = dfPlace.loc[geneLst].T;corrP=dfPlace.corr(method='pearson');corrS=dfPlace.corr(method='spearman')
    corr = (corrP+corrS)/2;keep = corr>= thres;keepLst=[]
    for i in keep.index:
        ser = keep[i][keep[i]]
        for j in ser.index:
            if i!=j and not (i,j) in keepLst:keepLst.append((i,j))
    G = nx.Graph();G.add_edges_from(keepLst);out = nx.find_cliques(G);ls = list(out);n=0;graphDict={};meanDict={}
    for l in ls:
        graphDict[n]=l;df2=dfPlace[l].T;mean=df2.mean();meanDict[n]=mean;n+=1
    inputs=geneLst
    out = Parallel(n_jobs=7)(delayed(parallelGraphDict)(gene,graphDict,dfPlace,meanDict,thres) for gene in inputs)
    finalDict={}
    for n in out[0][1]:finalDict[n]=[]
    for gene, dic in out:
        for n in dic:
            if dic[n]:finalDict[n].append(gene)
    finalDict2={}
    for n in out[0][1]:finalDict2[n]=[]
    for gene,dic in out:
        for n in dic:
            if dic[n] and not gene in finalDict2[n]:finalDict2[n].append(gene)
    finalDict3={}
    for k in sorted(finalDict2, key=lambda k: len(finalDict2[k]), reverse=True):
        finalDict3[n] = finalDict2[k];n+=1
    noLst=[];finalDict2={}
    for n in finalDict3:
        ls=finalDict3[n];ls2=[]
        for elem in ls:
            if not elem in noLst:ls2.append(elem)
        finalDict2[n]=ls2;noLst.extend(ls2)
    finalDict={};m=0
    for n in finalDict2:
        if len(finalDict2[n])>0:
            finalDict[m]=finalDict2[n];m+=1
    maxx=-1
    for n in finalDict:
        ls = finalDict[n]
        if len(ls)>maxx:maxx=len(ls)
    for n in finalDict:
        ls=finalDict[n]
        for i in range(maxx-len(ls)):ls.append('')
        finalDict[n]=ls
    title2=title+', '+place+', N = '+str(numSamples)
    df = pd.DataFrame.from_dict(finalDict) ;df.to_csv(title2+'.csv')
    return 1

In [18]:
def postOmniCleanup(df):
    dict2={};immuneLst= ['T-effector', 'IFNG induced', 'Antigen processing machinery']
    for path in everyDict:
        if path in immuneLst:
            lst=__runGenes(everyDict[path])
            for elem in lst:
                if elem in dict2:dict2[elem].append(path)
                else:dict2[elem]=[path]
    lst=[]
    for i in df.index:
        j = df.loc[i]['target']
        if j in dict2:lst.append(i)
    df=df.loc[lst];dropLst=[]
    for n in range(len(df.index)):
        i = df.index[n];ser=df.loc[i]
        for m in range(n+1,len(df.index)):
            j=df.index[m];ser2=df.loc[j]
            if ser.equals(ser2):dropLst.append(ser2)
    df=df.drop(dropLst)
    return df
def createOmniInputFile(outDict,pairLst,nonDf,place,corVal,fullDict,thres,includeNumKinase=False):
    blDict2=getBlDict2(fullDict,removeExtra=True)
    immDict={};nonDict={};immLst=[];nonLst=[]
    immFiles=glob.glob('TF Anaylsis/Avi Results/Kinase/*'+place+'* Immune*')
    if len(immFiles)==0:
        raise
    for file in immFiles:
        fileName=file.split('\\')[-1].split('.')[0]
        dfImm=pd.read_excel(file,index_col=0)
        if len(dfImm.index)>0:
            immDict[fileName]=(list(dfImm['name']),list(dfImm['pvalue']));immLst.extend(list(dfImm['name']))
    nonFiles=glob.glob('TF Anaylsis/Avi Results/Kinase/*'+place+'*NonImm*')
    for file in nonFiles:
        fileName=file.split('\\')[-1].split('.')[0];dfNon=pd.read_excel(file,index_col=0)
        if len(dfNon.index)>0:
            nonDict[fileName]=(list(dfNon['name']),list(dfNon['pvalue']));nonLst.extend(list(dfNon['name']))
    immSet=set(immLst);nonSet=set(nonLst);both=list(immSet.intersection(nonSet))
    immUnique=[i for i in immSet if i not in both];nonUnique=[i for i in nonSet if i not in both]
    outImmDict={}
    for elem in immLst:
        if elem in outImmDict:outImmDict[elem]+=1
        else:outImmDict[elem]=1
    outNonDict={}
    for elem in nonLst:
        if elem in outNonDict:outNonDict[elem]+=1
        else:outNonDict[elem]=1
    inputs=pairLst
    dfLst= Parallel(n_jobs=7)(delayed(__parallelOmniInput)(outDict=outDict,nonImm=nonImm,imm=imm,fullDict=fullDict,
                                                           nonDf=nonDf,includeNumKinase=includeNumKinase,thres=thres,
                                                           outImmDict=outImmDict,outNonDict=outNonDict,blDict=blDict2) 
                              for nonImm,imm in inputs)
    if len(dfLst)>0:
        dfAll = pd.concat(dfLst,ignore_index=True)
        lst=place.split(' ')
        if 'Kidney' in place or 'Head' in place:place = ' '.join([lst[0],lst[1],lst[2]])
        else:
            if 'Brain' in place:place=lst[0]
            else:place=' '.join([lst[0],lst[1]])
        title='TF Anaylsis/Avi Results/Omni Inputs/'+place
        title+=', OmniPath Input, NonImm-Imm Connections Cor '+str(corVal)+'.xlsx'
        dfAll.to_excel(title)
    return 1
def __parallelOmniInput(outDict,nonImm,imm,nonDf,outImmDict,outNonDict,blDict,includeNumKinase,fullDict,thres):
    immKin = outDict[imm];nonKin = outDict[nonImm]
    print(immKin,nonKin);tr=ty
    if len(immKin.columns)>0 and len(nonKin.columns)>0:
        immKin=immKin[immKin['pvalue']<=thres];nonKin=nonKin[nonKin['pvalue']<=thres]
    if len(immKin.index)>0 and len(nonKin.index)>0:
        nonImmNum=nonImm.split('_')[0].split('ne')[-1];targetKin=set(immKin['name']);sourceKin=set(nonKin['name'])
        targetLst = [g for g in targetKin if g not in sourceKin];sourceLst = [g for g in sourceKin if g not in targetKin]
        sourceGenes=list(set(nonDf[nonImmNum].dropna()))
        output =  omniInputHelper(targetLst,sourceLst,outImmDict,outNonDict,sourceGenes,blDict,fullDict=fullDict)
        numInTarg,numInSource,sourceGLst,pathLstTarg,pathLstSource= output
        df=pd.DataFrame()
        df['Source NonImmune Genes']=sourceGLst
        df['Source NonImmune Kinases']=sourceLst;df['Source Kinases Pathways']=pathLstSource
        if includeNumKinase:
            df['Source NonImmune Kinase Likely Immune']=numInSource
        df['Target Immune Kinases']=targetLst;df['Target Kinases Pathways']=pathLstTarg
        if includeNumKinase:
            df['Target Immune Kinase Likely Immune']=numInTarg
        df['Connection']=nonImm+' -> '+imm
    else: df =pd.DataFrame()
    return df

def omniInputHelper(targetLst,sourceLst,outImmDict,outNonDict,sourceGenes,blDict,fullDict):
    numInTarg,sourceGLst,pathLstTarg = omniInputHelper2(targetLst,outImmDict,outNonDict,sourceGenes,
                                                        sourceG=True,blDict=blDict,fullDict=fullDict)
    numInSource,_,pathLstSource = omniInputHelper2(sourceLst,outNonDict,outImmDict,sourceGenes,
                                                   sourceG=False,blDict=blDict,fullDict=fullDict)
    if len(targetLst)>len(sourceLst):
        a=targetLst;b=(sourceLst,numInSource,[],pathLstSource)
    else:
        a=sourceLst;b=(targetLst,numInTarg,sourceGLst,pathLstTarg)
    lst,numInImm,gLst,pathLst=b
    for i in range(len(a)-len(b[0])):
        numInImm.append(True)
        gLst.append(sourceGenes);lst.append('')
        pathLst.append([])
    return numInTarg,numInSource,sourceGLst,pathLstTarg,pathLstSource

def getImmuneLst(ciber=False,union=True):
    if ciber:
        ciberGenes=pd.read_csv('Cibersort Immune.txt',index_col=0,sep='\t')
        immLst=set(list(ciberGenes.index))
    else:
        iris=pd.read_csv('IRIS.csv',index_col=0);immLst=list(iris['name'])
        immunone=pd.read_csv('ImmunomeDB.csv',index_col=0);immLst.extend(list(immunone['name']))
        immport=pd.read_csv('Immport.csv',index_col=0);immLst.extend(list(immport['name']))
        if union:
            ciberGenes=pd.read_csv('Cibersort Immune.txt',index_col=0,sep='\t')
            immLst.extend(list(set(list(ciberGenes.index))))
    return immLst
def omniInputHelper2(lst,outDict1,outDict2,sourceGenes,sourceG,blDict,fullDict):
    immLst=getImmuneLst()
    sourceGLst=[];numIn1=[];pathLst=[]
    for elem in lst:
        if elem in immLst:
            numIn1.append(True)
        else:
            numIn1.append(False)
        if sourceG:
            sourceGLst.append(sourceGenes)
        if elem in blDict:
            pathLst.append(blDict[elem])
        else:
            pathLst.append([])
    pathLst=onlyKeepImportantPaths(pathLst,fullDict=fullDict)
    return numIn1,sourceGLst,pathLst
def getBlDict2(Dict,removeExtra=False):
    blDict2={}
    for path in Dict:
        continuu=True
        if removeExtra:
            if 'ST' in path or 'PID' in path or 'BIO' in path:
                continuu=False
        if continuu:
            lst=Dict[path]
            for elem in lst:
                if elem in blDict2:
                    blDict2[elem].append(path)
                else:
                    blDict2[elem]=[path]
    return blDict2

def onlyKeepImportantPaths(pathLst,fullDict,lstLen=90):
    outLst=[]
    for lst in pathLst:
        outPaths=onlyKeepImportantPathsHelper(lst,fullDict,lstLen)
        outPaths=list(set(outPaths))
        outLst.append(outPaths)
    return outLst
def onlyKeepImportantPathsHelper(lst,fullDict,lstLen):
    outPaths=[]
    for path2 in lst:
        genes2=fullDict[path2];genes2=set(genes2)
        if len(genes2)<lstLen:
            continuu=True
            for path1 in lst:
                genes1=fullDict[path1];genes1=set(genes1)
                if path2 != path1 and len(genes1)>lstLen:
                    if path1 not in outPaths:
                        outPaths.append(path1)
                    count=len(genes1.intersection(genes2))
                    total = count/len(genes2)
                    if total > 0.6:
                        continuu=False
                else:
                    continuu=False
            if continuu:
                outPaths.append(path2)
    lstLen=lstLen-10
    if len(outPaths)<4 and lstLen>0:
        outPaths=onlyKeepImportantPathsHelper(lst,fullDict,lstLen)
    return outPaths

In [19]:
def helpPlotClusterNonImm(i,df,path,version,metric,size,show=True,addTitle='',spltNum=1,spltLst=[],
                          allDf=False,Colors=(),addTitle2='',legendTitle='Pathway'):
    if len(df.columns)>0:
        if spltNum<1:
            spltNum=1
        out='Graphs'
        if not os.path.exists(out):os.makedirs(out)
        out+='/'
        sizeY = max(7 - (len(df.columns)/20),1)
        annotSize= max(4 - len(df.columns)/20,1)
        if spltNum==1 and len(df.columns)>2:
            row_cluster=True;col_cluster=True
        else:
            row_cluster=False;col_cluster=False
        if allDf:
            dfColors1,colors1Dict,dfColors2,colors2Dict=Colors
            colorLst1=[];colorLst2=[]
            for path in dfColors1:
                colorLst1.append(colors1Dict[path])
            for path in dfColors2:
                colorLst2.append(colors2Dict[path])
            colorLst=[colorLst1,colorLst2]
            colors1Dict.update(colors2Dict)
            
            cg = sb.clustermap(df,vmax=1,cmap=cmapSpecial,vmin=-1,xticklabels=0,yticklabels=1,
                               row_cluster=row_cluster,col_cluster=col_cluster,col_colors=colorLst,
                               figsize=size,metric=metric,cbar_pos=(0, 0.5, 0.05, 0.15))
            sizeY=2
            for x,y in zip(colors1Dict.keys(),colors1Dict.values()):
                cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=6, solid_capstyle="butt")
            cg.ax_heatmap.legend(loc=(-0.5,0), title=legendTitle)
        else:   
            cg = sb.clustermap(df,vmax=1,cmap=cmapSpecial,vmin=-1,xticklabels=1,yticklabels=1,
                               row_cluster=row_cluster,col_cluster=col_cluster,
                               figsize=size,metric=metric,annot=True,annot_kws={"size":annotSize})
        if sizeY < 2:
            plt.close()
            if spltNum==1:
                row = cg.dendrogram_row.reordered_ind;col = cg.dendrogram_col.reordered_ind
                df = __setCluster(df,row,col)
            else:
                spltNum-=1
            splt=int(len(df.columns)/2)
            df1Cols=df.columns[:splt];df2Cols=df.columns[:splt]
            df1= df[df1Cols];df2=df[df2Cols]
            Colors1=Colors[:splt];Colors2=Colors[splt:]
            spltNum,spltLst=helpPlotClusterNonImm(i,df1,path,version,metric,addTitle=' Part '+str(spltNum),
                                                  spltNum=spltNum+1,show=show,spltLst=spltLst,allDf=allDf,Colors=Colors1)
            spltNum,spltLst=helpPlotClusterNonImm(i,df2,path,version,metric,addTitle=' Part '+str(spltNum),
                                                  spltNum=spltNum+1,show=show,spltLst=spltLst,allDf=allDf,Colors=Colors2)
            return spltNum,spltLst
        else:
            title=i+' to Immune Genes, Correlation, v'+str(version)+', '+metric+addTitle2+addTitle
            cg.fig.suptitle(title,size=15)
            plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=7)
            plt.setp(cg.ax_heatmap.get_xticklabels(),ha='right',size=sizeY,rotation='45')
            cg.ax_heatmap.set_xlabel('Non-Immune Immune Pairings',size=12);cg.ax_heatmap.set_ylabel('Cancer Areas',size=12)
            cg.ax_row_dendrogram.set_visible(False);cg.ax_col_dendrogram.set_visible(False)
            title=out+title;
            if not allDf:cg.ax_heatmap.set_title(i+', '+path,size=7)
            plt.savefig(title+'.pdf',bbox_inches = "tight",dpi=170);spltLst.append(title+'.pdf')
            if show:plt.show()
            else:plt.close()
    return spltNum+1,spltLst

def plotCorrImmuneNon(place,version,size=(9,6),metric='cosine'):
    blDict,blDict2=__getBLDict(False)
    immuneGenes=[];nonGenes=[]
    immuneLst=['T-effector','IFNG induced','Antigen processing machinery']
    for path in blDict:
        lst=blDict[path]
        if path in immuneLst:
            immuneGenes.extend(lst)
        else:
            nonGenes.extend(lst)
    immuneGenes=list(set(immuneGenes))
    nonGenes=list(set(nonGenes))
    df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
    df = __heatDF(df,place,insert=0,zscore=True)[0];df=__pureDf(df)
    immGenesP=__goodGenes(immuneGenes,df.index);nonGenesP=__goodGenes(nonGenes,df.index)
    immGenesP.extend(['Purity Scores','Heat Scores'])
    genes=list(set(immGenesP+nonGenesP))
    df.index.name='Non-Immune Genes';df=df.loc[genes];df=df.T
    corP=df.corr(method='pearson');corS=df.corr(method='spearman')
    cor=(corP+corS)/2;cor=cor.loc[immGenesP][nonGenesP];cor.index.name='Immune Genes'
    out='Graphs/'+place
    if not os.path.exists(out):os.makedirs(out)
    out+='/'
    cg = sb.clustermap(cor,vmax=1,cmap=cmapSpecial,vmin=-1,xticklabels=0,yticklabels=1,
                       figsize=size,metric=metric)
    title=place+', v'+str(version)+', NonImmune to Immune Correlation, '+metric
    cg.fig.suptitle(title,size=12)
    plt.setp(cg.ax_heatmap.get_yticklabels(),va='bottom',size=10,rotation='0')
    cg.ax_heatmap.set_xlabel('Non-Immune Genes',size=15);cg.ax_heatmap.set_ylabel('Immune Genes',size=15)
    col = cg.dendrogram_col.reordered_ind;row= list(range(len(cor.index)));cor2=__setCluster(cor,row,col)
    lst=list(cor2.columns);lstTitle='Order of Non-Immune Genes for '+title
    saveLst(lst,out+lstTitle+'.txt');title=out+title
    plt.savefig(title+'.png',bbox_inches = "tight",dpi=170);plt.close()
    return 1

In [20]:
def __getBLDict(useSuperSet=True):
    kobeGenes = pd.read_excel('Genelist_desert.xlsx',index_col=0)
    df2 = pd.read_csv('Pathways4model.csv',index_col=0);df2=df2.dropna()
    useLst = ['c2.cp','c5.go.bp','h.all'];fullDict = getFullDict(outputLst=False,useLst=useLst);dict2={};dict2Out={}
    for i in df2.index:
        if i in fullDict:
            lst = fullDict[i];dict2[i]=lst
            for elem in lst:
                dict2Out[elem]=i
    genes=list(kobeGenes.index)
    for col in dict2:
        lst = dict2[col];genes.extend(lst)
    genes = list(set(genes))
    blDict, blDict2 = __helpGetBLDict(df2,'MANUAL CATEGORIES ',useGenes=True,genesIn=genes)
    blDict, blDict2 = __helpGetBLDict(kobeGenes,'pathway',blDict,blDict2,usePathwayName=False,useGenes=True,genesIn=genes)
    if useSuperSet:
        for path in everyDict:
            lst=list(set(__runGenes(everyDict[path])))
            path=path.replace('-A','/A')
            if path in blDict:
                lst0=blDict[path];lst0.extend(lst);lst=list(set(lst0))
            blDict[path]=lst
        blDict2={}
        for path in blDict:
            lst=blDict[path]
            for elem in lst:
                if elem in blDict2:blDict2[elem].append(path)
                else:blDict2[elem]=[path]
    return blDict,blDict2,dict2Out
def plotImmVsHeat(place,version):
    df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
    df = __heatDF(df,place,insert=0,zscore=True)[0];df=__pureDf(df)
    immGenesP=__goodGenes(immuneGenes,df.index);immGenesP.extend(['Purity Scores','Heat Scores'])
    dfImm=df.loc[immGenesP];dfImm=dfImm.T
    titleLst=[]
    for gene in dfImm.columns:
        if gene !='Purity Scores' and gene!='Heat Scores':
            g1=dfImm[gene];g2=dfImm['Heat Scores']
            plt.plot(g1,g2,'o');plt.xlabel('Heat Score (Z-Score)');plt.ylabel(gene)
            title=gene+' vs Heat, '+place+' v8';plt.title(title)
            plt.savefig('Graphs/'+title+'.pdf',bbox_inches = "tight",dpi=300);plt.close()
            titleLst.append('Graphs/'+title+'.pdf')
    name='Graphs/'+place+', v'+str(version)+', Immune Genes vs Heat Scatter Plots'
    __mergePDFs(titleLst,name,erase=True)
    return 1

In [21]:
def findPathInOmni(sourceG,targetG,maxSteps,pathsWanted):
    return findPathInOmniHelper(sourceG,targetG,maxSteps,step=0,lst=[],doneLst=[],found=0,pathsWanted=pathsWanted)
def findPathInOmniHelper(sourceG,targetG,maxSteps=4,step=0,lst=[],doneLst=[],found=0,pathsWanted=1):
    if step > maxSteps:
        return False,[],targetG
    else:
        try:
            OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&sources='+sourceG
            OmniPathURL+='&fields=sources,references'
            __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
            
            step+=1
            doneLst.append(sourceG)
            if type(df)==pd.core.frame.DataFrame:
                if targetG in list(df['target_genesymbol']):
                    connect = df[df['target_genesymbol']==targetG]
                    lst.append(connect)
                    return True,lst,targetG
                else:
                    outLst=[]
                    if step <= maxSteps:
                        for row in df.index:
                            lst2=lst.copy();ser=df.loc[row];lst2.append(ser)
                            gene = df.loc[row]['target_genesymbol']
                            if not gene in doneLst:
                                val, out,g = findPathInOmniHelper(gene,targetG,maxSteps=maxSteps,step=step,lst=lst2,
                                                                  doneLst=doneLst,found=found,pathsWanted=pathsWanted)
                                if val:
                                    found+=1
                                    outLst.append(out)
                                    if found>pathsWanted:
                                        break
                    if len(outLst)>0:
                        return True,outLst,targetG
                    else:
                        return False,outLst,targetG
            else:
                return False,[],targetG
        except:
            findPathInOmniHelper(sourceG,targetG,maxSteps,step,lst,doneLst,found,pathsWanted)
def findAndSavePaths(sourceKin,targetKin,maxSteps,pathsWanted,place,connect):
    directory='TF Anaylsis/Outputs/OmniPaths/'+str(maxSteps+1)+' Steps'
    if not os.path.exists(directory):os.makedirs(directory)
    directory+='/'+place
    if not os.path.exists(directory):os.makedirs(directory)
    outDict={}
    for sourceG in tq(sourceKin,desc=connect):
        inputs=targetKin
        out = Parallel(n_jobs=10)(delayed(findPathInOmni)(sourceG,targetG,maxSteps=maxSteps,
                                                          pathsWanted=pathsWanted) for targetG in inputs)
        for tup in out:
            if tup:
                val,path,g = tup
                if val:
                    df=recurGetSer(path)
                    outDict[sourceG+' '+g]=df
    for pair in outDict:
        dfAll = outDict[pair]
        name=directory+'/'+pair+' '+connect+'.xlsx'
        name=name.replace(' -> ',' ')
        dfAll.to_excel(name)
    return 1
def parallelFindAndSaveOmni(df,connect,maxSteps,pathsWanted,place):
    dfCliq=df[df['Connection']==connect]
    dfCliqSource = dfCliq[dfCliq['Source NonImmune Kinase Likely Immune']==False]
    dfCliqTarg = dfCliq[dfCliq['Target Immune Kinase Likely Immune']]
    sourceKin = list(dfCliqSource['Source NonImmune Kinases'].dropna())
    targetKin = list(dfCliqTarg['Target Immune Kinases'].dropna())
    if len(sourceKin)>0 and len(targetKin)>0:
        findAndSavePaths(sourceKin,targetKin,maxSteps,pathsWanted,place,connect)
    return 1

def recurGetSer(path,done=True):
    outPut=[]
    for elem in path:
        if type(elem) is list:
            out2 = recurGetSer(elem,False)
            outPut.extend(out2)
        else:
            if type(elem) is pd.core.series.Series:
                elem=pd.DataFrame(elem).T
            if "b'source" in elem.columns and "target" in elem.columns:
                elem=elem.drop(columns=["b'source","target"])
            outPut.append(elem)
    if done:
        df = pd.concat(outPut,ignore_index=True)
        return df
    else:
        return outPut
    
def getImmandNonDict():
    blDict0,blDict20,dict31=__getBLDict(False);immuneGenes=[];nonGenes=[]
    immuneLst=['T-effector','IFNG induced','Antigen processing machinery']
    notLst=['FGFR3', 'ACVR1', 'PCSK9', 'ACAT1', 'CISD1'];blDict={};blDict2={}
    for path in blDict0:
        lst=list(set(blDict0[path]))
        if not path in notLst:
            blDict[path] = lst
            for elem in lst:
                if elem in blDict2:blDict2[elem].append(path)
                else:blDict2[elem]=[path]
    nonGenesDict={};immGenesDict={}
    for path in blDict:
        lst=blDict[path]
        if path in immuneLst:
            immuneGenes.extend(lst)
            for elem in lst:
                immGenesDict[elem]=path
        else:
            nonGenes.extend(lst)
            for elem in lst:
                nonGenesDict[elem]=path
    immuneGenes=list(set(immuneGenes));nonGenes=list(set(nonGenes))
    return immGenesDict,nonGenesDict,immuneGenes,nonGenes

def plotImmuneNonGenes(plotNonImmCorr=False,plotAll=False,makeBN=False,version=7,size=(12,8),metric='cosine',numCancers=4,
                       negThres=-0.2,placeLst=[],placeLstBN=[],bnTitle='Kobe Genes',panCancer=True,onlyKobe=False):
    immGenesDict,nonGenesDict,immuneGenes,nonGenes = getImmandNonDict();dfLst=[];immuneGenesNameDict={}
    kobeGenes = pd.read_excel('Genelist_desert.xlsx',index_col=0);kobeGenes=list(kobeGenes.index)
    immuneGenesNameDict['Purity Scores']='Puirty Scores';immuneGenesNameDict['Heat Scores']='Heat Scores'
    for imm in immuneGenes:
        path=immGenesDict[imm].split(' ')[0];immuneGenesNameDict[imm]=imm+'_'+path
    for place in tq(placeLst,desc='dfLst'):
        df = pd.read_csv('Purity/TPM/v'+str(version)+'/TCGA_'+place+'_tpm.tsv',index_col=0,sep='\t')
        df = __heatDF(df,place,insert=0,zscore=True)[0];df=__pureDf(df)
        immGenesP=__goodGenes(immuneGenes,df.index);nonGenesP=__goodGenes(nonGenes,df.index)
        immGenesP.extend(['Purity Scores','Heat Scores']);genes=list(set(immGenesP+nonGenesP))
        df.index.name='Non-Immune Genes';df=df.loc[genes];df=df.T
        corP=df.corr(method='pearson');corS=df.corr(method='spearman')
        cor=(corP+corS)/2;cor=cor.loc[immGenesP][nonGenesP];cor.index.name='Immune Genes'
        dfLst.append((place,cor))
    frst=True
    for place,df in dfLst:
        if frst:
            immGenes=list(df.index);nonGenes=list(df.columns);frst=False
        else:
            immGenes=__goodGenes(immGenes,df.index);nonGenes=__goodGenes(nonGenes,df.columns)
    if plotNonImmCorr:
        total=0
        for i in tq(nonGenes,desc='Plotting'):
        #def _parallelPlotImmNonCorr(i,dfLst,immGenes)
            corDict={};geneOutLst=[];dfLst2=[]
            for place,df in dfLst:
                df=df.T;ser=list(df.loc[i][immGenes]);corDict[place]=ser
            nameLst=[]
            for imm in immGenes:
                name=immuneGenesNameDict[imm];nameLst.append(name)
            df = pd.DataFrame.from_dict(corDict, orient='index',columns=nameLst)
            df.index.name='Cancer Area';df=df.T;df.index.name='Immune Genes';df=df.T
            negCorDf=df[df<=negThres];keepLst=[];n=0
            for col in negCorDf.columns:
                numNeg=np.count_nonzero(~np.isnan(negCorDf[col]))
                if numNeg >= len(negCorDf.index)/4:n+=1
            if n >= numCancers: continuu=True
            else: continuu=False
            if continuu:
                path = nonGenesDict[i]
                pathTitle=i+' '+path;pathTitle=pathTitle.replace('/','-')
                if i in dict31:
                    path+=', '+dict31[i]
                path=path.replace('/','-')
                spltNum,spltLst=helpPlotClusterNonImm(i,df,path,size,version,metric,show=False,spltLst=[],spltNum=0)
                if spltNum > 2:
                    name=spltLst[0].replace(' Part 1','').replace('.pdf','')
                    __mergePDFs(spltLst,name,erase=True);spltLst=[name+'.pdf']
                geneOutLst.extend(spltLst);path=path.replace('/','-')
                name='Graphs/'+pathTitle+', to Imm Genes, v7, cosine, Neg Corr '+str(negThres)+' in Half Cancer Areas in '
                name+=str(numCancers)+' Imm Genes'
                __mergePDFs(geneOutLst,name,erase=True)
                total+=1
    if plotAll or makeBN:
        dfLst2=[]
        for i in tq(nonGenes,desc='Filtering Genes'):
        #def _parallelPlotImmNonCorr(i,dfLst,immGenes)
            corDict={};geneOutLst=[]
            for place,df in dfLst:
                df=df.T;ser=list(df.loc[i][immGenes]);corDict[place]=ser
            nameLst=[]
            for imm in immGenes:
                name=immuneGenesNameDict[imm]
                nameLst.append(name)
            df = pd.DataFrame.from_dict(corDict, orient='index',columns=immGenes)
            df.index.name='Cancer Area';df=df.T;df.index.name='Immune Genes';df=df.T
            negCorDf=df[df<=negThres];keepLst=[];n=0
            for col in negCorDf.columns:
                numNeg=np.count_nonzero(~np.isnan(negCorDf[col]))
                if numNeg >= len(negCorDf.index)/4:n+=1
            if n >= numCancers:
                dfLst2.append((i,df))
            #else:
            #    if onlyKobe:
            #        dfLst2.append((i,df))
        if makeBN:
            for tpy in tq(set(immGenesDict.values()),desc='Making BN Files'):
                bnTitle2 = bnTitle+' '+tpy
                genesIn=[]
                for non, df in dfLst2:
                    #if nonGenesDict[non]=='Fatty acid':
                    genesIn.append(non)
                for col in df.columns:
                    if not 'Scores' in col and immGenesDict[col]==tpy:
                        genesIn.append(col)
                genesIn=list(set(genesIn))
                
                setupBNLearnGenes(placeLstBN,runVLst=[version],panCancer=panCancer,onlyKobeGenes=False,genesIn=genesIn,
                                  tqOn=False,areaTitle=bnTitle2)
        if plotAll:
            seenImmLst=[];seenNonLst=[]
            dfAllLst={};dfColors1=[];dfColors2=[]
            #immGenesDict['Purity Scores']='Purity Scores'
            immGenesDict['Heat Scores']='Heat Scores'
            for non, df in dfLst2:
                for col in df.columns:
                    if not 'Purity' in col:
                        if (not onlyKobe) or (non in kobeGenes and col in kobeGenes):
                            if not non in seenNonLst:
                                seenNonLst.append(non)
                            if not col in seenImmLst:
                                seenImmLst.append(col)
                            ser=df[col]
                            dfAllLst[non+', '+col]=ser
                            if onlyKobe:
                                dfColors1.append(non)
                                dfColors2.append(col)
                            else:
                                dfColors1.append(nonGenesDict[non])
                                dfColors2.append(immGenesDict[col])
            print(len(seenImmLst))
                            
            colors2=['red','orange','maroon','tomato','darksalmon','sienna','coral','pink','gold','yellow','fuchsia','brown']
            colors2.extend(['orangered','crimson','black'])
            colors1=['blue','green','gray','olive','cyan','gold','palegreen','teal','dodgerblue','lightskyblue','deepskyblue']
            colors1.extend(['skyblue','steelblue','darkseagreen','violet','purple'])
            colors1Dict=dict(zip(set(dfColors1),colors1))
            colors2Dict=dict(zip(set(dfColors2),colors2))
            colorLst1=[];colorLst2=[]
            for path in dfColors1:
                colorLst1.append(colors1Dict[path])
            for path in dfColors2:
                colorLst2.append(colors2Dict[path])
            dfAll = pd.DataFrame.from_dict(dfAllLst)
            colors=(dfColors1,colors1Dict,dfColors2,colors2Dict)
            addTitle = ', # Pairs '+str(len(dfAll.columns))+', # NonImmune '+str(len(seenNonLst))
            tittleFront='NonImmune'
            if onlyKobe:
                titleFront='Only Kobe NonImmune'
            spltNum,spltLst = helpPlotClusterNonImm(titleFront,df=dfAll,path='',version=version,size=size,metric=metric,
                                                    show=False,addTitle2=addTitle,legendTitle='Genes',
                                                    spltLst=[],spltNum=0,allDf=True,Colors=colors)
            geneOutLst=[]
            if spltNum > 2:
                name=spltLst[0].replace(' Part 1','').replace('.pdf','')
                __mergePDFs(spltLst,name,erase=True);spltLst=[name+'.pdf']
            geneOutLst.extend(spltLst);path=path.replace('/','-')
            pathTitle='All NonImm Imm Gene Pairings'
            if onlyKobe:
                'All Kobe NonImm Imm Gene Pairings'
            name='Graphs/'+pathTitle+', v'+str(version)+', cosine, Neg Corr '+str(negThres)+' in '
            name+=str(numCancers)+' Cancer Areas in '+str(numCancers)+' Imm Genes'
            __mergePDFs(geneOutLst,name,erase=True)
            dfAll.to_excel(name+'.xlsx')
    return 1

In [22]:
def BNLearnCleanLists(frm,immGenesDict,nonGenesDict,immOrNonLst):
    if frm in immGenesDict:
        immOrNonLst.append(' Immune, '+immGenesDict[frm])
    else:
        if frm in nonGenesDict:
            immOrNonLst.append('Non-Immune, '+nonGenesDict[frm])
        else:
            immOrNonLst.append(frm)
    return 1
def cleanBNLearn(directory='Run BNLearn/Outputs/*Sparse.*',cleaned=True,pruned=True):
    immGenesDict,nonGenesDict,immuneGenes,nonGenes = getImmandNonDict()
    dfLst=[]
    immuneGenesNameDict={}
    immuneGenesNameDict['Purity.Scores']='Puirty Scores';immuneGenesNameDict['Heat.Scores']='Heat Scores'
    for imm in immuneGenes:
        path=immGenesDict[imm].split(' ')[0]
        immuneGenesNameDict[imm]=imm+'_'+path
        
    for file in glob.glob(directory):
        df = pd.read_csv(file,index_col=0)
        df=df[df['strength']>0.3]
        df=df[df['direction']>0.5]
        file=file.replace('.csv',', Cleaned.csv')
        immOrNonLstFrm=[];immOrNonLstTo=[]
        for row in df.index:
            frm=df.loc[row]['from'].replace('.','-')
            to=df.loc[row]['to'].replace('.','-')
            BNLearnCleanLists(frm,immGenesDict,nonGenesDict,immOrNonLstFrm)
            BNLearnCleanLists(to,immGenesDict,nonGenesDict,immOrNonLstTo)     
        df.insert(2,'To Type',immOrNonLstTo)
        df.insert(1,'From Type',immOrNonLstFrm)
        if cleaned:
            df.to_csv(file)

        keepLst=[]
        for i in df.index:
            val=False
            frmType=df.loc[i]['From Type'];toType=df.loc[i]['To Type']
            if 'Non-Imm' in frmType and ' Immune' in toType:
                val=True
            else:
                if ('Heat' in frmType or 'Heat' in toType) or ('Cancer' in frmType or 'Cancer' in toType):
                    val=True
            if val:
                keepLst.append(i)
        df2 = df.loc[keepLst]
        file=file.replace('Cleaned','Pruned')
        if pruned:
            df2.to_csv(file)
    return 1

def fasterOmniConnection(geneLst):
    splt=int(round(len(geneLst)/800,0))
    if splt:
        dfLst=[]
        for i in range(splt):
            gLst = geneLst[i*800:i+1*800]
            gS = ','.join(gLst)
            df=fasterOmniConnectionHelper(gS)
            if type(df)==pd.core.frame.DataFrame:
                dfLst.append(df)
        if len(dfLst)>0:
            dfAll = pd.concat(dfLst)
            dfAll.index=range(len(dfAll.index))
        else:
            dfAll = False
        return dfAll
    else:
        gS = ','.join(geneLst)
        df=fasterOmniConnectionHelper(gS)
        if type(df)==pd.core.frame.DataFrame:
            dfAll=df
        else:
            dfAll=False
    return dfAll
def fasterOmniConnectionHelper(gS):
    try:
        OmniPathURL='https://omnipathdb.org/interactions/?genesymbols=1&sources='+gS
        OmniPathURL+='&fields=sources,references'
        __r = requests.get(OmniPathURL);page = str(__r.content);df=__OmniPathHelperFunc(page)
        return df
    except:
        return fasterOmniConnectionHelper(gS)

In [23]:
def findOmnipathPathsFaster(sourceGene,targetGene,directory,maxSteps,useYellow=False,recivers=[]):
    name=directory+'/'+sourceGene+' '+targetGene+'.xlsx';name=name.replace(' -> ',' ')
    if os.path.exists(name):
        out=(False,[],sourceGene)
    else:
        useYellow=not(useYellow)
        doneLst=[targetGene]
        gLst=[sourceGene]
        rowsDict={};connectionsDict={}
        foundTarget=False;foundYellow=False
        for num in range(maxSteps):
            gLst = [i for i in gLst if not i in doneLst]
            df = fasterOmniConnection(gLst)
            if type(df)==pd.core.frame.DataFrame:
                for row in df.index:
                    ser=df.loc[row]
                    surce = ser['source_genesymbol']
                    targ = ser['target_genesymbol']
                    if not surce in doneLst and surce != targetGene:
                        rowsDict[(surce,targ)]=ser
                        if surce in connectionsDict:
                            connectionsDict[surce].append(targ)
                        else:
                            connectionsDict[surce]=[targ]
                doneLst.extend(gLst)
                if any(reciv in list(df['target_genesymbol']) for reciv in recivers):
                    foundYellow=True
                if targetGene in list(df['target_genesymbol']):
                    foundTarget=True
                gLst=list(df['target_genesymbol'])
            else:break
        if foundTarget and (foundYellow or useYellow):
            out=recurFindConnections(sourceGene,targetGene,connectionsDict,rowsDict,maxSteps)
        else:out=(False,[],sourceGene)
    return out
def cleanOmniConnectionLst(lst):
    while True:
        out=[];allGood=True;n=-1
        for i in range(len(lst)-1):
            n=i;_,g1 = lst[i];g2,__= lst[i+1]
            if g1 != g2:allGood=False
            else:out.append(lst[i])
        out.append(lst[n+1])
        if allGood:break
        lst = out
    return out
def getBottomLst(lst):
    if type(lst[0])==list and len(lst)==1:
        return getBottomLst(lst[0])
    else:
        return lst
def recurFindConnections(startG,findG,connectionsDict,rowsDict,maxNum):
    return recurFindConnectionsHelper(startG,findG,connectionsDict,rowsDict,lst=[],doneLst=[findG],maxNum=maxNum)
def recurFindConnectionsHelper(gene,findG,connectionsDict,rowsDict,lst,doneLst,num=0,maxNum=4):
    if num > maxNum or not gene in connectionsDict:
        return False,[], findG
    else:
        doneLst.append(gene)
        if findG in connectionsDict[gene]:
            lst.append((gene,findG))
            lst = cleanOmniConnectionLst(lst)
            rowsLst=[]
            for g1,g2 in lst:
                rowsLst.append(rowsDict[(g1, g2)])
            return True, rowsLst,findG
        else:
            num+=1
            outLst=[]
            for i in connectionsDict[gene]:
                if not i in doneLst:
                    lst.append((gene,i))
                    val, out, g = recurFindConnectionsHelper(gene=i,findG=findG,connectionsDict=connectionsDict,
                                                             rowsDict=rowsDict,lst=lst,doneLst=doneLst,num=num)
                    if val:
                        outLst.append(out)
                    else:
                        lst=lst[:-1]
                else:
                    val=False
            if len(outLst)>0:
                return True,outLst,findG
            else:
                return False,[],findG

In [24]:
def runOmniSearch(maxSteps,useRecivers=False,directory='TF Anaylsis/Avi Results/Omni Inputs/'):
    maxSteps -=1
    recivers=[]
    if useRecivers:
        dfRec=pd.read_csv('OmniPath Ligands Transmitters and Receivers.csv',index_col=0)
        dfRec1=set(dfRec[dfRec['receiver']].index)
        dfRec2=set(dfRec[dfRec['transmitter']].index)
        keepList = list(set(dfRec1.union(dfRec2)))
        dfRec=dfRec.loc[keepList]
        recivers2=list(dfRec['genesymbol'].dropna())
        recivers2=list(set(recivers2))
        for i in recivers2:
            if ':' in i:
                lst = i.split(':')[1].split('_')
                recivers.extend(lst)
            else:
                recivers.append(i)
    globDir =directory+'*'
    outPutDict={}
    for file in tq(glob.glob(globDir),desc='Overall'):
        df=pd.read_excel(file,index_col=0)
        if len(df.index)>0:
            place=file.split('\\')[-1].split(',')[0]
            dfImm = df[df['Target Immune Kinase Likely Immune']]
            inputs = tq(set(df['Connection']),desc=place)
            countDict={}
            for connect in inputs:
                count=0
                dfCliq=df[df['Connection']==connect]
                dfCliqSource = dfCliq[dfCliq['Source NonImmune Kinase Likely Immune']==False]
                dfCliqTarg = dfCliq[dfCliq['Target Immune Kinase Likely Immune']]
                sourceKin = list(dfCliqSource['Source NonImmune Kinases'].dropna())
                targetKin = list(dfCliqTarg['Target Immune Kinases'].dropna())
                totalPathsLst=[];pathLenLst=[];yellowNumsLst=[];isYellowLst=[]
                if len(sourceKin)>0 and len(targetKin)>0:
                    directory='TF Anaylsis/Outputs/OmniPaths/'+str(maxSteps+1)+' Steps'
                    if not os.path.exists(directory):os.makedirs(directory)
                    directory+='/'+place
                    if not os.path.exists(directory):os.makedirs(directory)
                    outDict={}                
                    for sourceG in sourceKin:
                        geneInputs=targetKin
                        out = Parallel(n_jobs=10)(delayed(findOmnipathPathsFaster)(sourceG,targetG,maxSteps=maxSteps,
                                                                                   directory=directory,
                                                                                   useYellow=useRecivers,recivers=recivers)
                                                  for targetG in geneInputs)
                        for tup in out:
                            if tup:
                                val,path,g = tup
                                name=directory+'/'+sourceG+' '+g+'.xlsx';name=name.replace(' -> ',' ')
                                if val and not os.path.exists(name):
                                    dfAll=recurGetSer(path)
                                    if len(dfAll.index)>0:
                                        count+=1
                                        dfAll,totalPaths,pathLens,yellowNums,isYellows=cleanupOmniDf(dfAll,recivers,g)
                                        dfAll.to_excel(name)
                                        totalPathsLst.append(totalPaths);pathLenLst.extend(pathLens)
                                        yellowNumsLst.extend(yellowNums);isYellowLst.extend(isYellows)
                possiblePaths=len(sourceKin)*len(targetKin)                       
                countDict[connect]=(count,sourceKin,targetKin,totalPathsLst,pathLenLst,yellowNumsLst,isYellowLst)
            
            #saveCount = pd.DataFrame.from_dict(countDict,orient='index',
            #                                   columns=['# Kinase Pairs with at least 1 Path','# Total Kinase Pairs'])
            #countTitle='TF Anaylsis/Outputs/OmniPaths/'+str(maxSteps+1)
            #countTitle+=' Steps/'+place+' Number of gene pairs with paths and ligand recivers in each clique match'
            #saveCount.to_csv(countTitle)
            outPutDict[place]=(len(inputs),countDict)
    return outPutDict

In [25]:
def cleanupOmniDf(dfAllIn,recivers,targetGene):
    dfAll=dfAllIn.copy()
    foundLst = list(dfAll[dfAll['target_genesymbol']==targetGene].index)
    nName=0
    groupLst=[]
    pathLen=[]
    frst=True
    for n in range(len(foundLst)):
        if frst:
            elem = foundLst[n]
            pathLen.append(elem)
            for i in range(elem+1):
                groupLst.append(n)
            frst=False
        else:
            elem=foundLst[n]
            elemPrev=foundLst[n-1]
            pathLen.append(elem-elemPrev)
            for i in range(elemPrev+1,elem+1):
                groupLst.append(n)
        nName+=1
    dfAll.insert(0,'Path Group',groupLst)
    totalPaths = nName
    yellowNumLst,isYellowLst=countYellowsinOmniPaths(dfAll,recivers)
    return dfAll,totalPaths,pathLen,yellowNumLst,isYellowLst

def countYellowsinOmniPaths(dfAll,recivers):
    yellowNumLst=[];isYellowLst=[]
    for i in set(list(dfAll['Path Group'])):
        dfPath=dfAll[dfAll['Path Group']==i]
        yellowNum=0
        for elem in list(dfPath['target_genesymbol']):
            if elem in recivers:
                yellowNum+=1
        yellowNumLst.append(yellowNum)
        isYellowLst.append(yellowNum > 0)
    return yellowNumLst,isYellowLst

def createAndRunOmniSearch(corVal=-0.3,thres=0.01,tfs=False,globDir='Run BNLearn/Inputs/*Clique*',runOmni=False):
    useFile=False;file=''
    inputsLst=glob.glob(globDir)
    useLst = ['c2.cp','h.all']
    fullDict = getFullDict(outputLst=False,useLst=useLst)
    blDict2=getBlDict2(fullDict,removeExtra=True)
    for file in tq(inputsLst,desc='Overall'):
        place=file.split(',')[-2].split('N =')[0].strip()
        immDf=getCliqueDf(file,immOrNon='Immune');nonDf=getCliqueDf(file,immOrNon='Non-Immune')
        corr=__getCorrInputsLst([file],clique=True)
        df=pd.read_csv(file,index_col=0,sep='\t')
        df.columns=corr.columns
        pair1Lst=[];pair2Lst=[];pairLst=[]
        for col in corr.columns:
            for col2 in corr.index:
                if 'Heat' not in col and 'Heat' not in col2:
                    if ('Non' in col and 'Non' not in col2):
                        corValCheck=corr.loc[col][col2]
                        if corValCheck < corVal:
                            pair1Lst.append(col);pair2Lst.append(col2)
                            pairLst.append((col,col2))
        pair1Set=list(set(pair1Lst));pair2Set=list(set(pair2Lst))
        aviRunLst=list(pair1Set)+list(pair2Set)
        inputs=aviRunLst
        outDict={}
        out=Parallel(n_jobs=7,backend="threading")(delayed(__parallelRunAvi)
                                                   (imm,immDf,nonDf,useFile=useFile,file=file,outDict=outDict,
                                                    tfs=tfs,place=place) for imm in inputs)
        print(len(outDict),len(pairLst))
        createOmniInputFile(outDict=outDict,pairLst=pairLst,nonDf=nonDf,place=place,corVal=corVal,
                            fullDict=fullDict,includeNumKinase=True,thres=thres)
    if runOmni:
        x=1
    return 1

In [26]:
def plotKinasePairsHM(directory='TF Anaylsis/Outputs/OmniPaths/10 Steps/Lung adenocarcinoma/*'):
    recivers=[]
    dfRec=pd.read_csv('OmniPath Ligands Transmitters and Receivers.csv',index_col=0)
    dfRec1=set(dfRec[dfRec['receiver']].index)
    dfRec2=set(dfRec[dfRec['transmitter']].index)
    keepList = list(set(dfRec1.union(dfRec2)))
    dfRec=dfRec.loc[keepList]
    recivers2=list(dfRec['genesymbol'].dropna())
    recivers2=list(set(recivers2))
    for i in recivers2:
        if ':' in i:
            lst = i.split(':')[1].split('_')
            recivers.extend(lst)
        else:
            recivers.append(i)

    files = glob.glob(directory)
    lungApairLst=[]
    for file in tq(files,desc='Finding Kinase Pairs'):
        genes=file.split('\\')[-1].replace('.xlsx','').split()
        try:
            df = pd.read_excel(file,index_col=0)
            if any(reciv in list(df['target_genesymbol']) for reciv in recivers):
                lungApairLst.append((genes[0],genes[1]))
        except:
            os.remove(file)
    dfLst=[]
    for folder in glob.glob('TF Anaylsis/Avi Results/Omni Inputs/*'):
        if not '~$' in folder:
            place = folder.split('\\')[-1].split(',')[0]
            try:
                dfPlace = pd.read_excel(folder,index_col=0)
                if len(dfPlace.index)>0:
                    dfLst.append((place,dfPlace))
            except:
                bad=1

    placeDict={}
    placeLst1=[]
    for place, dfPlace in tq(dfLst,desc='Counting # of Cancers'):
        sourceKinLst=[];targetKinLst=[]
        inputs = set(dfPlace['Connection'])
        place = place.replace('and ','')
        placeLst1.append(place)
        for connect in inputs:
            dfCliq=dfPlace[dfPlace['Connection']==connect]
            dfCliqSource = dfCliq[dfCliq['Source NonImmune Kinase Likely Immune']==False]
            try:
                dfCliqTarg = dfCliq[dfCliq['Target Immune Kinase Likely Immune']]
            except:
                dfCliqTarg = dfCliq[dfCliq['Target Immune Kinase Likely Immune']==1]
            sourceKin = list(dfCliqSource['Source NonImmune Kinases'].dropna())
            targetKin = list(dfCliqTarg['Target Immune Kinases'].dropna())

            sourceKinLst.extend(sourceKin);targetKinLst.extend(targetKin)
        sourceKinLst=set(sourceKinLst);targetKinLst=set(targetKinLst)
        for source,target in lungApairLst:
            if source in sourceKinLst and target in targetKinLst:val=1
            else:val=0
            if (source+' -> '+target) in placeDict:placeDict[(source+' -> '+target)].append(val)
            else:placeDict[(source+' -> '+target)]=[val]
    plotDf = pd.DataFrame.from_dict(placeDict,orient='index',columns=placeLst1)
    plotDf=plotDf.T

    c1Lst=[]
    for place in plotDf.index:
        if any(c in place for c in c1):
            c1Lst.append(place)
    c1Df= plotDf.loc[c1Lst]
    indxLst=list(c1Df.sum()[c1Df.sum() >= 2].index)
    plotAllC1=plotDf[indxLst]

    indxLst=list(plotDf.sum()[plotDf.sum() >= 12].index)
    plot12Can=plotDf[indxLst]

    indxLst=list(plotDf.sum()[plotDf.sum() >= 14].index)
    plot14Can=plotDf[indxLst]
    plotLst=[(plotDf,'All '+str(len(plotDf.columns))+' Kinase Pairs in Lung Aderno',0),
             (plot14Can,str(len(plot14Can.columns))+' Kinase Pairs in at least 14 Cancers',1),
             (plot12Can,str(len(plot12Can.columns))+' Kinase Pairs in at least 12 Cancers',1),
             (plotAllC1,str(len(plotAllC1.columns))+' Kinase Pairs in 2 C1 Cancers',1)]

    for df, title,xticks in tq(plotLst,desc='plotting'):
        if len(df.columns)>0:
            try:
                cg = sb.clustermap(df,cmap='YlGn',standard_scale=None,metric='cosine',vmin=0,vmax=1,
                                   yticklabels=1,xticklabels=xticks,row_cluster=False)
                plt.setp(cg.ax_heatmap.get_xticklabels(),size=6,rotation=45,ha='right')
                plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom')
                cg.ax_heatmap.set_ylabel('Cancers');cg.ax_heatmap.set_xlabel('Kinase Pairs')
                cg.fig.suptitle(title)
                plt.savefig(title+'.png')
                plt.show()
            except:
                print(df);tr=ty
    return plot12Can,plotAllC1,plotDf,plot14Can
def onlyInhibitors(df):
    cols = [i for i in df.columns if 'Inhibitor' in i]
    df2 = df[cols]
    keepLst=[]
    for i in df2.index:
        row=df2.loc[i]
        lst=list(row.values)
        for j in range(len(lst)):
            if '' in lst:
                lst.remove('')
        summ=0
        for elem in lst:
            if elem:
                summ+=1
                break
        if summ > 0:
            keepLst.append(i)
    dfOut = df.loc[keepLst]
    return dfOut
def createFullOmniCSV(lst):
    outDict={}
    name2=0
    for i in tq(lst,desc='Creating CSV'):
        g1,g2 = i.split(' -> ')
        fileName = 'TF Anaylsis/Outputs/OmniPaths/10 Steps/Lung adenocarcinoma/'+g1+' '+g2+'.xlsx'
        path = pd.read_excel(fileName,index_col=0)
        name2=int(name2);name2 +=1;name2=str(name2)
        paths=set(path['Path Group'])
        inputs = zip(paths,range(len(paths)))
        out = Parallel(n_jobs=7,backend="threading")(delayed(__superOmniCSVHelper)(path,group,outDict,name,name2,g2)
                                                     for group,name in inputs)
    fullCSV = getFullOmniDF(outDict)
    fullCSV = addinDBtoOmni(fullCSV)
    fullCSV = noRepeatsFullCSV(fullCSV)
    onlyInhib = onlyInhibitors(fullCSV)
    return fullCSV,onlyInhib
def getFullOmniDF(outDict):
    maxx=0
    for i in outDict:
        num = len(outDict[i])
        if num > maxx:
            maxx = num
    for i in outDict:
        lst = outDict[i]
        for n in range(maxx-len(lst)):
            lst.append('')
        outDict[i]=lst
    fullCSV = pd.DataFrame.from_dict(outDict, orient='index')
    numGenes = int(len(fullCSV.columns)/4)
    colLst=[]
    for i in range(numGenes):
        cols=['Kinase '+str(i),str(i)+' InterCell',str(i)+' Likely Immune',str(i)+' Inhibitor']
        colLst.extend(cols)
    fullCSV.columns = colLst
    fullCSV.index = range(len(fullCSV.index))
    return fullCSV

def noRepeatsFullCSV(fullCSV):
    geneCols=[]
    for i in fullCSV.columns:
        if 'Kinase' in i:
            geneCols.append(i)
    genes=fullCSV[geneCols]

    keepDict={}
    for i in genes.index:
        row=genes.loc[i]
        name=''
        for elem in row:
            if not elem is np.nan:
                name+=','+str(elem)
        keepDict[name]=i
    fullCSV = fullCSV.loc[list(keepDict.values())]
    fullCSV.index = range(len(fullCSV.index))
    return fullCSV

def getReciversLst():
    recivers=[];transmitters=[];both=[]
    dfRec=pd.read_csv('OmniPath Ligands Transmitters and Receivers.csv',index_col=0)
    dfRec1=set(dfRec[dfRec['receiver']].index)
    dfRec2=set(dfRec[dfRec['transmitter']].index)
    keepList = list(set(dfRec1.union(dfRec2)))
    dfRecB=dfRec.loc[keepList]
    recivers2=list(dfRecB['genesymbol'].dropna())
    recivers2=list(set(recivers2))
    for i in recivers2:
        if ':' in i:
            lst = i.split(':')[1].split('_')
            both.extend(lst)
        else:
            both.append(i)
    dfRec1 = dfRec.loc[dfRec1]
    recivers2=list(dfRec1['genesymbol'].dropna())
    recivers2=list(set(recivers2))
    for i in recivers2:
        if ':' in i:
            lst = i.split(':')[1].split('_')
            recivers.extend(lst)
        else:
            recivers.append(i)
    dfRec2 = dfRec.loc[dfRec2]        
    recivers2=list(dfRec2['genesymbol'].dropna())
    recivers2=list(set(recivers2))
    for i in recivers2:
        if ':' in i:
            lst = i.split(':')[1].split('_')
            transmitters.extend(lst)
        else:
            transmitters.append(i)
    return both,recivers,transmitters

def __superOmniCSVHelper(path,group,outDict,name,name2,g2):
    pathGroup = path[path['Path Group']==group]
    both,recivers,transmitters = getReciversLst()
    immuneLst=getImmuneLst()
    groupLst=[]
    last=False
    for i in pathGroup.index:
        row=pathGroup.loc[i]
        if row['target_genesymbol']==g2:
            last=True
        kinase = row['source_genesymbol']

        isYellow=''
        if kinase in transmitters:isYellow+= 'Is an InterCell Transmitter'
        if kinase in both:isYellow+=' and '
        if kinase in recivers:isYellow+= 'Is an InterCell Reciver'

        isImmune=False
        if kinase in immuneLst:
            isImmune=True
        inhibitor=row['is_inhibition'].value
        lst=[kinase,isYellow,isImmune,inhibitor]
        if last:
            isYellow=''
            if g2 in transmitters:isYellow+= 'Is an InterCell Transmitter'
            if g2 in recivers and g2 in transmitters:isYellow+=' and '
            if g2 in recivers:isYellow+= 'Is an InterCell Reciver'
            isImmune = g2 in immuneLst
            lst.extend([g2,isYellow,isImmune,False])
        groupLst.extend(lst)
        outDict[str(name2)+' '+str(name)+' '+str(i)]=groupLst
    return
def addinDBtoOmni(dfIn):
    df=dfIn.copy()
    useLst = ['c2.cp','h.all']
    fullDict = getFullDict(outputLst=False,useLst=useLst)

    geneDict={}
    for path in fullDict:
        lst=fullDict[path]
        for elem in lst:
            if elem in geneDict:
                geneDict[elem].append(path)
            else:
                geneDict[elem]=[path]
    cols =[i for i in df.columns if 'Kinase' in i]
    dfCols=df[cols]
    n=0
    n2=0
    odd=True
    lstLst=[];n3=0
    for col in dfCols.columns:
        n+=4;n+=n3;kLst=[]
        genes=list(dfCols[col])
        for g in genes:
            if g in geneDict:kLst.append(geneDict[g])
            else:kLst.append('')
        lstLst.append((n,kLst,n2))
        n2+=1
        if odd:n3=1
        else:n3=0
    for insert,lst,n2 in lstLst:df.insert(insert,str(n2)+' Databases',lst)
    return df

In [27]:
# example of colorLst input colorLst=[('set312',12),('pastel28',8)]
# a list of tuples, where the first element tells which color scheme you want to use, 
# and the 2nd element tells the length of unique colors in that color shceme
# color scheme names refer to these colors: http://graphviz.org/doc/info/colors.html
# 'Heat' and 'Purity' do not need to be included in immGenesDict or nonGenesDict
def getUseandColorDict(colorLst,immGenesDict={},nonGenesDict={},defaultDicts=True):
    if defaultDicts:
        immGenesDict,nonGenesDict,immuneGenes,nonGenes = getImmandNonDict()
    useDict={}
    dicLst=[immGenesDict,nonGenesDict]
    for dic in dicLst:
        for i in dic:
            elem = dic[i]
            i=i.replace('-','')
            useDict[i]=elem
    useDict['Heat']='Heat';useDict['Purity']='Purity'
    useDict['CancerArea']='Cancer Area'
    colorDict={}
    for elem in set(immGenesDict.values()):
        colorDict[elem]=(0,'red')
    n=1;colorNum=0
    colorTup=colorLst[colorNum]
    colorSchemeName,colorLen=colorTup
    for i in set(nonGenesDict.values()):
        colorDict[i]=(n,colorSchemeName)
        n+=1
        if n > colorLen:
            n=1
            colorNum+=1
            colorTup=colorLst[colorNum]
            colorSchemeName,colorLen=colorTup
    colorDict['Heat']=(0,'orangered');colorDict['Purity']=(0,'grey');colorDict['Cancer Area']=(0,'aliceblue')
    return useDict,colorDict

def nonPathsplotBNimmuneNonHMsHelper(dfLst,nonGenes,immuneGenes):
    pairDict={};frst=True
    for place,df in dfLst:
        for non in nonGenes:
            non2=non.replace('-','.')
            for imm in immuneGenes:
                df2=df[df['from']==non2] 
                df2=df2[df2['to']==imm]
                if len(df2.index)>0:
                    if df2['direction'].values[0]>= 0.35:
                        strength=df2['strength'].values[0]
                    else:
                        strength=0
                else:
                    strength=0
                if frst:
                    pairDict[non+' --> '+imm]=[strength]
                else:
                    pairDict[non+' --> '+imm].append(strength)
        frst=False
    return pairDict


def pathsPathsplotBNimmuneNonHMsHelperParallel(place,df,nonGenes,immuneGenes,thres,pathwaysDict,immGenesDict,
                                               nonGenesDict,grandparents):
    badPath = 'REACTOME_RESPIRATORY_ELECTRON_TRANSPORT_ATP_SYNTHESIS'
    badPath2='REACTOME_PRESYNAPTIC_DEPOLARIZATION_AND_CALCIUM'
    badPath3='WP_MITOCHONDRIAL_COMPLEX_I_ASSEMBLY_MODEL_OXPHOS'
    placeDict={}
    for non in nonGenes:
        non2=non.replace('-','.')
        for imm in immuneGenes:
            df2=df[df['from']==non2] 
            df2=df2[df2['to']==imm]
            if len(df2.index)>0:
                strength=df2['strength'].values[0];direction=df2['direction'].values[0]
            else:strength=0;direction=0
            immPath = immGenesDict[imm]
            try:
                nonPath = pathwaysDict[non]
            except:
                nonPath = nonGenesDict[non]
            if grandparents:
                fullPathLst=[]
                df3 = df[df['to']==non2]
                if len(df3.index)>0:
                    for ances in df3['from']:
                        ances2=ances.replace('.','-')
                        if ances2 in nonGenes:
                            df4=df3[df3['from']==ances]
                            strength3 = df4['strength'].values[0];direction3=df4['direction'].values[0]
                            try:
                                ancesPath = pathwaysDict[ances2]
                            except:
                                ancesPath = nonGenesDict[ances2]
                            
                            if badPath in ancesPath:
                                ancesPath = badPath
                            if badPath in nonPath:
                                nonPath = badPath
                            if badPath2 in ancesPath:
                                ancesPath = badPath2
                            if badPath2 in nonPath:
                                nonPath = badPath2
                            if badPath3 in ancesPath:
                                ancesPath = badPath3
                            if badPath3 in nonPath:
                                nonPath = badPath3
                                
                            fullPath = ancesPath+' --> '+nonPath+' --> '+immPath
                            fullPathLst.append((fullPath,strength,strength3,direction3))
            else:
                if badPath in nonPath:
                    nonPath = badPath
                if badPath2 in nonPath:
                    nonPath = badPath2
                if badPath3 in nonPath:
                    nonPath = badPath3
                fullPath=nonPath+' --> '+immPath
                fullPathLst=[(fullPath,stregnth,0,0)]
            for fullPath,stregnth1,strength2,direction2 in fullPathLst:
                if not fullPath in placeDict:
                    placeDict[fullPath]=0
                if strength >= thres and direction >= 0.35:
                    if not grandparents or (strength2 >= thres and direction2 >=0.35):
                        placeDict[fullPath]+=1
    return placeDict

def pathsPathsplotBNimmuneNonHMsHelper(dfLst,nonGenes,immuneGenes,thres,pathwaysDict,
                                       immGenesDict,nonGenesDict,grandparents,showProg):
    pairDict={};frst=True
    inputs=dfLst
    if showProg:
        inputs=tq(dfLst,desc='Finding Pairings')
    dictLst = Parallel(n_jobs=7)(delayed(pathsPathsplotBNimmuneNonHMsHelperParallel)
                                 (place,df,nonGenes,immuneGenes,thres,pathwaysDict,immGenesDict,
                                  nonGenesDict,grandparents)
                                 for place,df in inputs)
    for placeDict in dictLst:
        for pair in placeDict:
            if frst:
                pairDict[pair]=[placeDict[pair]]
            else:
                pairDict[pair].append(placeDict[pair])
        frst=False
    return pairDict

def plotBNimmuneNonHMs(files,title,showProg=False,thres=0,pathways=False,show=False,makeExcel=False,grandparents=False,
                       pruned=False,superPruned=False):
    df2 = pd.read_csv('Pathways4model.csv',index_col=0);df2=df2.dropna()
    useLst = ['c2.cp','c5.go.bp','h.all'];fullDict = getFullDict(outputLst=False,useLst=useLst);dict2={}
    pathwaysDict={}
    for i in df2.index:
        if i in fullDict:
            for gene in fullDict[i]:
                pathwaysDict[gene]=i
    immGenesDict,nonGenesDict,immuneGenes,nonGenes = getImmandNonDict()
    num=0;dfLst=[]
    for file in files:
        place=file.split(',')[1].strip();tpy=file.split(',')[-1].replace('.csv','')
        place+= ' '+tpy
        df=pd.read_csv(file,index_col=0)
        dfLst.append((place,df))
    pLst=[]
    for place,df in dfLst:
        place=place.replace(' HC','').replace('Cell Carcinoma','').replace('carcinoma','')
        pLst.append(place)
    if pathways:
        pairDict=pathsPathsplotBNimmuneNonHMsHelper(dfLst,nonGenes,immuneGenes,thres,pathwaysDict,
                                                    immGenesDict,nonGenesDict,grandparents,showProg)
    else:
        pairDict=nonPathsplotBNimmuneNonHMsHelper(dfLst,nonGenes,immuneGenes)
        
    dfAll = pd.DataFrame.from_dict(pairDict,orient='index',columns=pLst)
    dfAllKeep=dfAll.T
    num25thPercent=0.00001
    if pruned:
        l=list(dfAllKeep.sum())
        num25thPercent = sum(l)/(len(l)*0.75)
    if superPruned:
        l=list(dfAllKeep.sum())
        num25thPercent = sum(l)/(len(l)*0.5)
    keepLst=list(dfAllKeep.sum()[dfAllKeep.sum() >= num25thPercent].index)
    dfAll = dfAll.loc[keepLst]
    
    if pathways:
        nonGenesDict.update(pathwaysDict)
    nonPaths=[];immPaths=[]
    for genes in dfAll.index:
        gLst = genes.split(' --> ')
        if pathways:
            nonPath = gLst[0]
            immPath = gLst[1]
        else:
            nonPath=nonGenesDict[gLst[0]];immPath=immGenesDict[gLst[1]]
        nonPaths.append(nonPath)
        immPaths.append(immPath)
    nonPaths=set(nonPaths);immPaths=set(immPaths)
    colors2=['red','black','white']
    colors1=['blue','green','purple','brown','pink','gray','olive','cyan','gold','palegreen','yellow','fuchsia','teal']
    colors1.extend(['rosybrown','lightcorel','saddlebrown','goldenrod','darkgoldenrod','olivedrab','yellowgreen','turquoise'])
    colors1.extend(['darkseagreen','darkkhaki','lawngreen','peachpuff','sandybrown','darkolivegreen','burlywood','lime'])
    colors1.extend(['aquamarine','orchid'])
    colorsNonDict=dict(zip(nonPaths,colors1))
    colorsImmDict=dict(zip(immPaths,colors2))
    
    if grandparents:
        ancesPaths = set(list(nonPaths)+list(immPaths))
        colorsNonDict=dict(zip(ancesPaths,colors1))
        colorsImmDict=dict(zip(ancesPaths,colors1))
    
    nonColors=[];immColors=[]
    for genes in dfAll.index:
        gLst = genes.split(' --> ')
        if pathways:
            nonColor = colorsNonDict[gLst[0]]
            immColor = colorsImmDict[gLst[1]]
        else:
            nonColor=colorsNonDict[nonGenesDict[gLst[0]]];immColor=colorsImmDict[immGenesDict[gLst[1]]]
        nonColors.append(nonColor)
        immColors.append(immColor)
    colors=[nonColors,immColors]
    if not grandparents:
        colorsNonDict.update(colorsImmDict)
    if grandparents:
        title=title.replace('Pairs','Grandparent Tuples')
    if pathways:
        cmap='Greens'
    else:
        cmap='Oranges'
    try:
        cg = sb.clustermap(dfAll,cmap=cmap,standard_scale=None,metric='cosine',vmin=0,
                           yticklabels=0,xticklabels=1,row_colors=colors,cbar_pos=(1, 0.8, 0.05, 0.15))
    except:
        plt.close()
        cg = sb.clustermap(dfAll,cmap=cmap,standard_scale=None,metric='euclidean',vmin=0,
                           yticklabels=0,xticklabels=1,row_colors=colors,cbar_pos=(1, 0.8, 0.05, 0.15))
    cg.ax_col_dendrogram.set_visible(False)
    for x,y in zip(colorsNonDict.keys(),colorsNonDict.values()):
        cg.ax_heatmap.plot(0, 0, color=y, label=x, linewidth=6, solid_capstyle="butt")
    cg.ax_heatmap.legend(loc=(1.05,0.05), title="Pathway")
    plt.setp(cg.ax_heatmap.get_xticklabels(),size=7,rotation=45,ha='right')
    plt.setp(cg.ax_heatmap.get_yticklabels(),rotation=45,va='bottom')
    
    ylabel='NonImm Immune Pairs'
    if grandparents:
        ylabel=ylabel.replace('Pairs','Ancestor Tuples')
    title+=' Number of Pairs '+str(len(dfAll.index))
    cg.ax_heatmap.set_ylabel(ylabel);cg.ax_heatmap.set_xlabel('Cancers')
    cg.fig.suptitle(title,size=10,x=0.55);figure = plt.gcf()
    if pathways:
        location='Graphs/Confidence above '+str(thres)+'/'+title+'.png'
    else:
        location='Graphs/'+title+'.png'
    plt.savefig(location,bbox_inches="tight",dpi=100)
        
    if show:plt.show()
    else:plt.close()
    title2=title.replace('HM of ','')
    if pathways:
        location='Graphs/Confidence above '+str(thres)+'/'+title2+' CSV version.xlsx'
    else:
        location='Graphs/'+title2+' CSV version.xlsx'
    if makeExcel:dfAll.to_excel(location)
    return 1

In [28]:
def changeOmniInputFilesImmune(ciber=False,union=False,directory='TF Anaylsis/Avi Results/Omni Inputs/*Lung a*'):
    errors=''
    immLst = getImmuneLst(ciber=ciber,union=union)
    for file in glob.glob(directory):
        if not '$' in file:
            try:
                df= pd.read_excel(file,index_col=0)
                sourceLst=[];targetLst=[]
                for i in df['Source NonImmune Kinases']:
                    if i in immLst or i is np.nan:
                        sourceLst.append(True)
                    else:
                        sourceLst.append(False)
                for i in df['Target Immune Kinases']:
                    if i in immLst or i is np.nan:
                        targetLst.append(True)
                    else:
                        targetLst.append(False)
                df['Source NonImmune Kinase Likely Immune']=sourceLst
                df['Target Immune Kinase Likely Immune']=targetLst
                df.to_excel(file)
            except:
                errors+='Unable to open file '+str(file)+'\n'
    return df

In [29]:
kobePlaceLst1=kobePlaceLst(thres=30,cares=True)
# recommened palceLsts for most functions 
placeLst=kobePlaceLst(cares=False,thres=30)
placeLst2=kobePlaceLst(cares=False,thres=1)
placeLst3=kobePlaceLst(cares=False,thres=105)

c1=['Adrenocortical','Breast HR','Lung','Thyroid','Skin']
c1Lst=[]
for pl in placeLst:
    for elem in c1:
        if elem in pl:
            c1Lst.append(pl)
c1Lst=list(set(c1Lst))

In [30]:
def __superOmniCSVHelperPar(i,outDict,name2):
    g1,g2 = i.split(' ')
    fileName = 'TF Anaylsis/Outputs/OmniPaths/10 Steps/Lung adenocarcinoma/'+g1+' '+g2+'.xlsx'
    path = pd.read_excel(fileName,index_col=0)
    paths=set(path['Path Group'])
    inputs = zip(paths,range(len(paths)))
    out = Parallel(n_jobs=7,backend="threading")(delayed(__superOmniCSVHelper)(path,group,outDict,name,name2,g2)
                                                 for group,name in inputs)
    return out

In [31]:
for v in [8]:
    for file in tq(glob.glob('Purity/TPM/v'+str(v)+'/*')):
        place=file.split('_')[1]
        df=pd.read_csv(file,index_col=0,sep='\t')
        df,_,_ = __heatDF(df)
        df.to_excel('Share Data/V'+str(v)+' Data/'+place+'_tpm_V'+str(v)+'.xlsx')